In [1]:
import numpy as np
import astropy
import bz2
import pandas as pd
import requests
import glob
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy.io import fits
from astropy.table import Table
from PIL import Image
from photutils.isophote import EllipseGeometry
from photutils.aperture import EllipticalAperture
from photutils.isophote import Ellipse
from photutils.isophote import build_ellipse_model
from photutils.psf import subtract_psf
from photutils.psf import EPSFModel
from astroquery.sdss import SDSS
from astroquery.skyview import SkyView
from astropy.wcs import WCS
from astropy import coordinates as coords
from astropy import units as u
from astropy.nddata.utils import Cutout2D
import scipy
import scipy.ndimage as ndimage
import scipy.ndimage.filters as filters
from scipy import stats
import urllib.error as url_err
import itertools
import warnings
warnings.filterwarnings("ignore")

from pylab import figure, cm
from matplotlib.colors import LogNorm

from scipy.signal import convolve2d as conv2
from skimage import color, data, restoration

import pylab as pyl

from skimage.transform import resize

from astropy.coordinates import Angle

from sklearn.linear_model import LinearRegression

import time

from scipy.special import gammaincinv

KeyboardInterrupt: 

For small galaxies, allow the fitting algorithm to determine the center of the galaxy. Also put the thresholds for the fitting in the PA. Use maybe more databases (DES or DESI).

Get more checks like ellipticity between average angle difference, redshift dependance, magnitude or size cut. Look up other SDSS papers to see what other people have done, what cut-off value.


One more VLBI database that we could use: The US Naval observatory. Maybe they don't have PAs but with my firts attempt we could try to get those as well.


Remeber to get Jeff to email Eyleen from Baltimore (HST) and try to see if we can use some HST images as groundcheck. 

In [ ]:
import photutils
photutils.__version__

In [ ]:
import sys
print(sys.version)

In [ ]:
def f(init_params,args):
    #init_params = np.array([init_sma,init_eps,init_pa])
    try:
        geometry = EllipseGeometry(x0=n, y0=n, sma=init_params[0], eps=init_params[1], pa=init_params[2] * np.pi / 180.)
        aper = EllipticalAperture((geometry.x0, geometry.y0), geometry.sma,  geometry.sma * (1 - geometry.eps), geometry.pa)
        ellipse = Ellipse(image_zoom, geometry)
        isolist = ellipse.fit_image()
        model_image = build_ellipse_model(image_zoom.shape, isolist)
        
        Residuals = image_zoom - model_image
        Chi_squared = np.sum(abs((image_zoom - model_image)**2/image_zoom))
    except Exception as e:
        Chi_squared = 1e6
    
    return Chi_squared    

    

In [ ]:
### from astropy.io import fits
import numpy as np

def reconstructPSF(psFieldFilename, filt, row, col):

    filterIdx = 'ugriz'.index(filt) + 1
    psField = fits.open(psFieldFilename)
    pStruct = psField[filterIdx].data

    nrow_b = pStruct['nrow_b'][0]
    ncol_b = pStruct['ncol_b'][0]

    rnrow = pStruct['rnrow'][0]
    rncol = pStruct['rncol'][0]

    nb = nrow_b * ncol_b
    coeffs = np.zeros(nb.size, float)
    ecoeff = np.zeros(3, float)
    cmat = pStruct['c']

    rcs = 0.001
    for ii in range(0, nb.size):
        coeffs[ii] = (row * rcs)**(ii % nrow_b) * (col * rcs)**(ii / nrow_b)

    for jj in range(0, 3):
        for ii in range(0, nb.size):
            ecoeff[jj] = ecoeff[jj] + cmat[int(ii / nrow_b), ii % nrow_b, jj] * coeffs[ii]

    psf = pStruct['rrows'][0] * ecoeff[0] + \
        pStruct['rrows'][1] * ecoeff[1] + \
        pStruct['rrows'][2] * ecoeff[2]

    psf = np.reshape(psf, (rnrow, rncol))
    # psf = psf[10:40, 10:40]  # Trim non-zero regions.

    return psf

In [ ]:
import numpy as np
import astropy.io.fits as fits
import os
from astropy.stats import mad_std
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from lmfit.models import GaussianModel
from astropy.modeling import models, fitting
from scipy import signal


def twoD_Gaussian(xycoords, amplitude, x0, y0, sigma_x, sigma_y, theta, offset):
    x,y = xycoords
    x0 = float(x0)
    y0 = float(y0)    
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    g = offset + amplitude*np.exp( - (a*((x-x0)**2) + 2*b*(x-x0)*(y-y0) + c*((y-y0)**2)))
    return g.ravel()

def twoD_Gaussian_psf(xycoords, amplitude, x0, y0, sigma_x, sigma_y, theta, offset):
    x,y = xycoords
    x0 = float(x0)
    y0 = float(y0)    
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    g = offset + amplitude*np.exp( - (a*((x-x0)**2) + 2*b*(x-x0)*(y-y0) + c*((y-y0)**2)))
    g_psf = signal.fftconvolve(g, psf.ravel(), mode='same')
    return g_psf  #.ravel()


def fit(image):
    med = np.median(image)
    image = image-med
    #image = image[0,0,:,:]

    side_length = image.shape[0]
    galaxy_center = int(round(side_length/2))
    #galaxy_center = np.where(image == np.amax(image))
    #x0 = galaxy_center[1][0]
    #y0 = galaxy_center[0][0]
    x0 = galaxy_center
    y0 = galaxy_center
    #max_index = np.where(image >= np.max(image))
    #x0 = max_index[1] #Middle of X axis
    #y0 = max_index[0] #Middle of Y axis
    x = np.arange(0, image.shape[1], 1) #Stars at 0, increases by 1, goes to length of axis
    y = np.arange(0, image.shape[0], 1) #Stars at 0, increases by 1, goes to length of axis
    xx, yy = np.meshgrid(x, y) #creates a grid to plot the function over
    sigma = np.std(image) #The standard dev given in the Gaussian
    amp = np.max(image) #amplitude
    #xycoor,amplitude, x0, y0, sigma_x, sigma_y, theta, offset
    guess = [amp, x0, y0, 5, 5, np.pi, 0] #The initial guess for the gaussian fitting
    low = [0,0,0,2,2,0,0] #start of data array
    #Upper Bounds x0: length of x axis, y0: length of y axis, st dev: max value in image, amplitude: 2x the max value
    upper = [np.max(image)*2, image.shape[1], image.shape[0], image.shape[0] , image.shape[0], 2*np.pi, 1e-5] 
    bounds = [low, upper]

    params, pcov = curve_fit(twoD_Gaussian, (xx.ravel(), yy.ravel()), image.ravel(), p0 = guess, bounds = bounds) #optimal fit. Pcov is covariant matrix.

    return params,pcov

def fit_psf(image):
    med = np.median(image)
    image = image-med
    #image = image[0,0,:,:]

    side_length = image.shape[0]
    galaxy_center = int(round(side_length/2))
    #galaxy_center = np.where(image == np.amax(image))
    #x0 = galaxy_center[1][0]
    #y0 = galaxy_center[0][0]
    x0 = galaxy_center
    y0 = galaxy_center
    #max_index = np.where(image >= np.max(image))
    #x0 = max_index[1] #Middle of X axis
    #y0 = max_index[0] #Middle of Y axis
    x = np.arange(0, image.shape[1], 1) #Stars at 0, increases by 1, goes to length of axis
    y = np.arange(0, image.shape[0], 1) #Stars at 0, increases by 1, goes to length of axis
    xx, yy = np.meshgrid(x, y) #creates a grid to plot the function over
    sigma = np.std(image) #The standard dev given in the Gaussian
    amp = np.max(image) #amplitude
    #xycoor,amplitude, x0, y0, sigma_x, sigma_y, theta, offset
    guess = [amp, x0, y0, 5, 5, np.pi, 0] #The initial guess for the gaussian fitting
    low = [0,x0-1,y0-1,2,2,0,0] #start of data array
    #Upper Bounds x0: length of x axis, y0: length of y axis, st dev: max value in image, amplitude: 2x the max value
    upper = [np.max(image)*2, x0+1, y0+1, image.shape[0] , image.shape[0], 2*np.pi, 1e-5] 
    bounds = [low, upper]

    params, pcov = curve_fit(twoD_Gaussian_psf, (xx.ravel(), yy.ravel()), image.ravel(), p0 = guess, bounds = bounds) #optimal fit. Pcov is covariant matrix.

    return params,pcov

In [ ]:
# Create x and y indices
x = np.linspace(0, 51, 51)
y = np.linspace(0, 51, 51)
x, y = np.meshgrid(x, y)
amplitude = 10 
x0 = 25
y0 = 25
sigma_x = 20
sigma_y = 10
theta = 0*np.pi/180
offset = 0

#create data
data = twoD_Gaussian((x, y), amplitude, x0, y0, sigma_x, sigma_y, theta, offset)

# plot twoD_Gaussian data generated above
plt.figure()
plt.imshow(data.reshape(51, 51))
plt.colorbar()
plt.gca().invert_yaxis()
plt.show()

In [ ]:
#OCARS_NAME = []
#J2000_NAME = []
#RA = []
#DEC = []
#RA_ERR_sec = []
#DEC_ERR_arcsec = []
#RA_DEC_CORRELATION = []
#SEMIMAJOR_AXIS_OF_ERROR_ELLIPSE_mas = []
#CATALOG_CODE = []
#EPOCH_YEAR = []
#GAL_LON_dec = []
#GAL_LAT_dec = []
#Z_FROM_NED_OR_LITERATURE = []
#Z = [] 
#Z_FLAG = []
#Z_SIMBAD = []
#Z_SIMBAD_FLAG = []
#Z_SDSS = []
#OPTICAL_OR_NIR_MAGNITUDE = []
#BAND = []
#OBJECT_PRIMARY_TYPE = []
#OTHER_ID_AND_NOTES = []
#LINE_128 = []
#file = open("New_source_coords.txt", "r")
#list_of_lines = file.readlines()
#for i,line in enumerate(file.readlines()):
#    if line[0] == '#':
#        continue
#    else:
#        OCARS_NAME.append(line[0:8])
#        J2000_NAME.append(line[9:19])
#        RA.append(line[20:37].replace(' ',':'))
#        DEC.append(line[38:55].replace(' ',':'))
#        try:
#            RA_ERR_sec.append(float(line[56:66]))
#        except Exception:
#            RA_ERR_sec.append(np.nan)
#        try:    
#            DEC_ERR_arcsec.append(float(line[67:76]))
#        except Exception:
#            DEC_ERR_arcsec.append(np.nan)
#        try:    
#            RA_DEC_CORRELATION.append(float(line[77:84]))
#        except Exception:
#            RA_DEC_CORRELATION.append(np.nan)
#        try:    
#            SEMIMAJOR_AXIS_OF_ERROR_ELLIPSE_mas.append(float(line[85:93]))
#        except Exception:
#            SEMIMAJOR_AXIS_OF_ERROR_ELLIPSE_mas.append(np.nan)
#        CATALOG_CODE.append(line[94:96])
#        EPOCH_YEAR.append(line[97:104])
#        try:
#            GAL_LON_dec.append(float(line[105:112]))
#        except Exception:
#            GAL_LON_dec.append(np.nan)
#        try:
#            GAL_LAT_dec.append(float(line[113:120]))
#        except Exception:
#            GAL_LAT_dec.append(np.nan)
#        if line[121] == '*': 
#            Z_FROM_NED_OR_LITERATURE.append('LITERATURE')
#            Z.append(float(line[122:128]))
#        else:
#            try:
#                Z.append(float(line[122:128]))
#                Z_FROM_NED_OR_LITERATURE.append('NED (fiducial)')
#            except Exception:
#                Z.append(np.nan)
#                Z_FROM_NED_OR_LITERATURE.append(np.nan)   
#        LINE_128.append(line[128])
#        if line[128] == 'a':
#            Z_FLAG.append('Photometric')
#        if line[128] == 'b':
#            Z_FLAG.append('Unreliable (doubtful, imaging, estimated, ...)')
#        if line[128] == 'c':
#            Z_FLAG.append('Lower limit')
#        if line[128] == 'd':
#            Z_FLAG.append('Upper limit')
#        if line[128] == 'e':
#            Z_FLAG.append('Zero redshift assigned in OCARS for galactic objects')
#        if (line[128] == ' ') or (line[128] == '6'):
#            Z_FLAG.append('None')
#        try:
#            Z_SIMBAD.append(float(line[130]))
#        except Exception:
#            Z_SIMBAD.append(np.nan)
#        if line[136] == 'a':
#            Z_SIMBAD_FLAG.append('Photometric')
#        if line[136] == 'b':
#            Z_SIMBAD_FLAG.append('Unreliable (doubtful, imaging, estimated, ...)')
#        if line[136] == 'c':
#            Z_SIMBAD_FLAG.append('Lower limit')
#        if line[136] == 'd':
#            Z_SIMBAD_FLAG.append('Upper limit')
#        if line[136] == 'e':
#            Z_SIMBAD_FLAG.append('Zero redshift assigned in OCARS for galactic objects')
#        if line[136] == ' ':
#            Z_SIMBAD_FLAG.append('None')
#        try:
#            Z_SDSS.append(float(line[138:144]))
#        except Exception:
#            Z_SDSS.append(np.nan)
#        OPTICAL_OR_NIR_MAGNITUDE.append(line[145:149])
#        BAND.append(line[149])
#        OBJECT_PRIMARY_TYPE.append(line[151:153])
#        OTHER_ID_AND_NOTES.append(line[154:])
#
#
#file.close()

In [ ]:
#print(len(OCARS_NAME))
#print(len(J2000_NAME))
#print(len(RA))
#print(len(DEC))
#print(len(RA_ERR_sec))
#print(len(DEC_ERR_arcsec))
#print(len(RA_DEC_CORRELATION))
#print(len(SEMIMAJOR_AXIS_OF_ERROR_ELLIPSE_mas))
#print(len(CATALOG_CODE))
#print(len(EPOCH_YEAR))
#print(len(GAL_LON_dec))
#print(len(GAL_LAT_dec))
#print(len(Z_FROM_NED_OR_LITERATURE))
#print(len(Z))
#print(len(Z_FLAG))
#print(len(Z_SIMBAD))
#print(len(Z_SIMBAD_FLAG))
#print(len(Z_SDSS))
#print(len(OPTICAL_OR_NIR_MAGNITUDE))
#print(len(BAND))
#print(len(OBJECT_PRIMARY_TYPE))
#print(len(OTHER_ID_AND_NOTES))

In [ ]:
#test_elements = ['a','b', 'c', 'd', 'e', ' ']
#LINE_128 = np.array(LINE_128)
#mask = np.isin(LINE_128, test_elements)
#index = np.where(mask == False)[0]
#LINE_128[index][0]

In [ ]:
#Data = {'OCARS_NAME':OCARS_NAME,'J2000_NAME':J2000_NAME,'RA':RA,'DEC':DEC,
#'RA_ERR_sec':RA_ERR_sec,'DEC_ERR_arcsec':DEC_ERR_arcsec, 
#'RA_DEC_CORRELATION':RA_DEC_CORRELATION,
#'SEMIMAJOR_AXIS_OF_ERROR_ELLIPSE_mas':SEMIMAJOR_AXIS_OF_ERROR_ELLIPSE_mas,
#'CATALOG_CODE':CATALOG_CODE,'EPOCH_YEAR':EPOCH_YEAR,
#'GAL_LON_dec':GAL_LON_dec,'GAL_LAT_dec':GAL_LAT_dec,
#'Z_FROM_NED_OR_LITERATURE':Z_FROM_NED_OR_LITERATURE,'Z':Z,
#'Z_FLAG':Z_FLAG,'Z_SIMBAD':Z_SIMBAD,'Z_SIMBAD_FLAG':Z_SIMBAD_FLAG,
#'Z_SDSS':Z_SDSS,'OPTICAL_OR_NIR_MAGNITUDE':OPTICAL_OR_NIR_MAGNITUDE,
#'BAND':BAND,'OBJECT_PRIMARY_TYPE':OBJECT_PRIMARY_TYPE,
#'OTHER_ID_AND_NOTES':OTHER_ID_AND_NOTES}
    
#New_source_coords_csv = pd.DataFrame(Data)
#New_source_coords_csv

In [ ]:
#a = Angle('00:00:03.12449300',u.hourangle)
#a.deg

#b = Angle('-13:52:00.7581900',u.deg)
#b.deg

#sec = 0.345
#sec = format(sec,'.8f')
#string = '0 0 '+sec+' hours'
#a = Angle(string) 
#a.deg

#arcsec = 0.345
#a = Angle(arcsec, u.arcsec) 
#a.deg

#RA_ERR_deg = np.array(New_source_coords_csv['RA_ERR_deg'])
#DEC_ERR_deg = np.array(New_source_coords_csv['DEC_ERR_deg'])
#DEC_deg = np.array(New_source_coords_csv['DEC_deg'])
#PA_ERR_deg = np.sqrt((RA_ERR_deg*np.cos(DEC_deg*np.pi/180))**2+DEC_ERR_deg**2)

In [ ]:
#New_source_coords_csv = New_source_coords_csv.drop(['DEC_deg'], axis=1)
#New_source_coords_csv.insert(loc=10, column='SKY_PA_ERR_deg', value=PA_ERR_deg)

In [ ]:
New_source_coords_csv = pd.read_csv('New_source_coords_csv.csv')
New_source_coords_csv
#New_source_coords_csv.iloc[[128]]
#New_source_coords_csv.to_csv('New_source_coords_csv.csv',index=False)

There is 4 pairs of repeated names, but none of those are on our original list of VLBI PA angles, so we will not use them.

In [ ]:
#names = New_source_coords_csv['J2000_NAME']
#repeated_indexes_1 = np.where(names == 'J0008-2339')
#repeated_indexes_2 = np.where(names == 'J0744-1414')
#repeated_indexes_3 = np.where(names == 'J1300+1417')
#repeated_indexes_4 = np.where(names == 'J1559+2556')
#print(New_source_coords_csv['J2000_NAME'][repeated_indexes_1[0]],repeated_indexes_1[0])
#print(New_source_coords_csv['J2000_NAME'][repeated_indexes_2[0]],repeated_indexes_2[0])
#print(New_source_coords_csv['J2000_NAME'][repeated_indexes_3[0]],repeated_indexes_3[0])
#print(New_source_coords_csv['J2000_NAME'][repeated_indexes_4[0]],repeated_indexes_4[0])

In [ ]:
PA = pd.read_csv('jet_directions_table2.csv')
PA_values = PA['pa']
PA_names = PA['J2000']
VLBI_names = glob.glob('images\\J*')

In [ ]:
PA

In [ ]:
Source_coords_csv = pd.read_csv('Source_coords_csv.csv')
Source_coords_DEC = Source_coords_csv['DEC']
Source_coords_RA = Source_coords_csv['RA']
Source_coords_csv
#Source_coords_csv = Source_coords_csv.drop(['name_first_part','name_last_part','XMatch_nasa_dec_indexes','XMatch_nasa_ra_indexes','XMatch_dec_common','XMatch_ra_common'],axis=1) 
#Source_coords_csv.to_csv('Source_coords_csv.csv',index=False)

In [ ]:
plt.figure()
plt.hist(Source_coords_csv['Z'])
plt.xlabel('Z',fontsize=15)
plt.ylabel('Counts',fontsize=15)
plt.title('Redshift distribution',fontsize=20)
plt.show()

In [ ]:
SDSS_input = pd.read_csv('SDSS_input.csv')
min(SDSS_input.ra[(SDSS_input.ra > 0) & (SDSS_input.ra < 360)])
#SDSS_input['int_id'] = np.arange(9220)      # SDSS_input.drop(['J2000_NAME'],axis=1)
#SDSS_input.to_csv('SDSS_input.csv',index=False)

In [ ]:
SDSS_input1 = SDSS_input[0:1000]
SDSS_input2 = SDSS_input[1000:2000]
SDSS_input3 = SDSS_input[2000:3000]
SDSS_input4 = SDSS_input[3000:4000]
SDSS_input5 = SDSS_input[4000:5000]
SDSS_input6 = SDSS_input[5000:6000]
SDSS_input7 = SDSS_input[6000:7000]
SDSS_input8 = SDSS_input[7000:8000]
SDSS_input9 = SDSS_input[8000:9000]
SDSS_input10 = SDSS_input[9000:]


SDSS_input1.to_csv('SDSS_input1.csv',index=False)
SDSS_input2.to_csv('SDSS_input2.csv',index=False)
SDSS_input3.to_csv('SDSS_input3.csv',index=False)
SDSS_input4.to_csv('SDSS_input4.csv',index=False)
SDSS_input5.to_csv('SDSS_input5.csv',index=False)
SDSS_input6.to_csv('SDSS_input6.csv',index=False)
SDSS_input7.to_csv('SDSS_input7.csv',index=False)
SDSS_input8.to_csv('SDSS_input8.csv',index=False)
SDSS_input9.to_csv('SDSS_input9.csv',index=False)
SDSS_input10.to_csv('SDSS_input10.csv',index=False)

Some elements of the original data had the angles wrong. The DEC had the RA, and the RA had a wrong value that I changed for nan. Now the DEC are correct, but not the RA. After that I also updated the values of the angles with the new list.

In [ ]:
#indexes_new = Source_coords_csv['INDEX_NEW_TABLE']
#OCARS_NAME = []
#RA_ERR_sec = [] 
#DEC_ERR_arcsec = []
#RA_ERR_deg = [] 
#DEC_ERR_deg = []
#SKY_PA_ERR_deg = []
#RA_DEC_CORRELATION = []
#SEMIMAJOR_AXIS_OF_ERROR_ELLIPSE_mas = []
#CATALOG_CODE = []
#EPOCH_YEAR = []
#GAL_LON_deg = []
#GAL_LAT_deg = []
#Z_FROM_NED_OR_LITERATURE = []
#Z = []
#Z_FLAG = []
#Z_SIMBAD = []
#Z_SIMBAD_FLAG = [] 
#Z_SDSS = [] 
#OPTICAL_OR_NIR_MAGNITUDE = [] 
#BAND = []
#OBJECT_PRIMARY_TYPE = [] 
#OTHER_ID_AND_NOTES = []
#for i,index_new in enumerate(indexes_new):
#    if np.isnan(index_new):
#        OCARS_NAME.append(np.nan)
#        RA_ERR_sec.append(np.nan)
#        DEC_ERR_arcsec.append(np.nan)
#        RA_ERR_deg.append(np.nan) 
#        DEC_ERR_deg.append(np.nan)
#        SKY_PA_ERR_deg.append(np.nan)
#        RA_DEC_CORRELATION.append(np.nan)
#        SEMIMAJOR_AXIS_OF_ERROR_ELLIPSE_mas.append(np.nan)
#        CATALOG_CODE.append(np.nan)
#        EPOCH_YEAR.append(np.nan)
#        GAL_LON_deg.append(np.nan)
#        GAL_LAT_deg.append(np.nan)
#        Z_FROM_NED_OR_LITERATURE.append(np.nan)
#        Z.append(np.nan)
#        Z_FLAG.append(np.nan)
#        Z_SIMBAD.append(np.nan)
#        Z_SIMBAD_FLAG.append(np.nan)
#        Z_SDSS.append(np.nan)
#        OPTICAL_OR_NIR_MAGNITUDE.append(np.nan)
#        BAND.append(np.nan)
#        OBJECT_PRIMARY_TYPE.append(np.nan)
#        OTHER_ID_AND_NOTES.append(np.nan)
#        continue
#    else:
#        OCARS_NAME.append(New_source_coords_csv['OCARS_NAME'][index_new])
#        RA_ERR_sec.append(New_source_coords_csv['RA_ERR_sec'][index_new])
#        DEC_ERR_arcsec.append(New_source_coords_csv['DEC_ERR_arcsec'][index_new])
#        RA_ERR_deg.append(New_source_coords_csv['RA_ERR_deg'][index_new]) 
#        DEC_ERR_deg.append(New_source_coords_csv['DEC_ERR_deg'][index_new])
#        SKY_PA_ERR_deg.append(New_source_coords_csv['SKY_PA_ERR_deg'][index_new])
#        RA_DEC_CORRELATION.append(New_source_coords_csv['RA_DEC_CORRELATION'][index_new])
#        SEMIMAJOR_AXIS_OF_ERROR_ELLIPSE_mas.append(New_source_coords_csv['SEMIMAJOR_AXIS_OF_ERROR_ELLIPSE_mas'][index_new])
#        CATALOG_CODE.append(New_source_coords_csv['CATALOG_CODE'][index_new])
#        EPOCH_YEAR.append(New_source_coords_csv['EPOCH_YEAR'][index_new])
#        GAL_LON_deg.append(New_source_coords_csv['GAL_LON_deg'][index_new])
#        GAL_LAT_deg.append(New_source_coords_csv['GAL_LAT_deg'][index_new])
#        Z_FROM_NED_OR_LITERATURE.append(New_source_coords_csv['Z_FROM_NED_OR_LITERATURE'][index_new])
#        Z.append(New_source_coords_csv['Z'][index_new])
#        Z_FLAG.append(New_source_coords_csv['Z_FLAG'][index_new])
#        Z_SIMBAD.append(New_source_coords_csv['Z_SIMBAD'][index_new])
#        Z_SIMBAD_FLAG.append(New_source_coords_csv['Z_SIMBAD_FLAG'][index_new])
#        Z_SDSS.append(New_source_coords_csv['Z_SDSS'][index_new])
#        OPTICAL_OR_NIR_MAGNITUDE.append(New_source_coords_csv['OPTICAL_OR_NIR_MAGNITUDE'][index_new])
#        BAND.append(New_source_coords_csv['BAND'][index_new])
#        OBJECT_PRIMARY_TYPE.append(New_source_coords_csv['OBJECT_PRIMARY_TYPE'][index_new])
#        OTHER_ID_AND_NOTES.append(New_source_coords_csv['OTHER_ID_AND_NOTES'][index_new])
#
#Source_coords_csv.insert(loc=1, column='OCARS_NAME', value=OCARS_NAME)
#Source_coords_csv['RA_ERR_sec'] = RA_ERR_sec 
#Source_coords_csv['DEC_ERR_arcsec'] = DEC_ERR_arcsec
#Source_coords_csv['RA_ERR_deg'] = RA_ERR_deg
#Source_coords_csv['DEC_ERR_deg'] = DEC_ERR_deg
#Source_coords_csv['SKY_PA_ERR_deg'] = SKY_PA_ERR_deg
#Source_coords_csv['RA_DEC_CORRELATION'] = RA_DEC_CORRELATION
#Source_coords_csv['SEMIMAJOR_AXIS_OF_ERROR_ELLIPSE_mas'] = SEMIMAJOR_AXIS_OF_ERROR_ELLIPSE_mas
#Source_coords_csv['CATALOG_CODE'] = CATALOG_CODE 
#Source_coords_csv['EPOCH_YEAR'] = EPOCH_YEAR 
#Source_coords_csv['GAL_LON_deg'] = GAL_LON_deg
#Source_coords_csv['GAL_LAT_deg'] = GAL_LAT_deg
#Source_coords_csv['Z_FROM_NED_OR_LITERATURE'] = Z_FROM_NED_OR_LITERATURE
#Source_coords_csv['Z'] = Z
#Source_coords_csv['Z_FLAG'] = Z_FLAG
#Source_coords_csv['Z_SIMBAD'] = Z_SIMBAD
#Source_coords_csv['Z_SIMBAD_FLAG'] = Z_SIMBAD_FLAG
#Source_coords_csv['Z_SDSS'] = Z_SDSS 
#Source_coords_csv['OPTICAL_OR_NIR_MAGNITUDE'] = OPTICAL_OR_NIR_MAGNITUDE
#Source_coords_csv['BAND'] = BAND
#Source_coords_csv['OBJECT_PRIMARY_TYPE'] = OBJECT_PRIMARY_TYPE
#Source_coords_csv['OTHER_ID_AND_NOTES'] = OTHER_ID_AND_NOTES
#
#Source_coords_csv.to_csv('Source_coords_csv.csv',index=False)

In [ ]:
#SDSS_index_new = np.zeros(len(skymapper_csv['SDSS_index']))
#for i,e in enumerate(skymapper_csv['SDSS_index']):
        #if pd.isnull(e):
            #SDSS_index_new[i] = np.nan
            #continue
        #if len(e) > 6:
            #e = e[0:5]+e[-1]
        #SDSS_index_new[i] = float(e[1:-1])
#skymapper_csv['SDSS_index'] = SDSS_index_new
#skymapper_csv.to_csv('skymapper.csv',index=False)

In [ ]:
#indexes_Source_coords_csv_new = np.zeros(len(skymapper_csv['indexes_Source_coords_csv']))
#for i,e in enumerate(skymapper_csv['indexes_Source_coords_csv']):
        #if pd.isnull(e):
            #continue
        #if len(e) > 6:
            #e = e[0:5]+e[-1]
        #indexes_Source_coords_csv_new[i] = float(e[1:-1])
#skymapper_csv['indexes_Source_coords_csv'] = indexes_Source_coords_csv_new

In [ ]:
#indexes_Source_coords_csv_array = np.array(skymapper_csv['indexes_Source_coords_csv'])
#SDSS_index = []
#SDSS_present = np.zeros(len(indexes_Source_coords_csv_array))
#for i,ind in enumerate(indexes_Source_coords_csv_array):
    #if pd.isnull(ind):
        #SDSS_present[i] = np.nan
        #SDSS_index.append(np.nan)
    #else:
        #index = np.where(SDSS['Source_coords_index'] == ind)[0]
        #if len(index) == 0:
            #SDSS_present[i] = 0
            #SDSS_index.append(np.nan)
        #else:
            #SDSS_present[i] = 1
            #SDSS_index.append(index)
            
#SDSS_index = np.array(SDSS_index)            
#skymapper_csv['SDSS_index'] = SDSS_index
#skymapper_csv.to_csv('skymapper.csv',index=False)

In [ ]:
#types_SDSS = np.zeros(len(skymapper_csv['SDSS_index']))
#pos_angle_SDSS_gaussian = np.zeros(len(VLBI_source_matches_csv['SDSS_index']))
#BA_ratio_SDSS_gaussian = np.zeros(len(VLBI_source_matches_csv['SDSS_index']))
#BA_ratio_err_SDSS_gaussian = np.zeros(len(VLBI_source_matches_csv['SDSS_index']))
#a_SDSS_gaussian = np.zeros(len(VLBI_source_matches_csv['SDSS_index']))
#a_err_SDSS_gaussian = np.zeros(len(VLBI_source_matches_csv['SDSS_index']))
#b_SDSS_gaussian = np.zeros(len(VLBI_source_matches_csv['SDSS_index']))
#b_err_SDSS_gaussian = np.zeros(len(VLBI_source_matches_csv['SDSS_index']))
#PA_diff_SDSS_gaussian = np.zeros(len(VLBI_source_matches_csv['SDSS_index']))
#PA_diff_err_SDSS_gaussian = np.zeros(len(VLBI_source_matches_csv['SDSS_index']))

#for i,ind in enumerate(skymapper_csv['SDSS_index']):
    #if pd.isnull(ind):
        #types_SDSS[i] = np.nan
        #pos_angle_SDSS_gaussian[i] = np.nan
        #BA_ratio_SDSS_gaussian[i] = np.nan
        #BA_ratio_err_SDSS_gaussian[i] = np.nan
        #a_SDSS_gaussian[i] = np.nan
        #a_err_SDSS_gaussian[i] = np.nan
        #b_SDSS_gaussian[i] = np.nan
        #b_err_SDSS_gaussian[i] = np.nan
        #PA_diff_SDSS_gaussian[i] = np.nan
        #PA_diff_err_SDSS_gaussian[i] = np.nan
        #continue
    #name = SDSS['SDSS names'][ind]
    #if pd.isnull(name):
        #types_SDSS[i] = np.nan
        #pos_angle_SDSS_gaussian[i] = np.nan
        #BA_ratio_SDSS_gaussian[i] = np.nan
        #BA_ratio_err_SDSS_gaussian[i] = np.nan
        #a_SDSS_gaussian[i] = np.nan
        #a_err_SDSS_gaussian[i] = np.nan
        #b_SDSS_gaussian[i] = np.nan
        #b_err_SDSS_gaussian[i] = np.nan
        #PA_diff_SDSS_gaussian[i] = np.nan
        #PA_diff_err_SDSS_gaussian[i] = np.nan
        #continue
        
    #index = np.where(SDSS_new_results['Name'] == name)[0]
    #if len(index) == 0:
        #pos_angle_SDSS_gaussian[i] = np.nan
        #BA_ratio_SDSS_gaussian[i] = np.nan
        #BA_ratio_err_SDSS_gaussian[i] = np.nan
        #a_SDSS_gaussian[i] = np.nan
        #a_err_SDSS_gaussian[i] = np.nan
        #b_SDSS_gaussian[i] = np.nan
        #b_err_SDSS_gaussian[i] = np.nan
        #PA_diff_SDSS_gaussian[i] = np.nan
        #PA_diff_err_SDSS_gaussian[i] = np.nan
        #continue  
    #if len(index) > 1:
        #index = index[0]
    
    #types_SDSS[i] = SDSS['type'][ind] 
    
    #pos_angle = SDSS_new_results['pa_major'][index]
    #pos_angle_SDSS_gaussian[i] = pos_angle
    
    #BA_ratio = SDSS_new_results['b_a_list'][index]
    #BA_ratio_SDSS_gaussian[i] = BA_ratio
    
    #BA_ratio_err = SDSS_new_results['b_a_errs'][index]
    #BA_ratio_err_SDSS_gaussian[i] = BA_ratio_err
    
    #a = SDSS_new_results['a'][index]
    #a_SDSS_gaussian[i] = a
    
    #a_err = SDSS_new_results['a_err'][index]
    #a_err_SDSS_gaussian[i] = a_err
    
    #b = SDSS_new_results['b'][index]
    #b_SDSS_gaussian[i] = b
    
    #b_err = SDSS_new_results['b_err'][index]
    #b_err_SDSS_gaussian[i] = b_err
    
    #PA_diff = SDSS_new_results['jet_minor_difference'][index]
    #PA_diff_SDSS_gaussian[i] = PA_diff
    
    #PA_diff_err = SDSS_new_results['jet_minor_difference_err'][index]
    #PA_diff_err_SDSS_gaussian[i] = PA_diff_err
    
#VLBI_source_matches_csv['BA_ratio_SDSS_gaussian'] = BA_ratio_SDSS_gaussian
#VLBI_source_matches_csv['PA_SDSS_gaussian'] = PA_SDSS_gaussian
#VLBI_source_matches_csv['BA_ratio_err_SDSS_gaussian'] = BA_ratio_err_SDSS_gaussian
#VLBI_source_matches_csv['a_SDSS_gaussian'] = a_SDSS_gaussian
#VLBI_source_matches_csv['a_err_SDSS_gaussian'] = a_err_SDSS_gaussian
#VLBI_source_matches_csv['b_SDSS_gaussian'] = b_SDSS_gaussian
#VLBI_source_matches_csv['b_err_SDSS_gaussian'] = b_err_SDSS_gaussian
#VLBI_source_matches_csv['PA_diff_SDSS_gaussian'] = PA_diff_SDSS_gaussian
#VLBI_source_matches_csv['PA_diff_err_SDSS_gaussian'] = PA_diff_err_SDSS_gaussian
#skymapper_csv['type_SDSS'] = types_SDSS
#skymapper_csv.to_csv('skymapper.csv')

In [ ]:
#pa_difference_SDSS_gaussian = np.zeros(len(VLBI_source_matches_csv['SDSS_index']))
#for i,ind in enumerate(VLBI_source_matches_csv['SDSS_index']):
    #if pd.isnull(ind):
        #pa_difference_SDSS_gaussian[i] = np.nan
        #continue
        
    #name = SDSS['SDSS names'][ind]
    #if pd.isnull(name):
        #pa_difference_SDSS_gaussian[i] = np.nan
        #continue
        
    #index = np.where(SDSS_new_results['Name'] == name)[0]
    #if len(index) == 0:
        #pa_difference_SDSS_gaussian[i] = np.nan
        #continue  
    #if len(index) > 1:
        #index = index[0]
    #pa_difference = SDSS_new_results['jet_minor_difference'][index]
    #pa_difference_SDSS_gaussian[i] = pa_difference
#VLBI_source_matches_csv['pa_difference_SDSS_gaussian'] = pa_difference_SDSS_gaussian
#VLBI_source_matches_csv.to_csv('VLBI_source_matches.csv',index=False)

In [ ]:
#indexes = np.array(skymapper_csv.indexes_Source_coords_csv)
#unique = []
#for i,ind in enumerate(indexes):
    #if ind in unique:
        #indexes[i] = np.nan
    #else:
        #indexes[i] = ind
    

#PA_indexes_Source_coords_csv = np.zeros(len(indexes))
#for i,ind in enumerate(indexes):
    #if np.isnan(ind):
        #PA_indexes_Source_coords_csv[i] = np.nan
    #else:
        #PA_indexes_Source_coords_csv[i] = PA_values[ind]
        
    

#skymapper_csv['PA_indexes_Source_coords_csv'] = PA_indexes_Source_coords_csv

#This is the major angle PA I think so I add 90
#diff = abs(skymapper_csv.PA+90-skymapper_csv.PA_indexes_Source_coords_csv)
#for i,angle in enumerate(diff):
    #if 90 < angle <= 180:
        #angle = 180 - angle
    #if 180 < angle <= 270:
        #angle = angle - 180
    #if 270 < angle <= 360:
        #angle = 360 - angle
    #diff[i] = angle
    
#skymapper_csv['pa_difference'] = diff
#skymapper_csv.to_csv('skymapper.csv',index=False)

In [ ]:
SDSS = pd.read_csv('SDSS.csv')
SDSS

#len(SDSS.loc[SDSS.type == 3])
#[int(x) for x in SDSS.specObjID]

In [ ]:
SDSS_catalogue = pd.read_csv('SDSS_DR17_Gal_QSO_jasorey.csv',header=0)
SDSS_catalogue

In [ ]:
SDSS_new_results = pd.read_csv('SDSS_new_results.csv',header=0)
SDSS_new_results
#SDSS_new_results.to_csv('SDSS_new_results.csv',index=False)

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord

SDSS_RA,SDSS_DEC = np.array(SDSS_catalogue.ra),np.array(SDSS_catalogue.dec)

Source_coords_csv_RA = np.array(Source_coords_csv.RA_deg)
Source_coords_csv_DEC = np.array(Source_coords_csv.DEC_deg)
RA_bad = (((Source_coords_csv_RA > 0) & (Source_coords_csv_RA < 360)) == False) & (np.isnan(Source_coords_csv_RA) == True)
DEC_bad = (((Source_coords_csv_DEC > (-90)) & (Source_coords_csv_DEC < 90)) == False) & (np.isnan(Source_coords_csv_DEC) == True)
RA_good = (((Source_coords_csv_RA > 0) & (Source_coords_csv_RA < 360)) == True) & (np.isnan(Source_coords_csv_RA) == False)
DEC_good = (((Source_coords_csv_DEC > (-90)) & (Source_coords_csv_DEC < 90)) == True) & (np.isnan(Source_coords_csv_DEC) == False)

Source_coords_csv_RA_good = Source_coords_csv_RA[RA_good & DEC_good]
Source_coords_csv_DEC_good = Source_coords_csv_DEC[RA_good & DEC_good]

indexes_Source_coords_csv = np.arange(0,len(Source_coords_csv_RA))

indexes_Source_coords_csv_good = indexes_Source_coords_csv[RA_good & DEC_good]


SDSS_catalogue_coords = SkyCoord(SDSS_RA,SDSS_DEC,frame='icrs',unit='deg')
Astrogeo_coords_good = SkyCoord(Source_coords_csv_RA_good,Source_coords_csv_DEC_good,frame='icrs',unit='deg')
max_sep = 20 * u.arcsecond
idx, d2d, d3d = Astrogeo_coords_good.match_to_catalog_3d(SDSS_catalogue_coords)
sep_constraint = d2d < max_sep
Astrogeo_matches_good = Astrogeo_coords_good[sep_constraint]
SDSS_catalogue_Astrogeo_matches = SDSS_catalogue_coords[idx[sep_constraint]]


#SDSS_new_results_coords_good = SkyCoord(SDSS_new_results_RA_good,SDSS_new_results_DEC_good,frame='icrs',unit='deg')
#max_sep = 10 * u.arcsecond
#idx, d2d, d3d = SDSS_new_results_coords_good.match_to_catalog_3d(KIDS_catalogue)
#sep_constraint = d2d < max_sep
#SDSS_new_results_matches_good = SDSS_new_results_coords_good[sep_constraint]
#KIDS_catalogue_SDSS_new_results_matches = KIDS_catalogue[idx[sep_constraint]]

In [ ]:
plt.figure()
plt.scatter(Astrogeo_matches_good.ra.deg,Astrogeo_matches_good.dec.deg,s=1,c='b',label='Astrogeo')
plt.scatter(SDSS_catalogue_Astrogeo_matches.ra.deg,SDSS_catalogue_Astrogeo_matches.dec.deg,s=1,c='r',label='SDSS')
plt.xlabel('RA')
plt.ylabel('DEC')
plt.legend()
plt.title('SDSS and Astrogeo crossmatch for sep < 20 arcsec')
plt.savefig('Astrogeo_SDSS_match.png',dpi=900)
plt.show()

In [ ]:
len(SDSS_catalogue_Astrogeo_matches.ra.deg)

In [ ]:
#Hacerlo pero cambiar DES por SDSS_catalogue

Source_coords_csv_RA = np.array(Source_coords_csv.RA_deg)
Source_coords_csv_DEC = np.array(Source_coords_csv.DEC_deg)
RA_bad = (((Source_coords_csv_RA > 0) & (Source_coords_csv_RA < 360)) == False) & (np.isnan(Source_coords_csv_RA) == True)
DEC_bad = (((Source_coords_csv_DEC > (-90)) & (Source_coords_csv_DEC < 90)) == False) & (np.isnan(Source_coords_csv_DEC) == True)
RA_good = (((Source_coords_csv_RA > 0) & (Source_coords_csv_RA < 360)) == True) & (np.isnan(Source_coords_csv_RA) == False)
DEC_good = (((Source_coords_csv_DEC > (-90)) & (Source_coords_csv_DEC < 90)) == True) & (np.isnan(Source_coords_csv_DEC) == False)

Source_coords_csv_RA_good = Source_coords_csv_RA[RA_good & DEC_good]
Source_coords_csv_DEC_good = Source_coords_csv_DEC[RA_good & DEC_good]

indexes_Source_coords_csv = np.arange(0,len(Source_coords_csv_RA))

indexes_Source_coords_csv_good = indexes_Source_coords_csv[RA_good & DEC_good]

SDSS_new_results_RA = np.array(SDSS_new_results.ra)
SDSS_new_results_DEC = np.array(SDSS_new_results.dec)
SDSS_new_res_RA_bad = (((SDSS_new_results_RA > 0) & (SDSS_new_results_RA < 360)) == False) & (np.isnan(SDSS_new_results_RA) == True)
SDSS_new_res_DEC_bad = (((SDSS_new_results_DEC > (-90)) & (SDSS_new_results_DEC < 90)) == False) & (np.isnan(SDSS_new_results_DEC) == True)
SDSS_new_res_RA_good = (((SDSS_new_results_RA > 0) & (SDSS_new_results_RA < 360)) == True) & (np.isnan(SDSS_new_results_RA) == False)
SDSS_new_res_DEC_good = (((SDSS_new_results_DEC > (-90)) & (SDSS_new_results_DEC < 90)) == True) & (np.isnan(SDSS_new_results_DEC) == False)

SDSS_new_results_RA_good = SDSS_new_results_RA[SDSS_new_res_RA_good & SDSS_new_res_DEC_good]
SDSS_new_results_DEC_good = SDSS_new_results_DEC[SDSS_new_res_RA_good & SDSS_new_res_DEC_good]

indexes_SDSS_new_results = np.arange(0,len(SDSS_new_results_RA))
indexes_SDSS_new_results_good = indexes_SDSS_new_results[SDSS_new_res_RA_good & SDSS_new_res_DEC_good]


SDSS_catalogue_coords = SkyCoord(SDSS_RA,SDSS_DEC,frame='icrs',unit='deg')
SDSS_new_results_coords_good = SkyCoord(SDSS_new_results_RA_good,SDSS_new_results_DEC_good,frame='icrs',unit='deg')
max_sep = 5 * u.arcsecond
idx, d2d, d3d = SDSS_new_results_coords_good.match_to_catalog_3d(SDSS_catalogue_coords)
sep_constraint = d2d < max_sep
SDSS_new_results_matches_good = SDSS_new_results_coords_good[sep_constraint]
SDSS_catalogue_SDSS_matches = SDSS_catalogue_coords[idx[sep_constraint]]


plt.figure()
plt.scatter(SDSS_new_results_matches_good.ra.deg,SDSS_new_results_matches_good.dec.deg,s=1,c='b',label='SDSS')
plt.scatter(SDSS_catalogue_SDSS_matches.ra.deg,SDSS_catalogue_SDSS_matches.dec.deg,s=1,c='r',label='DES')
plt.xlabel('RA')
plt.ylabel('DEC')
plt.legend()
plt.title('SDSS and SDSS fits crossmatch for sep < {} arcsec'.format(max_sep.value))
plt.savefig('SDSS_SDSS_results_match.png',dpi=900)
plt.show()


SDSS_catalogue_SDSS_BA_matches = np.array(SDSS_catalogue.deVAB_g)[idx[sep_constraint]]
SDSS_catalogue_SDSS_THETA_J2000_matches = np.array(SDSS_catalogue.deVPhi_g)[idx[sep_constraint]]
SDSS_B_matches = SDSS_new_results.b[indexes_SDSS_new_results_good[sep_constraint]] 
SDSS_BA_matches = SDSS_new_results.b_a_list[indexes_SDSS_new_results_good[sep_constraint]]
SDSS_THETA_matches = SDSS_new_results.pa_major[indexes_SDSS_new_results_good[sep_constraint]]

plt.figure()
plt.plot(SDSS_catalogue_SDSS_BA_matches,SDSS_BA_matches,'b.')
plt.plot(SDSS_catalogue_SDSS_BA_matches[SDSS_B_matches > 0.5],SDSS_BA_matches[SDSS_B_matches > 0.5],'r.',label='own fit b > 0.5px')
plt.xlabel('b/a SDSS')
plt.ylabel('b/a own fit from SDSS')
plt.legend()
plt.savefig('SDSS_SDSS_results_BA.png',dpi=900)
plt.show()


SDSS_THETA_matches = np.fmod(SDSS_THETA_matches+360,180)
SDSS_THETA_matches[SDSS_THETA_matches > 90] = SDSS_THETA_matches - 180

#Convert west of north to east of north
#DES_SDSS_THETA_J2000_matches = -KIDS_SDSS_PAgaap_matches
#It's already counterclockwise but it's from x axis
#DES_SDSS_THETA_WORLD_matches = KIDS_SDSS_THETA_WORLD_matches-90


plt.figure()
plt.plot(SDSS_catalogue_SDSS_THETA_J2000_matches,SDSS_THETA_matches,'b.')
plt.xlabel('pa (deg) SDSS')
plt.ylabel('pa (deg) own fit from SDSS')
plt.savefig('SDSS_SDSS_results_PA.png',dpi=900)
plt.show()

In [ ]:
DES_catalogue = pd.read_csv('DES_data_DR2/DES_0000{}.csv.gz'.format('01'))
DES_catalogue

In [ ]:
numbers = ['0'+str(i) for i in range(1,10)] + [str(i) for i in range(10,56)]
DES_catalogue_RA = []
DES_catalogue_DEC = []
DES_catalogue_A = []
DES_catalogue_B = []
DES_catalogue_THETA_J2000 = []
DES_catalogue_THETAWIN_IMAGE_G = []
DES_catalogue_SPREAD_MODEL_G = []
for number in numbers[:5]:
    DES_catalogue = pd.read_csv('DES_data_DR2/DES_0000{}.csv.gz'.format(number))
    DES_catalogue_RA = DES_catalogue_RA + list(DES_catalogue.RA)
    DES_catalogue_DEC = DES_catalogue_DEC + list(DES_catalogue.DEC)
    DES_catalogue_A = DES_catalogue_A + list(DES_catalogue.A_IMAGE)
    DES_catalogue_B = DES_catalogue_B + list(DES_catalogue.B_IMAGE)
    DES_catalogue_THETA_J2000 = DES_catalogue_THETA_J2000 + list(DES_catalogue.THETA_J2000)
    DES_catalogue_THETAWIN_IMAGE_G = DES_catalogue_THETAWIN_IMAGE_G + list(DES_catalogue.THETAWIN_IMAGE_G)
    DES_catalogue_SPREAD_MODEL_G = DES_catalogue_SPREAD_MODEL_G + list(DES_catalogue.SPREAD_MODEL_G)
    print(number)

In [ ]:
Source_coords_csv_RA = np.array(Source_coords_csv.RA_deg)
Source_coords_csv_DEC = np.array(Source_coords_csv.DEC_deg)
RA_bad = (((Source_coords_csv_RA > 0) & (Source_coords_csv_RA < 360)) == False) & (np.isnan(Source_coords_csv_RA) == True)
DEC_bad = (((Source_coords_csv_DEC > (-90)) & (Source_coords_csv_DEC < 90)) == False) & (np.isnan(Source_coords_csv_DEC) == True)
RA_good = (((Source_coords_csv_RA > 0) & (Source_coords_csv_RA < 360)) == True) & (np.isnan(Source_coords_csv_RA) == False)
DEC_good = (((Source_coords_csv_DEC > (-90)) & (Source_coords_csv_DEC < 90)) == True) & (np.isnan(Source_coords_csv_DEC) == False)

Source_coords_csv_RA_good = Source_coords_csv_RA[RA_good & DEC_good]
Source_coords_csv_DEC_good = Source_coords_csv_DEC[RA_good & DEC_good]

indexes_Source_coords_csv = np.arange(0,len(Source_coords_csv_RA))

indexes_Source_coords_csv_good = indexes_Source_coords_csv[RA_good & DEC_good]

SDSS_new_results_RA = np.array(SDSS_new_results.ra)
SDSS_new_results_DEC = np.array(SDSS_new_results.dec)
SDSS_new_res_RA_bad = (((SDSS_new_results_RA > 0) & (SDSS_new_results_RA < 360)) == False) & (np.isnan(SDSS_new_results_RA) == True)
SDSS_new_res_DEC_bad = (((SDSS_new_results_DEC > (-90)) & (SDSS_new_results_DEC < 90)) == False) & (np.isnan(SDSS_new_results_DEC) == True)
SDSS_new_res_RA_good = (((SDSS_new_results_RA > 0) & (SDSS_new_results_RA < 360)) == True) & (np.isnan(SDSS_new_results_RA) == False)
SDSS_new_res_DEC_good = (((SDSS_new_results_DEC > (-90)) & (SDSS_new_results_DEC < 90)) == True) & (np.isnan(SDSS_new_results_DEC) == False)

SDSS_new_results_RA_good = SDSS_new_results_RA[SDSS_new_res_RA_good & SDSS_new_res_DEC_good]
SDSS_new_results_DEC_good = SDSS_new_results_DEC[SDSS_new_res_RA_good & SDSS_new_res_DEC_good]

indexes_SDSS_new_results = np.arange(0,len(SDSS_new_results_RA))
indexes_SDSS_new_results_good = indexes_SDSS_new_results[SDSS_new_res_RA_good & SDSS_new_res_DEC_good]


DES_catalogue = SkyCoord(DES_catalogue_RA,DES_catalogue_DEC,frame='icrs',unit='deg')
SDSS_new_results_coords_good = SkyCoord(SDSS_new_results_RA_good,SDSS_new_results_DEC_good,frame='icrs',unit='deg')
max_sep = 20 * u.arcsecond
idx, d2d, d3d = SDSS_new_results_coords_good.match_to_catalog_3d(DES_catalogue)
sep_constraint = d2d < max_sep
SDSS_new_results_matches_good = SDSS_new_results_coords_good[sep_constraint]
DES_catalogue_SDSS_matches = DES_catalogue[idx[sep_constraint]]

In [ ]:
plt.figure()
plt.scatter(SDSS_new_results_matches_good.ra.deg,SDSS_new_results_matches_good.dec.deg,s=1,c='b',label='SDSS')
plt.scatter(DES_catalogue_SDSS_matches.ra.deg,DES_catalogue_SDSS_matches.dec.deg,s=1,c='r',label='DES')
plt.xlabel('RA')
plt.ylabel('DEC')
plt.legend()
plt.title('DES and SDSS crossmatch for sep < 20 arcsec')
plt.savefig('DES_SDSS_match.png',dpi=900)
plt.show()

In [ ]:
len(DES_catalogue_SDSS_matches.ra.deg)

In [ ]:



DES_SDSS_A_matches = np.array(DES_catalogue_A)[idx[sep_constraint]]
DES_SDSS_B_matches = np.array(DES_catalogue_B)[idx[sep_constraint]]
DES_SDSS_BA_matches = DES_SDSS_B_matches/DES_SDSS_A_matches
DES_SDSS_THETA_J2000_matches = np.array(DES_catalogue_THETA_J2000)[idx[sep_constraint]]
DES_SDSS_THETAWIN_IMAGE_G_matches = np.array(DES_catalogue_THETAWIN_IMAGE_G)[idx[sep_constraint]]
SDSS_BA_matches = SDSS_new_results.b_a_list[indexes_SDSS_new_results_good[sep_constraint]]
SDSS_THETA_matches = SDSS_new_results.pa_major[indexes_SDSS_new_results_good[sep_constraint]]

plt.figure()
plt.plot(DES_SDSS_BA_matches,SDSS_BA_matches,'b.')
plt.xlabel('b/a DES')
plt.ylabel('b/a own from SDSS')
plt.savefig('DES_KIDS_BA.png',dpi=900)
plt.show()


SDSS_THETA_matches = np.fmod(SDSS_THETA_matches+360,180)
SDSS_THETA_matches[SDSS_THETA_matches > 90] = SDSS_THETA_matches - 180

#Convert west of north to east of north
#DES_SDSS_THETA_J2000_matches = -KIDS_SDSS_PAgaap_matches
#It's already counterclockwise but it's from x axis
#DES_SDSS_THETA_WORLD_matches = KIDS_SDSS_THETA_WORLD_matches-90


plt.figure()
plt.plot(DES_SDSS_THETA_J2000_matches,SDSS_THETA_matches,'b.')
plt.xlabel('pa (deg) DES')
plt.ylabel('pa (deg) own from SDSS')
plt.savefig('DES_SDSS_PA.png',dpi=900)
plt.show()

In [ ]:
#SDSS_new_results = pd.read_csv('SDSS_new_results.csv',header=0)
#types = []
#for Name in np.array(SDSS_new_results.Name):
    #index = np.where(np.array(SDSS['SDSS names']) == Name)[0]
    #if len(index)>1:
        #index = index[0]
    #ttype = np.array(SDSS.type)[index]
    #types.append(int(ttype))
#SDSS_new_results['type_SDSS'] = types
#SDSS_new_results.to_csv('SDSS_new_results.csv',index=False)

In [ ]:
VLBI_source_matches_csv = pd.read_csv('VLBI_source_matches.csv',header=0) 
#VLBI_source_matches_csv.insert(71, 'pos_angle_SDSS_gaussian', pos_angle_SDSS_gaussian)
#VLBI_source_matches_csv = VLBI_source_matches_csv.drop(labels=['Unnamed: 0','pa_difference_SDSS_gaussian'],axis=1)
VLBI_source_matches_csv
#VLBI_source_matches_csv.to_csv('VLBI_source_matches.csv',index=False)

In [ ]:
VLBI_source_matches_csv_SDSS = VLBI_source_matches_csv[VLBI_source_matches_csv.SDSS_present == 1.]

BA_ratio_filter = 0.9

plt.figure()
plt.plot(VLBI_source_matches_csv_SDSS.BA_ratio,VLBI_source_matches_csv_SDSS.BA_ratio_SDSS_gaussian,'b.')
plt.plot(VLBI_source_matches_csv_SDSS[(VLBI_source_matches_csv_SDSS.type_SDSS == 3) & (VLBI_source_matches_csv_SDSS.b_SDSS_gaussian > 0.5) & (VLBI_source_matches_csv_SDSS.BA_ratio < BA_ratio_filter) & (VLBI_source_matches_csv_SDSS.BA_ratio_SDSS_gaussian < BA_ratio_filter)].BA_ratio,VLBI_source_matches_csv_SDSS[(VLBI_source_matches_csv_SDSS.type_SDSS == 3) & (VLBI_source_matches_csv_SDSS.b_SDSS_gaussian > 0.5) & (VLBI_source_matches_csv_SDSS.BA_ratio < BA_ratio_filter) & (VLBI_source_matches_csv_SDSS.BA_ratio_SDSS_gaussian < BA_ratio_filter)].BA_ratio_SDSS_gaussian,'r.',label='"Galaxy" type, own b > 0.5 px and b/a < {:.1f} for both'.format(BA_ratio_filter))
plt.xlabel('b/a from DESI',fontsize=15)
plt.ylabel('b/a from my own fit to SDSS',fontsize=15)
plt.title('DESI all sources in common with SDSS',fontsize=20)
plt.legend()
plt.show() 


VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian[VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian < 0] = VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian[VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian < 0] + 360
VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian[(90 < VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian) & (VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian <= 180)] = VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian[(90 < VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian) & (VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian <= 180)] - 180
VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian[(180 < VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian) & (VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian <= 270)] = VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian[(180 < VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian) & (VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian <= 270)] - 180
VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian[(270 < VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian) & (VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian <= 360)] = VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian[(270 < VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian) & (VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian <= 360)] - 360


plt.figure()
plt.plot(VLBI_source_matches_csv_SDSS.pos_angle,VLBI_source_matches_csv_SDSS.pos_angle_SDSS_gaussian,'b.')
plt.plot(VLBI_source_matches_csv_SDSS[(VLBI_source_matches_csv_SDSS.type_SDSS == 3) & (VLBI_source_matches_csv_SDSS.b_SDSS_gaussian > 0.5) & (VLBI_source_matches_csv_SDSS.BA_ratio < BA_ratio_filter) & (VLBI_source_matches_csv_SDSS.BA_ratio_SDSS_gaussian < BA_ratio_filter)].pos_angle,VLBI_source_matches_csv_SDSS[(VLBI_source_matches_csv_SDSS.type_SDSS == 3) & (VLBI_source_matches_csv_SDSS.b_SDSS_gaussian > 0.5) & (VLBI_source_matches_csv_SDSS.BA_ratio < BA_ratio_filter) & (VLBI_source_matches_csv_SDSS.BA_ratio_SDSS_gaussian < BA_ratio_filter)].pos_angle_SDSS_gaussian,'r.',label='"Galaxy" type, own b > 0.5 px and b/a < {:.1f} for both'.format(BA_ratio_filter))
plt.xlabel('Major PA from DESI',fontsize=15)
plt.ylabel('Major PA from my own fit to SDSS',fontsize=15)
plt.title('DESI all sources in common with SDSS',fontsize=20)
plt.legend()
plt.show()



VLBI_source_matches_csv_SER_SDSS = VLBI_source_matches_csv[(VLBI_source_matches_csv.TYPE == 'SER') & (VLBI_source_matches_csv.SDSS_present == 1.)]
plt.figure()
plt.plot(VLBI_source_matches_csv_SER_SDSS.BA_ratio,VLBI_source_matches_csv_SER_SDSS.BA_ratio_SDSS_gaussian,'b.')
plt.plot(VLBI_source_matches_csv_SER_SDSS[(VLBI_source_matches_csv_SDSS.type_SDSS == 3) & (VLBI_source_matches_csv_SDSS.b_SDSS_gaussian > 0.5) & (VLBI_source_matches_csv_SDSS.BA_ratio < BA_ratio_filter) & (VLBI_source_matches_csv_SDSS.BA_ratio_SDSS_gaussian < BA_ratio_filter)].BA_ratio,VLBI_source_matches_csv_SER_SDSS[(VLBI_source_matches_csv_SDSS.type_SDSS == 3) & (VLBI_source_matches_csv_SDSS.b_SDSS_gaussian > 0.5) & (VLBI_source_matches_csv_SDSS.BA_ratio < BA_ratio_filter) & (VLBI_source_matches_csv_SDSS.BA_ratio_SDSS_gaussian < BA_ratio_filter)].BA_ratio_SDSS_gaussian,'r.',label='"Galaxy" type, own b fit > 0.5 px, b/a < {:.1f} for both'.format(BA_ratio_filter))
plt.xlabel('b/a from DESI',fontsize=15)
plt.ylabel('b/a from my own fit to SDSS',fontsize=15)
plt.title('DESI type Sérsic sources in common with SDSS',fontsize=20)
plt.legend()
plt.show() 





VLBI_source_matches_csv_SER_SDSS = VLBI_source_matches_csv[(VLBI_source_matches_csv.TYPE == 'SER') & (VLBI_source_matches_csv.SDSS_present == 1.)]

VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian = np.fmod(VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian+360,180)
VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian[VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian > 90] = VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian[VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian > 90] - 180


Rho = np.corrcoef(VLBI_source_matches_csv_SER_SDSS.pos_angle, VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian)
Pearson_rho = Rho[0,1]

plt.figure()
plt.plot(VLBI_source_matches_csv_SER_SDSS.pos_angle,VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian,'b.')
plt.plot(VLBI_source_matches_csv_SER_SDSS[(VLBI_source_matches_csv_SDSS.type_SDSS == 3) & (VLBI_source_matches_csv_SDSS.b_SDSS_gaussian > 0.5) & (VLBI_source_matches_csv_SDSS.BA_ratio < BA_ratio_filter) & (VLBI_source_matches_csv_SDSS.BA_ratio_SDSS_gaussian < BA_ratio_filter)].pos_angle,VLBI_source_matches_csv_SER_SDSS[(VLBI_source_matches_csv_SDSS.type_SDSS == 3) & (VLBI_source_matches_csv_SDSS.b_SDSS_gaussian > 0.5) & (VLBI_source_matches_csv_SDSS.BA_ratio < BA_ratio_filter) & (VLBI_source_matches_csv_SDSS.BA_ratio_SDSS_gaussian < BA_ratio_filter)].pos_angle_SDSS_gaussian,'r.',label='"Galaxy" type, own b > 0.5 px, b/a < {:.1f} for both'.format(BA_ratio_filter))
plt.xlabel('Major PA from DESI',fontsize=15)
plt.ylabel('Major PA from my own fit to SDSS',fontsize=15)
plt.title('DESI type Sérsic sources in common with SDSS',fontsize=20)
plt.legend()
plt.show()

print('Pearson corr. coefficient is {:.2f}'.format(Pearson_rho))


VLBI_source_matches_csv_SER_SDSS = VLBI_source_matches_csv[(VLBI_source_matches_csv.TYPE == 'SER') & (VLBI_source_matches_csv.SDSS_present == 1.)]
plt.figure()
plt.plot(VLBI_source_matches_csv_SER_SDSS.BA_ratio,VLBI_source_matches_csv_SER_SDSS.BA_ratio_SDSS_gaussian,'b.')
plt.plot(VLBI_source_matches_csv_SER_SDSS[(VLBI_source_matches_csv_SDSS.type_SDSS == 3) & (VLBI_source_matches_csv_SDSS.b_SDSS_gaussian > 0.5) & (VLBI_source_matches_csv_SDSS.BA_ratio < BA_ratio_filter) & (VLBI_source_matches_csv_SDSS.BA_ratio_SDSS_gaussian < BA_ratio_filter) & (VLBI_source_matches_csv_SER_SDSS.DEC < 30)].BA_ratio,VLBI_source_matches_csv_SER_SDSS[(VLBI_source_matches_csv_SDSS.type_SDSS == 3) & (VLBI_source_matches_csv_SDSS.b_SDSS_gaussian > 0.5) & (VLBI_source_matches_csv_SDSS.BA_ratio < BA_ratio_filter) & (VLBI_source_matches_csv_SDSS.BA_ratio_SDSS_gaussian < BA_ratio_filter) & (VLBI_source_matches_csv_SER_SDSS.DEC < 30)].BA_ratio_SDSS_gaussian,'r.',label='"Galaxy" type, own b fit > 0.5 px, b/a < {:.1f} and DEC < {} for both'.format(BA_ratio_filter,30))
plt.plot(VLBI_source_matches_csv_SER_SDSS[(VLBI_source_matches_csv_SDSS.type_SDSS == 3) & (VLBI_source_matches_csv_SDSS.b_SDSS_gaussian > 0.5) & (VLBI_source_matches_csv_SDSS.BA_ratio < BA_ratio_filter) & (VLBI_source_matches_csv_SDSS.BA_ratio_SDSS_gaussian < BA_ratio_filter) & (VLBI_source_matches_csv_SER_SDSS.DEC > 30)].BA_ratio,VLBI_source_matches_csv_SER_SDSS[(VLBI_source_matches_csv_SDSS.type_SDSS == 3) & (VLBI_source_matches_csv_SDSS.b_SDSS_gaussian > 0.5) & (VLBI_source_matches_csv_SDSS.BA_ratio < BA_ratio_filter) & (VLBI_source_matches_csv_SDSS.BA_ratio_SDSS_gaussian < BA_ratio_filter) & (VLBI_source_matches_csv_SER_SDSS.DEC > 30)].BA_ratio_SDSS_gaussian,'k.',label='"Galaxy" type, own b fit > 0.5 px, b/a < {:.1f} and DEC > {} for both'.format(BA_ratio_filter,30))
plt.xlabel('b/a from DESI',fontsize=15)
plt.ylabel('b/a from my own fit to SDSS',fontsize=15)
plt.title('DESI type Sérsic sources in common with SDSS',fontsize=20)
plt.legend()
plt.show() 





VLBI_source_matches_csv_SER_SDSS = VLBI_source_matches_csv[(VLBI_source_matches_csv.TYPE == 'SER') & (VLBI_source_matches_csv.SDSS_present == 1.)]

VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian = np.fmod(VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian+360,180)
VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian[VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian > 90] = VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian[VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian > 90] - 180


Rho = np.corrcoef(VLBI_source_matches_csv_SER_SDSS.pos_angle, VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian)
Pearson_rho = Rho[0,1]

plt.figure()
plt.plot(VLBI_source_matches_csv_SER_SDSS.pos_angle,VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian,'b.')
plt.plot(VLBI_source_matches_csv_SER_SDSS[(VLBI_source_matches_csv_SDSS.type_SDSS == 3) & (VLBI_source_matches_csv_SDSS.b_SDSS_gaussian > 0.5) & (VLBI_source_matches_csv_SDSS.BA_ratio < BA_ratio_filter) & (VLBI_source_matches_csv_SDSS.BA_ratio_SDSS_gaussian < BA_ratio_filter) & (VLBI_source_matches_csv_SDSS.DEC < 30)].pos_angle,VLBI_source_matches_csv_SER_SDSS[(VLBI_source_matches_csv_SDSS.type_SDSS == 3) & (VLBI_source_matches_csv_SDSS.b_SDSS_gaussian > 0.5) & (VLBI_source_matches_csv_SDSS.BA_ratio < BA_ratio_filter) & (VLBI_source_matches_csv_SDSS.BA_ratio_SDSS_gaussian < BA_ratio_filter) & (VLBI_source_matches_csv_SDSS.DEC < 30)].pos_angle_SDSS_gaussian,'r.',label='"Galaxy" type, own b > 0.5 px, b/a < {:.1f} and DEC < {} for both'.format(BA_ratio_filter,30))
plt.plot(VLBI_source_matches_csv_SER_SDSS[(VLBI_source_matches_csv_SDSS.type_SDSS == 3) & (VLBI_source_matches_csv_SDSS.b_SDSS_gaussian > 0.5) & (VLBI_source_matches_csv_SDSS.BA_ratio < BA_ratio_filter) & (VLBI_source_matches_csv_SDSS.BA_ratio_SDSS_gaussian < BA_ratio_filter) & (VLBI_source_matches_csv_SDSS.DEC > 30)].pos_angle,VLBI_source_matches_csv_SER_SDSS[(VLBI_source_matches_csv_SDSS.type_SDSS == 3) & (VLBI_source_matches_csv_SDSS.b_SDSS_gaussian > 0.5) & (VLBI_source_matches_csv_SDSS.BA_ratio < BA_ratio_filter) & (VLBI_source_matches_csv_SDSS.BA_ratio_SDSS_gaussian < BA_ratio_filter) & (VLBI_source_matches_csv_SDSS.DEC > 30)].pos_angle_SDSS_gaussian,'k.',label='"Galaxy" type, own b > 0.5 px, b/a < {:.1f} and DEC > {} for both'.format(BA_ratio_filter,30))
plt.xlabel('Major PA from DESI',fontsize=15)
plt.ylabel('Major PA from my own fit to SDSS',fontsize=15)
plt.title('DESI type Sérsic sources in common with SDSS',fontsize=20)
plt.legend()
plt.show()

print('Pearson corr. coefficient is {:.2f}'.format(Pearson_rho))




VLBI_source_matches_csv_SER_SDSS = VLBI_source_matches_csv[(VLBI_source_matches_csv.TYPE == 'SER') & (VLBI_source_matches_csv.SDSS_present == 1.)]
plt.figure()
plt.plot(VLBI_source_matches_csv_SER_SDSS.BA_ratio,VLBI_source_matches_csv_SER_SDSS.BA_ratio_SDSS_gaussian,'b.')
plt.plot(VLBI_source_matches_csv_SER_SDSS[(VLBI_source_matches_csv_SDSS.type_SDSS == 3) & (VLBI_source_matches_csv_SDSS.DEC < 30)].BA_ratio,VLBI_source_matches_csv_SER_SDSS[(VLBI_source_matches_csv_SDSS.type_SDSS == 3) & (VLBI_source_matches_csv_SDSS.DEC < 30)].BA_ratio_SDSS_gaussian,'r.',label='"Galaxy" type and DEC < {} for both'.format(30))
plt.xlabel('b/a from DESI',fontsize=15)
plt.ylabel('b/a from my own fit to SDSS',fontsize=15)
plt.title('DESI type Sérsic sources in common with SDSS',fontsize=20)
plt.legend()
plt.show() 





VLBI_source_matches_csv_SER_SDSS = VLBI_source_matches_csv[(VLBI_source_matches_csv.TYPE == 'SER') & (VLBI_source_matches_csv.SDSS_present == 1.)]

VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian = np.fmod(VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian+360,180)
VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian[VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian > 90] = VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian[VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian > 90] - 180


Rho = np.corrcoef(VLBI_source_matches_csv_SER_SDSS.pos_angle, VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian)
Pearson_rho = Rho[0,1]

plt.figure()
plt.plot(VLBI_source_matches_csv_SER_SDSS.pos_angle,VLBI_source_matches_csv_SER_SDSS.pos_angle_SDSS_gaussian,'b.')
plt.plot(VLBI_source_matches_csv_SER_SDSS[(VLBI_source_matches_csv_SDSS.type_SDSS == 3) & (VLBI_source_matches_csv_SDSS.DEC < 30)].pos_angle,VLBI_source_matches_csv_SER_SDSS[(VLBI_source_matches_csv_SDSS.type_SDSS == 3) & (VLBI_source_matches_csv_SDSS.DEC < 30)].pos_angle_SDSS_gaussian,'r.',label='"Galaxy" type and DEC < {} for both'.format(30))
plt.xlabel('Major PA from DESI',fontsize=15)
plt.ylabel('Major PA from my own fit to SDSS',fontsize=15)
plt.title('DESI type Sérsic sources in common with SDSS',fontsize=20)
plt.legend()
plt.show()

print('Pearson corr. coefficient is {:.2f}'.format(Pearson_rho))

Check some examples of red dots that don't work

Use SHAPE_R (Reff), SHAPE_R_IVAR (inverse var of Reff), SHAPE_E1, SHAPE_E2 (e1 and e1, not same as e ), SERSIC (n), SERSIC_IVAR (inv var of n) pos_angle (theta), pos_angle_err (theta_err), BA_ratio (b/a), BA_ratio_err (b/a_err). For the pos_angle, take the different reference system into account.

In [ ]:
#skymapper_fits = fits.open('skymapper.fits')
#skymapper_fits[1].header

In [ ]:
skymapper_csv = pd.read_csv('skymapper.csv',header=0)
skymapper_csv
#1337 - np.sum(np.isnan(skymapper_csv.SDSS_index))
#skymapper_csv.to_csv('skymapper.csv',index=False)

In [ ]:
skymapper_csv_SDSS = skymapper_csv[np.isnan(skymapper_csv.SDSS_index) != 1.]

i = [11,25]

BA_ratio_i = skymapper_csv_SDSS[(skymapper_csv_SDSS.type_SDSS == 3) & (skymapper_csv_SDSS.b_SDSS_gaussian > 0.5) & (skymapper_csv_SDSS.b > 0.5)].iloc[i].BA_ratio
BA_ratio_SDSS_gaussian_i = skymapper_csv_SDSS[(skymapper_csv_SDSS.type_SDSS == 3) & (skymapper_csv_SDSS.b_SDSS_gaussian > 0.5) & (skymapper_csv_SDSS.b > 0.5)].iloc[i].BA_ratio_SDSS_gaussian

BA_ratio_filter = 1
stellar_index_filter = 0.5 

plt.figure()
plt.plot(skymapper_csv_SDSS.BA_ratio,skymapper_csv_SDSS.BA_ratio_SDSS_gaussian,'b.')
#plt.plot(skymapper_csv_SDSS[skymapper_csv_SDSS.b_SDSS_gaussian > 0.5].BA_ratio,skymapper_csv_SDSS[skymapper_csv_SDSS.b_SDSS_gaussian > 0.5].BA_ratio_SDSS_gaussian,'r.',label='own b > 0.5 px')
plt.plot(skymapper_csv_SDSS[(skymapper_csv_SDSS.type_SDSS == 3) & (skymapper_csv_SDSS.ClassStar < stellar_index_filter) & (skymapper_csv_SDSS.b_SDSS_gaussian > 0.5) & (skymapper_csv_SDSS.b > 0.5) & (skymapper_csv_SDSS.BA_ratio < BA_ratio_filter) & (skymapper_csv_SDSS.BA_ratio_SDSS_gaussian < BA_ratio_filter)].BA_ratio,skymapper_csv_SDSS[(skymapper_csv_SDSS.type_SDSS == 3) & (skymapper_csv_SDSS.ClassStar < stellar_index_filter) & (skymapper_csv_SDSS.b_SDSS_gaussian > 0.5) & (skymapper_csv_SDSS.b > 0.5) & (skymapper_csv_SDSS.BA_ratio < BA_ratio_filter) & (skymapper_csv_SDSS.BA_ratio_SDSS_gaussian < BA_ratio_filter)].BA_ratio_SDSS_gaussian,'r.',label='SDSS "Galaxy" type, Skymapper stellar index < {}, b > 0.5 px and b/a < {:.2f} for both'.format(stellar_index_filter,BA_ratio_filter))
#plt.plot(BA_ratio_i,BA_ratio_SDSS_gaussian_i,'ks',label='Choices')
plt.xlabel('b/a from skymapper',fontsize=15)
plt.ylabel('b/a from my own fit to SDSS',fontsize=15)
plt.title('Skymapper all sources in common with SDSS',fontsize=20)
plt.legend()
plt.show() 

skymapper_csv_SDSS.PA_SDSS_gaussian = np.fmod(skymapper_csv_SDSS.PA_SDSS_gaussian+360,180)
skymapper_csv_SDSS.PA_SDSS_gaussian[skymapper_csv_SDSS.PA_SDSS_gaussian > 90] = skymapper_csv_SDSS.PA_SDSS_gaussian - 180


Rho = np.corrcoef(skymapper_csv_SDSS.PA, skymapper_csv_SDSS.PA_SDSS_gaussian)
Pearson_rho = Rho[0,1]


PA_i = skymapper_csv_SDSS[(skymapper_csv_SDSS.type_SDSS == 3) & (skymapper_csv_SDSS.b_SDSS_gaussian > 0.5) & (skymapper_csv_SDSS.b > 0.5)].iloc[i].PA
PA_SDSS_gaussian_i = skymapper_csv_SDSS[(skymapper_csv_SDSS.type_SDSS == 3) & (skymapper_csv_SDSS.b_SDSS_gaussian > 0.5) & (skymapper_csv_SDSS.b > 0.5)].iloc[i].PA_SDSS_gaussian

plt.figure()
plt.plot(skymapper_csv_SDSS.PA,skymapper_csv_SDSS.PA_SDSS_gaussian,'b.')
#plt.plot(skymapper_csv_SDSS[skymapper_csv_SDSS.b_SDSS_gaussian > 0.5].PA,skymapper_csv_SDSS[skymapper_csv_SDSS.b_SDSS_gaussian > 0.5].PA_SDSS_gaussian,'r.',label='own b > 0.5 px')
plt.plot(skymapper_csv_SDSS[(skymapper_csv_SDSS.type_SDSS == 3) & (skymapper_csv_SDSS.ClassStar < stellar_index_filter) & (skymapper_csv_SDSS.b_SDSS_gaussian > 0.5) & (skymapper_csv_SDSS.b > 0.5) & (skymapper_csv_SDSS.BA_ratio < BA_ratio_filter) & (skymapper_csv_SDSS.BA_ratio_SDSS_gaussian < BA_ratio_filter)].PA,skymapper_csv_SDSS[(skymapper_csv_SDSS.type_SDSS == 3) & (skymapper_csv_SDSS.ClassStar < stellar_index_filter) & (skymapper_csv_SDSS.b_SDSS_gaussian > 0.5) & (skymapper_csv_SDSS.b > 0.5) & (skymapper_csv_SDSS.BA_ratio < BA_ratio_filter) & (skymapper_csv_SDSS.BA_ratio_SDSS_gaussian < BA_ratio_filter)].PA_SDSS_gaussian,'r.',label='SDSS "Galaxy" type, skymapper stellar index < {} b > 0.5 px and b/a < {:.2f} for both'.format(stellar_index_filter,BA_ratio_filter))
#plt.plot(PA_i,PA_SDSS_gaussian_i,'ks',label='Choices')
plt.xlabel('Major PA from skymapper',fontsize=15)
plt.ylabel('Major PA from my own fit to SDSS',fontsize=15)
plt.title('Skymapper all sources in common with SDSS',fontsize=20)
plt.legend()
plt.show() 

display(skymapper_csv_SDSS[(skymapper_csv_SDSS.type_SDSS == 3) & (skymapper_csv_SDSS.b_SDSS_gaussian > 0.5) & (skymapper_csv_SDSS.b > 0.5)].iloc[i])



print('Pearson corr. coefficient is {:.2f}'.format(Pearson_rho))

SDSS_index = skymapper_csv_SDSS[(skymapper_csv_SDSS.type_SDSS == 3) & (skymapper_csv_SDSS.b_SDSS_gaussian > 0.5) & (skymapper_csv_SDSS.b > 0.5)].iloc[i].SDSS_index
display(SDSS.iloc[[int(e) for e in SDSS_index]])

Pearson correlation and check that the Xmatch between SDSS and skymapper is done correctly. Yes it's correct. Now check two examples of galaxies going to SDSS and skymapper and checking the properties of both.

First example: J0057-0024 (lower right square)
    
SDSS (seems to be a star actually, with type 6 star but class QSO)
<img src="https://skyserver.sdss.org/dr17/SkyServerWS/ImgCutout/getjpeg?ra=14.3208364228579&dec=-0.40923294039877&scale=0.2&width=512&height=512&opt=">



skymapper

The ObjID belongs to data release 1, while now there is data release 2 and the ID is different for the new one, be careful! With the old one, the image does seem to match. This is it

<img src="https://api.skymapper.nci.org.au/public/siap/dr1/get_image?image=20150718174123-25&size=0.0333333333333,0.0333333333333&pos=14.320867,-0.409199&format=png">

But the quality is not the best. Using DR2 and looking with its coordinates, I get a better image:

<img src="https://api.skymapper.nci.org.au/public/siap/dr2/get_image?image=20140831164520-22&size=0.0333333333333,0.0333333333333&pos=14.320822,-0.409199&format=png">


The BA is more or less similar bewteen both (0.78 and 0.7) but the PA is way different. Because it's a star, probably the fits are just wrong.




Now the second example. Name: J0218-0923

<img src="https://skyserver.sdss.org/dr17/SkyServerWS/ImgCutout/getjpeg?ra=34.6432442799557&dec=-9.39401403876872&scale=0.2&width=512&height=512&opt=">

It's a STAR again


<img src="https://api.skymapper.nci.org.au/public/siap/dr1/get_image?image=20140926161300-10&size=0.0333333333333,0.0333333333333&pos=34.643162,-9.393959&format=png">

The redshift matches in both catalogues. In this case the PA is more similar bewteen both fits (-23 and -3) and the b/a is different (0.55 and 0.88). I think it's due to being a star and unresolved again. 

In [ ]:
for ra,source_ra,dec,source_dec in zip(skymapper_csv.ra[0:10],Source_coords_csv.RA_deg[skymapper_csv.indexes_Source_coords_csv][0:10],skymapper_csv.dec[0:10],Source_coords_csv.DEC_deg[skymapper_csv.indexes_Source_coords_csv][0:10]):
    print('{:.5f} {:.5f}    {:.5f} {:.5f}'.format(ra,source_ra,dec,source_dec))

Yes the XMatch is correct

In [ ]:
KIDS_fits = fits.open('KiDS_DR4.1_ugriZYJHKs_SOM_gold_WL_cat.fits')

header = KIDS_fits[1].header
data = KIDS_fits[1].data
KIDS_RA = np.array(data.RAJ2000)
KIDS_DEC = np.array(data.DECJ2000)
KIDS_A = data.A_IMAGE
KIDS_B = data.B_IMAGE
KIDS_THETA_J2000 = data.THETA_J2000
KIDS_THETA_WORLD = data.THETA_WORLD

In [ ]:
Source_coords_csv_RA = np.array(Source_coords_csv.RA_deg)
Source_coords_csv_DEC = np.array(Source_coords_csv.DEC_deg)
RA_bad = (((Source_coords_csv_RA > 0) & (Source_coords_csv_RA < 360)) == False) & (np.isnan(Source_coords_csv_RA) == True)
DEC_bad = (((Source_coords_csv_DEC > (-90)) & (Source_coords_csv_DEC < 90)) == False) & (np.isnan(Source_coords_csv_DEC) == True)
RA_good = (((Source_coords_csv_RA > 0) & (Source_coords_csv_RA < 360)) == True) & (np.isnan(Source_coords_csv_RA) == False)
DEC_good = (((Source_coords_csv_DEC > (-90)) & (Source_coords_csv_DEC < 90)) == True) & (np.isnan(Source_coords_csv_DEC) == False)

Source_coords_csv_RA_good = Source_coords_csv_RA[RA_good & DEC_good]
Source_coords_csv_DEC_good = Source_coords_csv_DEC[RA_good & DEC_good]

indexes_Source_coords_csv = np.arange(0,len(Source_coords_csv_RA))

indexes_Source_coords_csv_good = indexes_Source_coords_csv[RA_good & DEC_good]

#len_diff = len(KIDS_RA) - len(Source_coords_csv_RA)
#nan_array = np.empty(len_diff) 
#nan_array[:] = np.nan
#Source_coords_csv_RA = np.array(list(Source_coords_csv_RA) + list(nan_array))
#Source_coords_csv_DEC = np.array(list(Source_coords_csv_DEC) + list(nan_array))



SDSS_new_results_RA = np.array(SDSS_new_results.ra)
SDSS_new_results_DEC = np.array(SDSS_new_results.dec)
SDSS_new_res_RA_bad = (((SDSS_new_results_RA > 0) & (SDSS_new_results_RA < 360)) == False) & (np.isnan(SDSS_new_results_RA) == True)
SDSS_new_res_DEC_bad = (((SDSS_new_results_DEC > (-90)) & (SDSS_new_results_DEC < 90)) == False) & (np.isnan(SDSS_new_results_DEC) == True)
SDSS_new_res_RA_good = (((SDSS_new_results_RA > 0) & (SDSS_new_results_RA < 360)) == True) & (np.isnan(SDSS_new_results_RA) == False)
SDSS_new_res_DEC_good = (((SDSS_new_results_DEC > (-90)) & (SDSS_new_results_DEC < 90)) == True) & (np.isnan(SDSS_new_results_DEC) == False)

SDSS_new_results_RA_good = SDSS_new_results_RA[SDSS_new_res_RA_good & SDSS_new_res_DEC_good]
SDSS_new_results_DEC_good = SDSS_new_results_DEC[SDSS_new_res_RA_good & SDSS_new_res_DEC_good]

indexes_SDSS_new_results = np.arange(0,len(SDSS_new_results_RA))
indexes_SDSS_new_results_good = indexes_SDSS_new_results[SDSS_new_res_RA_good & SDSS_new_res_DEC_good]

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord



KIDS_catalogue = SkyCoord(KIDS_RA,KIDS_DEC,frame='icrs',unit='deg')
Astrogeo_coords_good = SkyCoord(Source_coords_csv_RA_good,Source_coords_csv_DEC_good,frame='icrs',unit='deg')
max_sep = 10 * u.arcsecond
idx, d2d, d3d = Astrogeo_coords_good.match_to_catalog_3d(KIDS_catalogue)
sep_constraint = d2d < max_sep
Astrogeo_matches_good = Astrogeo_coords_good[sep_constraint]
KIDS_catalogue_Astrogeo_matches = KIDS_catalogue[idx[sep_constraint]]


SDSS_new_results_coords_good = SkyCoord(SDSS_new_results_RA_good,SDSS_new_results_DEC_good,frame='icrs',unit='deg')
max_sep = 10 * u.arcsecond
idx, d2d, d3d = SDSS_new_results_coords_good.match_to_catalog_3d(KIDS_catalogue)
sep_constraint = d2d < max_sep
SDSS_new_results_matches_good = SDSS_new_results_coords_good[sep_constraint]
KIDS_catalogue_SDSS_new_results_matches = KIDS_catalogue[idx[sep_constraint]]

In [ ]:
plt.figure()
plt.scatter(Astrogeo_matches_good.ra.deg,Astrogeo_matches_good.dec.deg,s=1,c='b',label='Astrogeo')
plt.scatter(KIDS_catalogue_Astrogeo_matches.ra.deg,KIDS_catalogue_Astrogeo_matches.dec.deg,s=1,c='r',label='KIDS')
plt.xlabel('RA')
plt.ylabel('DEC')
plt.legend()
plt.title('KIDS and Astrogeo crossmatch for sep < 10 arcsec')
plt.savefig('Astrogeo_KIDS_match.png',dpi=900)
plt.show()


plt.figure()
plt.scatter(SDSS_new_results_matches_good.ra.deg,SDSS_new_results_matches_good.dec.deg,s=1,c='b',label='SDSS')
plt.scatter(KIDS_catalogue_SDSS_new_results_matches.ra.deg,KIDS_catalogue_SDSS_new_results_matches.dec.deg,s=1,c='r',label='KIDS')
plt.xlabel('RA')
plt.ylabel('DEC')
plt.legend()
plt.title('KIDS and SDSS crossmatch for sep < 10 arcsec')
plt.savefig('SDSS_KIDS_match.png',dpi=900)
plt.show()

In [ ]:
KIDS_A = data.A_IMAGE
KIDS_B = data.B_IMAGE
KIDS_PAgaap = data.PAgaap
KIDS_THETA_J2000 = data.THETA_J2000
KIDS_THETA_WORLD = data.THETA_WORLD
#KIDS_THETA_IMAGE = data.THETA_IMAGE  #Non existent


KIDS_SDSS_A_matches = KIDS_A[idx[sep_constraint]]
KIDS_SDSS_B_matches = KIDS_B[idx[sep_constraint]]
KIDS_SDSS_BA_matches = KIDS_SDSS_B_matches/KIDS_SDSS_A_matches
KIDS_SDSS_THETA_J2000_matches = KIDS_THETA_J2000[idx[sep_constraint]]
KIDS_SDSS_THETA_WORLD_matches = KIDS_THETA_WORLD[idx[sep_constraint]]
KIDS_SDSS_PAgaap_matches = KIDS_PAgaap[idx[sep_constraint]]
SDSS_BA_matches = SDSS_new_results.b_a_list[indexes_SDSS_new_results_good[sep_constraint]]
SDSS_THETA_matches = SDSS_new_results.pa_major[indexes_SDSS_new_results_good[sep_constraint]]

In [ ]:
plt.figure()
plt.plot(KIDS_SDSS_BA_matches,SDSS_BA_matches,'b.')
plt.xlabel('b/a KIDS')
plt.ylabel('b/a own from SDSS')
plt.savefig('SDSS_KIDS_BA.png',dpi=900)
plt.show()


SDSS_THETA_matches = np.fmod(SDSS_THETA_matches+360,180)
SDSS_THETA_matches[SDSS_THETA_matches > 90] = SDSS_THETA_matches - 180

#Convert west of north to east of north
KIDS_SDSS_THETA_J2000_matches = -KIDS_SDSS_PAgaap_matches
#It's already counterclockwise but it's from x axis
KIDS_SDSS_THETA_WORLD_matches = KIDS_SDSS_THETA_WORLD_matches-90
#Convert north of west to east of north
KIDS_SDSS_PAgaap_matches = KIDS_SDSS_PAgaap_matches - 90

plt.figure()
plt.plot(KIDS_SDSS_PAgaap_matches,SDSS_THETA_matches,'b.')
plt.xlabel('pa (deg) KIDS')
plt.ylabel('pa (deg) own from SDSS')
plt.savefig('SDSS_KIDS_PA.png',dpi=900)
plt.show()

In [ ]:
#Do DES_SDSS_comparison

DES_catalogue = SkyCoord(DES_catalogue_RA,DES_catalogue_DEC,frame='icrs',unit='deg')
SDSS_catalogue_coords = SkyCoord(SDSS_RA,SDSS_DEC,frame='icrs',unit='deg')
max_sep = 20 * u.arcsecond
idx, d2d, d3d = SDSS_catalogue_coords.match_to_catalog_3d(DES_catalogue)
sep_constraint = d2d < max_sep
SDSS_catalogue_matches = SDSS_catalogue_coords[sep_constraint]
DES_catalogue_SDSS_matches = DES_catalogue[idx[sep_constraint]]

In [ ]:
plt.figure()
plt.scatter(SDSS_catalogue_matches.ra.deg,SDSS_catalogue_matches.dec.deg,s=1,c='b',label='SDSS catalogue')
plt.scatter(DES_catalogue_SDSS_matches.ra.deg,DES_catalogue_SDSS_matches.dec.deg,s=1,c='r',label='DES')
plt.xlabel('RA')
plt.ylabel('DEC')
plt.legend()
plt.title('DES and SDSS catalogue crossmatch for sep < {} arcsec'.format(max_sep))
plt.savefig('SDSS_catalogue_DES_match.png',dpi=900)
plt.show()

In [ ]:
DES_SDSS_catalogue_A_matches = np.array(DES_catalogue_A)[idx[sep_constraint]]
DES_SDSS_catalogue_B_matches = np.array(DES_catalogue_B)[idx[sep_constraint]]
DES_SDSS_catalogue_BA_matches = DES_SDSS_catalogue_B_matches/DES_SDSS_catalogue_A_matches
DES_SDSS_catalogue_THETA_J2000_matches = np.array(DES_catalogue_THETA_J2000)[idx[sep_constraint]]
DES_SDSS_catalogue_THETAWIN_IMAGE_G_matches = np.array(DES_catalogue_THETAWIN_IMAGE_G)[idx[sep_constraint]]
SDSS_catalogue_DES_BA_matches = np.array(SDSS_catalogue.deVAB_g)[sep_constraint]
SDSS_catalogue_DES_THETA_J2000_matches = np.array(SDSS_catalogue.deVPhi_g)[sep_constraint]



plt.figure()
plt.plot(DES_SDSS_catalogue_BA_matches,SDSS_catalogue_DES_BA_matches,'b.')
plt.xlabel('b/a DES')
plt.ylabel('b/a SDSS catalogue')
plt.savefig('DES_SDSS_catalogue_BA.png',dpi=900)
plt.show()




#Convert west of north to east of north
#DES_SDSS_catalogue_THETA_J2000_matches = -DES_SDSS_catalogue_THETA_J2000_matches
#It's already counterclockwise but it's from x axis
#DES_SDSS_catalogue_THETAWIN_IMAGE_G_matches = DES_SDSS_catalogue_THETAWIN_IMAGE_G_matches-90


plt.figure()
plt.plot(DES_SDSS_catalogue_THETA_J2000_matches,SDSS_catalogue_DES_THETA_J2000_matches,'b.')
plt.xlabel('pa (deg) DES')
plt.ylabel('pa (deg) SDSS catalogue')
plt.savefig('SDSS_KIDS_PA.png',dpi=900)
plt.show()

In [ ]:
VLBI_source_matches_csv

In [ ]:
skymapper_csv

In [ ]:
VLBI_source_matches_csv_renamed = VLBI_source_matches_csv.rename(columns={'RA': 'ra', 'DEC': 'dec'})
skymapper_csv_rounded = skymapper_csv.round({'ra': 4, 'dec': 4})
VLBI_source_matches_csv_rounded = VLBI_source_matches_csv_renamed.round({'ra': 4, 'dec': 4})
DESI_skymapper_XMatch = pd.merge(skymapper_csv_rounded,VLBI_source_matches_csv_rounded,on=['ra', 'dec'], how='inner')
# Reset the index of the merged dataframe
DESI_skymapper_XMatch = DESI_skymapper_XMatch.reset_index(drop=True)

In [ ]:
class_star_filter = 1

plt.figure()
plt.plot(DESI_skymapper_XMatch[(DESI_skymapper_XMatch.TYPE=='SER')].BA_ratio_x,DESI_skymapper_XMatch[DESI_skymapper_XMatch.TYPE=='SER'].BA_ratio_y,'b.')
plt.plot(DESI_skymapper_XMatch[(DESI_skymapper_XMatch.TYPE=='SER') & (DESI_skymapper_XMatch.b > 0.5) & (DESI_skymapper_XMatch.ClassStar < class_star_filter)].BA_ratio_x,DESI_skymapper_XMatch[(DESI_skymapper_XMatch.TYPE=='SER')  & (DESI_skymapper_XMatch.b > 0.5) & (DESI_skymapper_XMatch.ClassStar < class_star_filter)].BA_ratio_y,'r.',label='Stellar index < {}, skymapper b > 0.5 px'.format(class_star_filter))

plt.xlabel('Skymapper b/a')
plt.ylabel('DESI Sérsic types b/a')
plt.legend()
plt.show()

plt.figure()
plt.plot(DESI_skymapper_XMatch[DESI_skymapper_XMatch.TYPE=='SER'].PA,DESI_skymapper_XMatch[DESI_skymapper_XMatch.TYPE=='SER'].pos_angle,'b.')
plt.plot(DESI_skymapper_XMatch[(DESI_skymapper_XMatch.TYPE=='SER') & (DESI_skymapper_XMatch.b > 0.5) & (DESI_skymapper_XMatch.ClassStar < class_star_filter)].PA,DESI_skymapper_XMatch[(DESI_skymapper_XMatch.TYPE=='SER') & (DESI_skymapper_XMatch.b > 0.5) & (DESI_skymapper_XMatch.ClassStar < class_star_filter)].pos_angle,'r.',label='Stellar index < {}, skymapper b > 0.5 px'.format(class_star_filter))
plt.xlabel('Skymapper PA (deg)')
plt.ylabel('DESI Sérsic types PA (deg)')
plt.legend()
plt.show()


In [ ]:
SDSS_new_results = pd.read_csv('SDSS_new_results.csv')
jet_minor_difference = SDSS_new_results['jet_minor_difference']
good_index = []
e_filter = 0.8
s_n_filter = 200
err_filter = 50
b_a_filter = 0.7
b_filter = 0.5
for i,(e,s_n,err,b_a,b,type_SDSS) in enumerate(zip(list(SDSS_new_results['ell']),list(SDSS_new_results['Signal_to_noise_ratio']),list(SDSS_new_results['jet_minor_difference_err']),list(SDSS_new_results['b_a_list']),list(SDSS_new_results['b']),list(SDSS_new_results['type_SDSS']))):
    if (b_a < b_a_filter) & (b > b_filter) & (type_SDSS == 3): # and  (err < err_filter) (s_n > s_n_filter) and 
        good_index.append(i)

jet_minor_difference_good = jet_minor_difference[good_index]

Bin_1_mean = np.mean(jet_minor_difference[jet_minor_difference<45])
Bin_1_std = np.std(jet_minor_difference[jet_minor_difference<45])
Bin_1_sqrtN = np.sqrt(len(jet_minor_difference[jet_minor_difference<45]))

Bin_2_mean = np.mean(jet_minor_difference[jet_minor_difference>45])
Bin_2_std = np.std(jet_minor_difference[jet_minor_difference>45])
Bin_2_sqrtN = np.sqrt(len(jet_minor_difference[jet_minor_difference>45]))

counts,bins= np.histogram(jet_minor_difference,bins=2)
bin_centers = (bins[1:]+bins[:-1])/2
yerrs = np.array([Bin_1_sqrtN,Bin_2_sqrtN])
plt.figure()
#plt.hist(jet_minor_difference,bins=2,color='blue',alpha=1)
plt.hist(jet_minor_difference_good,bins=2,color='red',label= 'Type "galaxy", b > {} px and b/a < {}'.format(b_filter,b_a_filter),alpha=1)
#plt.errorbar(bin_centers,counts,yerr=yerrs,fmt=' ',ecolor='black',label=r'$\sqrt{N}$')
#plt.hist(jet_minor_difference_good,bins=7,color='red',label='e > {}, S/N > {}, deg_err < {}'.format(e_filter,s_n_filter,err_filter),alpha=0.8)
plt.xlabel('Jet-minor axis angle difference (deg)',fontsize=15)
plt.ylabel('Counts',fontsize=15)
plt.legend(loc='lower center')
plt.title('SDSS sources',fontsize=20)
plt.show()

print('<45 Mean =',Bin_1_mean )
print('<45 Std = ',Bin_1_std)
print('<45 sqrt(N) =',Bin_1_sqrtN)
print('>45 Mean =',Bin_2_mean)
print('>45 Std = ',Bin_2_std)
print('>45 sqrt(N) =',Bin_2_sqrtN)





Bin_1_mean_good = np.mean(jet_minor_difference_good[jet_minor_difference_good<45])
Bin_1_std_good = np.std(jet_minor_difference_good[jet_minor_difference_good<45])
Bin_1_sqrtN_good = np.sqrt(len(jet_minor_difference_good[jet_minor_difference_good<45]))

Bin_2_mean_good = np.mean(jet_minor_difference_good[jet_minor_difference_good>45])
Bin_2_std_good = np.std(jet_minor_difference_good[jet_minor_difference_good>45])
Bin_2_sqrtN_good = np.sqrt(len(jet_minor_difference_good[jet_minor_difference_good>45]))

counts_good,bins_good= np.histogram(jet_minor_difference_good,bins=2)
bin_centers_good = (bins_good[1:]+bins_good[:-1])/2
yerrs_good = np.array([Bin_1_sqrtN_good,Bin_2_sqrtN_good])

print('<45 Mean =', np.mean(jet_minor_difference_good[jet_minor_difference<45]))
print('<45 Std = ',np.std(jet_minor_difference_good[jet_minor_difference<45]))
print('<45 sqrt(N) =',np.sqrt(len(jet_minor_difference[jet_minor_difference<45])))
print('>45 Mean =', np.mean(jet_minor_difference_good[jet_minor_difference>45]))
print('>45 Std = ',np.std(jet_minor_difference_good[jet_minor_difference>45]))
print('>45 sqrt(N) =',np.sqrt(len(jet_minor_difference[jet_minor_difference>45])))

In [ ]:
plt.figure()
plt.hist(VLBI_source_matches_csv.pa_difference,color='b')
plt.hist(VLBI_source_matches_csv[VLBI_source_matches_csv.TYPE == 'SER'].pa_difference,color='r',label='Sérsic type')
#plt.hist(VLBI_source_matches_csv[VLBI_source_matches_csv.b > 0.5].pa_difference,'r',label='b > 0.5')
plt.xlabel('Jet-minor axis difference (deg)',fontsize=15)
plt.ylabel('Counts',fontsize=15)
plt.title('DESI',fontsize=20)
plt.legend()
plt.show()

BA_ratio_filter = 0.7

n_sources = len(VLBI_source_matches_csv[VLBI_source_matches_csv.TYPE == 'SER'].pa_difference)
n_bins = 17 #int(np.sqrt(n_sources))
#print(n_sources,n_bins)
plt.figure()
counts,bins = np.histogram(VLBI_source_matches_csv[VLBI_source_matches_csv.TYPE == 'SER'].pa_difference,bins=n_bins)
bin_middles = (bins[:-1] + bins[1:])/2
p,cov = np.polyfit(bin_middles,counts,deg=1,cov=True,w=1/np.sqrt(counts))
plt.hist(VLBI_source_matches_csv[VLBI_source_matches_csv.TYPE == 'SER'].pa_difference,bins=n_bins,color='b')
plt.hist(VLBI_source_matches_csv[(VLBI_source_matches_csv.TYPE == 'SER') & (VLBI_source_matches_csv.BA_ratio < BA_ratio_filter)].pa_difference,bins=n_bins,color='r',label='b/a < {}'.format(BA_ratio_filter))
plt.plot(bin_middles,p[0]*bin_middles+p[1],'r-',label=r'Slope = {:.2f} $\pm$ {:.2f}'.format(p[0],np.sqrt(cov[0,0])))
#plt.hist(VLBI_source_matches_csv[(VLBI_source_matches_csv.TYPE == 'SER')&(VLBI_source_matches_csv.b > 0.5)].pa_difference,'r',label='b > 0.5 px')
plt.xlabel('Jet-minor axis difference (deg)',fontsize=15)
plt.ylabel('Counts',fontsize=15)
plt.title('DESI Sérsic type',fontsize=20)
plt.legend()
plt.show()



plt.figure()
plt.hist(VLBI_source_matches_csv.pa_difference,bins=15,color='red',label='DESI',alpha=1)
plt.hist(jet_minor_difference,bins=15,color='blue',label='SDSS',alpha=1)
plt.xlabel('Jet-minor axis difference (deg)',fontsize=15)
plt.ylabel('Counts',fontsize=15)
plt.legend()
plt.show()


b_filter = 0.5
plt.figure()
plt.hist(skymapper_csv.pa_difference,color='b')
plt.hist(skymapper_csv[skymapper_csv.b > b_filter].pa_difference,color='r',label='b > {:.1f} px'.format(b_filter))
plt.xlabel('Jet-minor axis difference (deg)',fontsize=15)
plt.ylabel('Counts',fontsize=15)
plt.title('Skymapper all sources',fontsize=20)
plt.legend()
plt.show()

In [ ]:
b_a_list = np.array(VLBI_source_matches_csv[VLBI_source_matches_csv.TYPE == 'SER'].BA_ratio)
jet_minor_difference = np.array(VLBI_source_matches_csv[VLBI_source_matches_csv.TYPE == 'SER'].pa_difference)

counts_b_a,bins_b_a = np.histogram(b_a_list,bins=5)
bins_b_a_center = (bins_b_a[:-1]+bins_b_a[1:])/2
b_a_sorted = np.sort(b_a_list)
b_a_index_sorted = np.argsort(b_a_list)

b_a_binedges = b_a_sorted.searchsorted(bins_b_a)

original_indexes_per_bin = [b_a_index_sorted[list(range(b_a_binedges[k], b_a_binedges[k+1]))] for k in range(len(b_a_binedges)-1)]
b_a_jet_minor_difference_per_bin = []
b_a_jet_minor_difference_means_per_bin = np.zeros(len(bins_b_a_center))
b_a_jet_minor_difference_sigmas_per_bin = np.zeros(len(bins_b_a_center))
for i,l in enumerate(original_indexes_per_bin):
    b_a_jet_minor_difference_per_bin.append(jet_minor_difference[l])
    mean = np.mean(jet_minor_difference[l])
    sigma = np.mean(jet_minor_difference[l])
    b_a_jet_minor_difference_means_per_bin[i] = mean
    b_a_jet_minor_difference_sigmas_per_bin[i] = sigma
    
regr = LinearRegression()
regr.fit(bins_b_a_center.reshape(len(bins_b_a_center), 1),b_a_jet_minor_difference_means_per_bin.reshape(len(b_a_jet_minor_difference_means_per_bin), 1), 0.01*b_a_jet_minor_difference_sigmas_per_bin)


plt.figure()
plt.plot(bins_b_a_center,b_a_jet_minor_difference_means_per_bin,'bo')
plt.errorbar(bins_b_a_center,b_a_jet_minor_difference_means_per_bin,yerr=0.1*b_a_jet_minor_difference_sigmas_per_bin,fmt='None',ecolor='black',label=r'0.1$\sigma$')
#plt.plot(bins_b_a_center.reshape(len(bins_b_a_center), 1), regr.predict(bins_b_a_center.reshape(len(bins_b_a_center), 1)))
plt.xlabel('b/a',fontsize=15)
plt.ylabel('Mean of the Jet-Minor axis angle difference',fontsize=15)
plt.title('DESI Sérsic type',fontsize=20)
plt.legend()
plt.show()



b_a_list = np.array(skymapper_csv[skymapper_csv.b > 0.5].BA_ratio)
jet_minor_difference = np.array(skymapper_csv[skymapper_csv.b > 0.5].pa_difference)

counts_b_a,bins_b_a = np.histogram(b_a_list,bins=5)
bins_b_a_center = (bins_b_a[:-1]+bins_b_a[1:])/2
b_a_sorted = np.sort(b_a_list)
b_a_index_sorted = np.argsort(b_a_list)

b_a_binedges = b_a_sorted.searchsorted(bins_b_a)

original_indexes_per_bin = [b_a_index_sorted[list(range(b_a_binedges[k], b_a_binedges[k+1]))] for k in range(len(b_a_binedges)-1)]
b_a_jet_minor_difference_per_bin = []
b_a_jet_minor_difference_means_per_bin = np.zeros(len(bins_b_a_center))
b_a_jet_minor_difference_sigmas_per_bin = np.zeros(len(bins_b_a_center))
for i,l in enumerate(original_indexes_per_bin):
    b_a_jet_minor_difference_per_bin.append(jet_minor_difference[l])
    mean = np.mean(jet_minor_difference[l])
    sigma = np.mean(jet_minor_difference[l])
    b_a_jet_minor_difference_means_per_bin[i] = mean
    b_a_jet_minor_difference_sigmas_per_bin[i] = sigma
    
regr = LinearRegression()
regr.fit(bins_b_a_center.reshape(len(bins_b_a_center), 1),b_a_jet_minor_difference_means_per_bin.reshape(len(b_a_jet_minor_difference_means_per_bin), 1), 0.01*b_a_jet_minor_difference_sigmas_per_bin)


plt.figure()
plt.plot(bins_b_a_center,b_a_jet_minor_difference_means_per_bin,'bo')
plt.errorbar(bins_b_a_center,b_a_jet_minor_difference_means_per_bin,yerr=0.1*b_a_jet_minor_difference_sigmas_per_bin,fmt='None',ecolor='black',label=r'0.1$\sigma$')
#plt.plot(bins_b_a_center.reshape(len(bins_b_a_center), 1), regr.predict(bins_b_a_center.reshape(len(bins_b_a_center), 1)))
plt.xlabel('b/a',fontsize=15)
plt.ylabel('Mean of the Jet-Minor axis angle difference',fontsize=15)
plt.title('Skymapper b > 0.5 px',fontsize=20)
plt.legend()
plt.show()


jet_minor_difference = np.array(SDSS_new_results[(SDSS_new_results.b > 0.5) & (SDSS_new_results.type_SDSS == 3)].jet_minor_difference)
b_a_list = np.array(SDSS_new_results[(SDSS_new_results.b > 0.5) & (SDSS_new_results.type_SDSS == 3)].b_a_list)


counts_b_a,bins_b_a = np.histogram(b_a_list,bins=5)
bins_b_a_center = (bins_b_a[:-1]+bins_b_a[1:])/2
b_a_sorted = np.sort(b_a_list)
b_a_index_sorted = np.argsort(b_a_list)

b_a_binedges = b_a_sorted.searchsorted(bins_b_a)

original_indexes_per_bin = [b_a_index_sorted[list(range(b_a_binedges[k], b_a_binedges[k+1]))] for k in range(len(b_a_binedges)-1)]
b_a_jet_minor_difference_per_bin = []
b_a_jet_minor_difference_means_per_bin = np.zeros(len(bins_b_a_center))
b_a_jet_minor_difference_sigmas_per_bin = np.zeros(len(bins_b_a_center))
for i,l in enumerate(original_indexes_per_bin):
    b_a_jet_minor_difference_per_bin.append(jet_minor_difference[l])
    mean = np.mean(jet_minor_difference[l])
    sigma = np.mean(jet_minor_difference[l])
    b_a_jet_minor_difference_means_per_bin[i] = mean
    b_a_jet_minor_difference_sigmas_per_bin[i] = sigma
    
regr = LinearRegression()
regr.fit(bins_b_a_center.reshape(len(bins_b_a_center), 1),b_a_jet_minor_difference_means_per_bin.reshape(len(b_a_jet_minor_difference_means_per_bin), 1), 0.01*b_a_jet_minor_difference_sigmas_per_bin)


plt.figure()
plt.plot(bins_b_a_center,b_a_jet_minor_difference_means_per_bin,'bo')
plt.errorbar(bins_b_a_center,b_a_jet_minor_difference_means_per_bin,yerr=0.01*b_a_jet_minor_difference_sigmas_per_bin,fmt='None',ecolor='black',label=r'0.01$\sigma$')
#plt.plot(bins_b_a_center.reshape(len(bins_b_a_center), 1), regr.predict(bins_b_a_center.reshape(len(bins_b_a_center), 1)))
plt.xlabel('b/a',fontsize=15)
plt.ylabel('Mean of the Jet-Minor axis angle difference',fontsize=15)
plt.title('SDSS type galaxy and b > 0.5 px',fontsize=20)
plt.legend()
plt.show()

In [ ]:
ell = np.array(SDSS_new_results['ell'])
ell_errs = np.array(SDSS_new_results['ell_errs'])
err = np.array(SDSS_new_results['jet_minor_difference_err'])
s_n = np.array(SDSS_new_results['Signal_to_noise_ratio'])
b_a_list = np.array(SDSS_new_results['b_a_list'])
plt.figure()
plt.scatter(err*np.exp(1),ell_errs,c=s_n)
plt.ylabel('ell_err')
plt.xlabel('PA err*e (deg)')
#plt.xlim([0,5000])
#plt.ylim([0,10])
plt.xlim([0,360])
plt.ylim([0,1])
plt.colorbar()
plt.show()



counts_ell,bins_ell = np.histogram(ell,bins=10)
bins_ell_center = (bins_ell[:-1]+bins_ell[1:])/2
ell_sorted = np.sort(ell)
ell_index_sorted = np.argsort(ell)

ell_binedges = ell_sorted.searchsorted(bins_ell)

ell_original_indexes_per_bin = [ell_index_sorted[list(range(ell_binedges[k], ell_binedges[k+1]))] for k in range(len(ell_binedges)-1)]
ell_jet_minor_difference_per_bin = []
ell_jet_minor_difference_means_per_bin = np.zeros(len(bins_ell_center))
for i,l in enumerate(ell_original_indexes_per_bin):
    ell_jet_minor_difference_per_bin.append(jet_minor_difference[l])
    mean = np.mean(jet_minor_difference[l])
    ell_jet_minor_difference_means_per_bin[i] = mean
    
plt.figure()
plt.plot(bins_ell_center,ell_jet_minor_difference_means_per_bin,'bo')
plt.xlabel('ell')
plt.ylabel('Mean of the Jet-Minor axis angle difference')
plt.show()


counts_b_a,bins_b_a = np.histogram(b_a_list,bins=10)
bins_b_a_center = (bins_b_a[:-1]+bins_b_a[1:])/2
b_a_sorted = np.sort(b_a_list)
b_a_index_sorted = np.argsort(b_a_list)

b_a_binedges = b_a_sorted.searchsorted(bins_b_a)

original_indexes_per_bin = [b_a_index_sorted[list(range(b_a_binedges[k], b_a_binedges[k+1]))] for k in range(len(b_a_binedges)-1)]
b_a_jet_minor_difference_per_bin = []
b_a_jet_minor_difference_means_per_bin = np.zeros(len(bins_b_a_center))
b_a_jet_minor_difference_sigmas_per_bin = np.zeros(len(bins_b_a_center))
for i,l in enumerate(original_indexes_per_bin):
    b_a_jet_minor_difference_per_bin.append(jet_minor_difference[l])
    mean = np.mean(jet_minor_difference[l])
    sigma = np.mean(jet_minor_difference[l])
    b_a_jet_minor_difference_means_per_bin[i] = mean
    b_a_jet_minor_difference_sigmas_per_bin[i] = sigma
    
regr = LinearRegression()
regr.fit(bins_b_a_center.reshape(len(bins_b_a_center), 1),b_a_jet_minor_difference_means_per_bin.reshape(len(b_a_jet_minor_difference_means_per_bin), 1), 0.01*b_a_jet_minor_difference_sigmas_per_bin)


plt.figure()
plt.plot(bins_b_a_center,b_a_jet_minor_difference_means_per_bin,'bo')
plt.errorbar(bins_b_a_center,b_a_jet_minor_difference_means_per_bin,yerr=0.01*b_a_jet_minor_difference_sigmas_per_bin,fmt='None',ecolor='black')
plt.plot(bins_b_a_center.reshape(len(bins_b_a_center), 1), regr.predict(bins_b_a_center.reshape(len(bins_b_a_center), 1)))
plt.xlabel('b/a')
plt.ylabel('Mean of the Jet-Minor axis angle difference')
plt.show()

En KIDS miden el ángulo desde el NORTE hacia el OESTE. El AR aumenta siempre hacia el este. PERO en las imágenes de VLBI y de SDSS, el eje de RA está disminuyendo! Es decir, el ESTE está a la izquierda. Así que el ángulo de KIDS estará midiendo desde el +y hacia la derecha, en sentido horario. Invertir!

In [ ]:
#KIDS = pd.DataFrame(data,columns = col_names)
#print(header)
#print(data[0])
#col_names = np.array([col.name for col in data.columns])
#print(col_names)
#lists = np.zeros(len(data))
#for i in range(len(data)):
    #lists[i] = list(data[i])
#Table(data[0:10])

#data['OBJECT_POS']

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.convolution import convolve_fft
from scipy.optimize import least_squares


# This is the Gaussian class that I wrote.
class Gaussian:
    def __init__(self, peak, cenx, ceny, sigx, sigy, theta):
        self.peak = peak
        self.cenx = cenx
        self.ceny = ceny
        self.sigx = sigx
        self.sigy = sigy
        self.theta = theta

    def get_postage(self, size):
        xx, yy = np.meshgrid(np.arange(size),np.arange(size))
        return background + self.peak * np.exp(-0.5* ( ( ( ( (xx-self.cenx) * np.cos(self.theta) + (yy-self.ceny) * np.sin(self.theta) ) / self.sigx )**2 ) + ( ( ( (xx-self.cenx) * np.sin(self.theta) - (yy-self.ceny) * np.cos(self.theta) ) / self.sigy )**2 ) ) )
    

#The Sersic profile fits surface brightness in Flux/arcsec**2. Our images
#are in Nanomaggies (approx 3.631×10-6 Jy) and that is a Flux. So we may
#have to divide bewteen the area of each pixel (which is 0.4 x 0.4 arcsec^2)
#to convert it to surface brightness.
class Sersic:
    def __init__(self,I_e,r_eff,n,x0,y0,ellip,theta):
        self.I_e = I_e
        self.r_eff = r_eff
        self.n = n
        self.x0 = x0
        self.y0 = y0
        self.ellip = ellip
        self.theta = theta
    def get_postage(self,size):
        xx, yy = np.meshgrid(np.arange(size),np.arange(size))
        bn = gammaincinv(2. * self.n, 0.5)
        a, b = self.r_eff, (1 - self.ellip) * self.r_eff #semi(maj and min)
        x_maj = (xx - self.x0) * np.cos(self.theta) + (yy - self.y0) * np.sin(self.theta)
        x_min = -(xx - self.x0) * np.sin(self.theta) + (yy - self.y0) * np.cos(self.theta)
        z = np.sqrt((x_maj / a) ** 2 + (x_min / b) ** 2)
        return self.I_e*np.exp(-bn * (z ** (1 / self.n) - 1))

def sersic_loss_function(parameters, data, psf,rms):
    model_galaxy = Sersic(*parameters).get_postage(51)
    model_observed = convolve_fft(model_galaxy, psf, boundary='fill')
    residual = (data - model_observed)/rms
    return residual.flatten() 

def direct_sersic_fit_loss_function(parameters, data,rms):
    model_observed_with_psf = Sersic(*parameters).get_postage(51)
    residual = (data - model_observed_with_psf)/rms
    return residual.flatten()    
    

# A least squares loss function. Used in minimization.
def gaussian_loss_function(parameters, data, psf,rms):
    model_galaxy = Gaussian(*parameters).get_postage(51)
    model_observed = convolve_fft(model_galaxy, psf, boundary='fill')
    residual = (data - model_observed)/rms
    return residual.flatten() 

def direct_gaussian_fit_loss_function(parameters, data,rms):
    model_observed_with_psf = Gaussian(*parameters).get_postage(51)
    residual = (data - model_observed_with_psf)/rms
    return residual.flatten()


In [ ]:
background = 0
fig, ax = plt.subplots(2,3)
for i,frac in enumerate(np.array([0.,0.2,0.4,0.6,0.8,1.])):
    ellipse = Gaussian(1, 50, 50, 15, frac*15, 0).get_postage(100)
    ax[i//3,i%3].imshow(ellipse)
    ax[i//3,i%3].set_title('b/a = {:.2f}'.format(frac))
fig.tight_layout()
plt.show()

In [ ]:
n = 4
bn = gammaincinv(2. * n, 0.5)
bn

In [ ]:
def sersic(r,I_e,r_eff,n):
    b_n = gammaincinv(2. * n, 0.5) 
    return I_e*np.exp(-b_n*((r/r_eff)**(1/n)-1))
    

In [ ]:
r = np.linspace(0,51,100)
I_e,r_eff,n = 10,10000,4


plt.figure()
plt.plot(sersic(r,I_e,r_eff,n))
plt.xlabel('r(px)')
plt.ylabel('I(r)')
plt.show()

plt.figure()
plt.plot(np.log10(sersic(r,I_e,r_eff,n)))
plt.xlabel('r(px)')
plt.ylabel('log 10 I(r)')
plt.show()

In [ ]:
I_e,r_eff,n,x0,y0,ellip,theta = 1,10000,4.8,22,22,0.7,57*np.pi/180
real_galaxy = Sersic(I_e,r_eff,n,x0,y0,ellip,theta).get_postage(51)

plt.figure()
plt.imshow(real_galaxy)
plt.gca().invert_yaxis()
plt.colorbar()
plt.title('Theoretical galaxy')
plt.show()

plt.figure()
plt.imshow(psf)
plt.gca().invert_yaxis()
plt.colorbar()
plt.title('PSF')
plt.show()


galaxy_observed = convolve_fft(real_galaxy, psf, boundary='fill')

n_galaxy_observed = galaxy_observed.shape[0]
alist=[galaxy_observed[0,:-1], galaxy_observed[:-1,-1], galaxy_observed[-1,::-1], galaxy_observed[-2:0:-1,0]]
galaxy_observed_rim = np.concatenate(alist)
average_noise = abs(np.mean(galaxy_observed_rim))
average_noise_sigma = np.std(galaxy_observed_rim)
background = average_noise
rms = average_noise_sigma 

plt.figure()
plt.imshow(galaxy_observed)
plt.gca().invert_yaxis()
plt.colorbar()
plt.title('Theoretical galaxy+PSF = Observed galaxy')
plt.show()

lower_bounds = [0,0,0.5,0,0,0,-np.inf]
p_guess = np.amax(galaxy_observed)/2,100,4,25,25,0.5,45*np.pi/180
upper_bounds = [np.inf,np.inf,16,51,51,1,np.inf]
# And put it all into the function minimization library.
res_lsq = least_squares(sersic_loss_function, p_guess, bounds=(lower_bounds,upper_bounds), args=(galaxy_observed, psf,rms))

I_e_fit,r_eff_fit,n_fit,x0_fit,y0_fit,ellip_fit,theta_fit = res_lsq.x[0],res_lsq.x[1],res_lsq.x[2],res_lsq.x[3],res_lsq.x[4],res_lsq.x[5],res_lsq.x[6]

model_galaxy = Sersic(I_e_fit,r_eff_fit,n_fit,x0_fit,y0_fit,ellip_fit,theta_fit).get_postage(51)
model_observed_galaxy = convolve_fft(model_galaxy, psf, boundary='fill')
residuals = galaxy_observed - model_observed_galaxy

plt.figure()
plt.imshow(model_galaxy)
plt.gca().invert_yaxis()
plt.colorbar()
plt.title('Model galaxy')
plt.show()

plt.figure()
plt.imshow(model_observed_galaxy)
plt.gca().invert_yaxis()
plt.colorbar()
plt.title('Model galaxy+PSF')
plt.show()



norm = mcolors.TwoSlopeNorm(vmin=residuals.min(), vmax = residuals.max(), vcenter=np.median(residuals))
plt.figure()


im = plt.imshow(residuals,cmap=plt.cm.RdBu, norm=norm)
ax = plt.gca()
ax.invert_yaxis()
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
plt.colorbar(im, cax=cax, orientation='vertical')
plt.title('Theoretical galaxy+PSF - Model galaxy+PSF')
plt.show()

print('The parameter names are I_e,r_eff,n,x0,y0,ellip,theta')
print('Theoretical parameteres are',I_e,r_eff,n,x0,y0,ellip,theta)
print('Fitted parameters are',I_e_fit,r_eff_fit,n_fit,x0_fit,y0_fit,ellip_fit,theta_fit)


n_param_sersic = len(lower_bounds)
Chi2_sersic = (residuals**2/average_noise_sigma**2).sum()
n_x,n_y = galaxy_observed.shape
n_data = n_x*n_y
dof = n_data - n_param_sersic
Chi2_redu_sersic = Chi2_sersic/dof

print('Reduced Chi squared',Chi2_redu_sersic)

In [ ]:
background = 0
cenx = 25
ceny = 25
theta = 60*np.pi/180
Jeff = 35
sigma_x = 5
sigma_y = 10
ellipse = Gaussian(50,25,25,sigma_x,sigma_y,theta).get_postage(51)


a = max(sigma_x,sigma_y)
b = min(sigma_x,sigma_y)

theta_from_positive_y = 180*(theta)/np.pi - 90
if sigma_y > sigma_x:
    theta_minor = theta_from_positive_y
    theta_major = theta_from_positive_y - 90
else:
    theta_minor = theta_from_positive_y - 90
    theta_major = theta_from_positive_y

theta_minor = np.fmod(theta_minor+2*360,360)    
theta_major = np.fmod(theta_major+2*360,360) 

minor_jet_angle = abs(Jeff - theta_minor)
if 90 < minor_jet_angle <= 180:
    minor_jet_angle = 180 - minor_jet_angle
if 180 < minor_jet_angle <= 270:
    minor_jet_angle = minor_jet_angle - 180
if 270 < minor_jet_angle <= 360:
    minor_jet_angle = 360 - minor_jet_angle


#Jeff
x0 = y0 = 25
Jeff_l = 20
x_Jeff = x0 + Jeff_l*np.cos((Jeff+90)*np.pi/180)
y_Jeff = y0 + Jeff_l*np.sin((Jeff+90)*np.pi/180)

#Major
x_M_1 = x0 + a*2*np.cos((theta_major+90)*np.pi/180)
y_M_1 = y0 + a*2*np.sin((theta_major+90)*np.pi/180)
x_M_2 = x0 + a*2*np.cos((theta_major-90)*np.pi/180)
y_M_2 = y0 + a*2*np.sin((theta_major-90)*np.pi/180)
#Minor
x_m_1 = x0 + b*2*np.cos((theta_minor+90)*np.pi/180)
y_m_1 = y0 + b*2*np.sin((theta_minor+90)*np.pi/180)
x_m_2 = x0 + b*2*np.cos((theta_minor-90)*np.pi/180)
y_m_2 = y0 + b*2*np.sin((theta_minor-90)*np.pi/180)
    
plt.figure()
plt.imshow(ellipse)
plt.plot([x0,x_M_1],[y0,y_M_1],'w-')
plt.plot([x0,x_M_2],[y0,y_M_2],'w-')
plt.plot([x0,x_m_1],[y0,y_m_1],'w:')
plt.plot([x0,x_m_2],[y0,y_m_2],'w:')
plt.plot([x0,x_Jeff],[y0,y_Jeff],'r-')
plt.gca().invert_yaxis()
plt.show()

print(theta_minor)
print(minor_jet_angle)

In [ ]:
psfile_string = 'https://data.sdss.org/sas/dr17/eboss/photo/redux/301/756/objcs/2/psField-000756-2-0427.fit'
print(psfile_string)



XPOS = 1664.169
YPOS = 135.3008

psfile_data = fits.open(psfile_string)
psf = reconstructPSF(psfile_string, 'r', YPOS, XPOS)
#psf = resize(psf, frame_image.shape)


frame_string= 'https://data.sdss.org/sas/dr17/eboss/photoObj/frames/301/756/2/frame-r-000756-2-0427.fits.bz2'
frame = fits.open(frame_string)
frame_image = frame[0].data
n = 25
frame_image_copy = frame_image.copy()
frame_zoom = frame_image_copy[int(round(YPOS))-n:int(round(YPOS))+n+1,int(round(XPOS))-n:int(round(XPOS))+n+1]
indexes = np.where(frame_zoom == np.amax(frame_zoom))
ceny,cenx = indexes 


pos = coords.SkyCoord(179.689293428393,-0.454379058425512, unit=(u.deg, u.deg), frame='icrs')
im = SkyView.get_images(pos, survey='SDSSr', grid=True, gridlabels=True, pixels=str(frame_zoom.shape)[1:-1])   #, radius=15*u.arcsec)
#si lo metes con pixels, luego la imagen no hay que invertirla                   
image = im[0][0].data.copy()
headers = im[0][0].header
galaxy_center = np.where(image == np.amax(image))
galaxy_center_x = galaxy_center[1][0]
galaxy_center_y = galaxy_center[0][0]

#n = 40
#image_copy = image.copy()
#image_zoom = image_copy[galaxy_center_y-n:galaxy_center_y+n,galaxy_center_x-n:galaxy_center_x+n]
#image_zoom = resize(image_zoom,frame_zoom.shape)

#npix = round((image.shape[0] - psf.shape[0])/2) + 1

#deconvolved_image = restoration.richardson_lucy(frame_image, psf)

average_noise = abs(np.mean(frame_zoom[0:10,0:10]))
average_noise_sigma = np.std(frame_zoom[0:10,0:10])
background = average_noise
# Let's take a guess at the intrinsic properties of the galaxy.
lower_bounds = [np.amax(frame_zoom)/4,5,5,0]
p_guess = np.amax(frame_zoom)/2, 10, 10, np.pi
upper_bounds = [np.amax(frame_zoom)*2,25,25,359*np.pi/180]
# And put it all into the function minimization library.
res_lsq = least_squares(gaussian_loss_function, p_guess, bounds=(lower_bounds,upper_bounds), args=(frame_zoom, psf,rms))

#print('True parameters are', p_gal)
#print('They should match!')




#params,pcov = fit_psf(frame_zoom)
#x = np.arange(0, frame_zoom.shape[1], 1) #Stars at 0, increases by 1, goes to length of axis
#y = np.arange(0, frame_zoom.shape[0], 1) #Stars at 0, increases by 1, goes to length of axis
#xx, yy = np.meshgrid(x, y) #creates a grid to plot the 
#xycoords = (xx, yy)
#amplitude,x0,y0,sigma_x,sigma_y,theta,offset = params
offset = average_noise
amplitude = res_lsq.x[0] 
x0 = cenx 
y0 = ceny
sigma_x = res_lsq.x[1] 
sigma_y = res_lsq.x[2] 
theta = res_lsq.x[3]

offset_err = average_noise_sigma
amplitude_err = res_lsq.fun[0] 
x0_err = 0
y0_err = 0
sigma_x_err = res_lsq.fun[1] 
sigma_y_err = res_lsq.fun[2] 
theta_err = res_lsq.fun[3]
theta_err = 180*theta_err/np.pi
#theta_err = 180*pcov[5,5]**0.5/np.pi
#sigma_x_err = pcov[3,3]**0.5
#sigma_y_err = pcov[4,4]**0.5
sigma_y_sigma_x_err = ((-sigma_y*sigma_x_err/sigma_x**2)**2 + (sigma_y_err/sigma_x)**2)**0.5
if sigma_x > sigma_y:
    a = sigma_x
    a_err = sigma_x_err
    b = sigma_y
    b_err = sigma_y_err
else:
    a = sigma_y
    a_err = sigma_y_err
    b = sigma_x
    b_err = sigma_x_err
b_a = b/a
b_a_err = ((-b*a_err/a**2)**2 + (b_err/a)**2)**0.5
e = (1-b_a**2)**0.5
e_err = -b_a*b_a_err/e

#vector = twoD_Gaussian(xycoords, amplitude, x0, y0, sigma_x, sigma_y, theta, offset)
#gaussian_model_psf = vector.reshape(frame_zoom.shape[0],frame_zoom.shape[1])
gaussian_modeled_intrinsic_galaxy = Gaussian(*res_lsq.x).get_postage(51)



#print('amplitude, x0, y0, sigma_x, sigma_y, theta, offset')
#print(params)


Signal_amp = offset + amplitude
signal_noise = Signal_amp/average_noise




#posflux_name = posflux_name.reshape((-1,1))
#pos0 = int(frame_zoom.shape[0]/2)
#pos1 = int(frame_zoom.shape[1]/2)
#origin_tuple = (int(frame_zoom.shape[0]/2),int(frame_zoom.shape[1]/2))  
#PSF = EPSFModel(psf, flux=frame_zoom[pos0][pos1], x_0=pos1, y_0=pos0, normalize=True, normalization_correction=1.0, origin=origin_tuple, oversampling=1, fill_value=0.0)
#posflux_name = np.array([pos1,pos0,frame_zoom[pos0][pos1]])
#deconvolved_image_zoom = subtract_psf(frame_zoom, PSF, posflux=posflux_name)
frame_zoom = scipy.ndimage.rotate(frame_zoom, -90)
gaussian_modeled_intrinsic_galaxy = scipy.ndimage.rotate(gaussian_modeled_intrinsic_galaxy, -90)
model_observed = convolve_fft(gaussian_modeled_intrinsic_galaxy, psf, boundary='fill')
frame_zoom_model_observed_residual = frame_zoom - model_observed

#deconvolved_image_zoom = scipy.ndimage.rotate(deconvolved_image_zoom, -90) 

#n = 25
#deconvolved_image_copy = deconvolved_image.copy()
#deconvolved_image_zoom = deconvolved_image_copy[int(round(YPOS))-n:int(round(YPOS))+n,int(round(XPOS))-n:int(round(XPOS))+n]


res_lsq_direct = least_squares(direct_gaussian_fit_loss_function, p_guess, bounds=(lower_bounds,upper_bounds), args=([frame_zoom,rms]))
offset_direct = average_noise
amplitude_direct = res_lsq_direct.x[0] 
x0_direct = cenx 
y0_direct = ceny
sigma_x_direct = res_lsq_direct.x[1] 
sigma_y_direct = res_lsq_direct.x[2] 
theta_direct = res_lsq_direct.x[3]

#offset_err_direct = res_lsq_direct.fun[0]
amplitude_err_direct = average_noise_sigma 
x0_err_direct = 0
y0_err_direct = 0 
sigma_x_err_direct = res_lsq_direct.fun[1] 
sigma_y_err_direct = res_lsq_direct.fun[2] 
theta_err_direct = res_lsq_direct.fun[3]
theta_err_direct = 180*theta_err_direct/np.pi
#theta_err = 180*pcov[5,5]**0.5/np.pi
#sigma_x_err = pcov[3,3]**0.5
#sigma_y_err = pcov[4,4]**0.5
sigma_y_sigma_x_err_direct = ((-sigma_y_direct*sigma_x_err_direct/sigma_x_direct**2)**2 + (sigma_y_err_direct/sigma_x_direct)**2)**0.5
if sigma_x_direct > sigma_y_direct:
    a_direct = sigma_x_direct
    a_err_direct = sigma_x_err_direct
    b_direct = sigma_y_direct
    b_err_direct = sigma_y_err_direct
else:
    a_direct = sigma_y_direct
    a_err_direct = sigma_y_err_direct
    b_direct = sigma_x_direct
    b_err_direct = sigma_x_err_direct
b_a_direct = b_direct/a_direct
b_a_err_direct = ((-b*a_err_direct/a_direct**2)**2 + (b_err_direct/a_direct)**2)**0.5
e_direct = (1-b_a_direct**2)**0.5
e_err_direct = -b_a_direct*b_a_err_direct/e_direct



#print('amplitude, x0, y0, sigma_x, sigma_y, theta, offset')
#print(params)
theta_from_positive_y = 180*(-theta)/np.pi - 90
if sigma_y > sigma_x:
    theta_minor = theta_from_positive_y
    theta_major = theta_from_positive_y - 90
else:
    theta_minor = theta_from_positive_y - 90
    theta_major = theta_from_positive_y

theta_minor = np.fmod(theta_minor+2*360,360)    
theta_major = np.fmod(theta_major+2*360,360) 

minor_jet_angle = abs(PA_values[source] - theta_minor)
if 90 < minor_jet_angle <= 180:
    minor_jet_angle = 180 - minor_jet_angle
if 180 < minor_jet_angle <= 270:
    minor_jet_angle = minor_jet_angle - 180
if 270 < minor_jet_angle <= 360:
    minor_jet_angle = 360 - minor_jet_angle

    
l = b*2
#minor
xf1 = x0 + l*np.cos((theta_minor+90)*np.pi/180)
yf1 = y0 + l*np.sin((theta_minor+90)*np.pi/180)
xf2 = x0 + l*np.cos((theta_minor-90)*np.pi/180)
yf2 = y0 + l*np.sin((theta_minor-90)*np.pi/180)

l = a*2
#major
xf3 = x0 + l*np.cos((theta_major+90)*np.pi/180)
yf3 = y0 + l*np.sin((theta_major+90)*np.pi/180)
xf4 = x0 + l*np.cos((theta_major-90)*np.pi/180)
yf4 = y0 + l*np.sin((theta_major-90)*np.pi/180)    
    

    
jet_side_length = gaussian_modeled_intrinsic_galaxy.shape[0]
#x0_jet = int(round((jet_side_length/2)))
#y0_jet = int(round((jet_side_length/2)))

l = 5
#Jet
PA_jet_angle_from_x = (PA_values[source]+90)*np.pi/180
xf_jet = x0 + l*np.cos(PA_jet_angle_from_x)
yf_jet = y0 + l*np.sin(PA_jet_angle_from_x)


    

plt.figure()
plt.imshow(frame_image, cmap=cm.gray_r, norm=LogNorm(vmin=0.01, vmax=1))
plt.gca().invert_yaxis()
plt.title('Frame with PSF')
plt.colorbar()
plt.savefig('PSF/Observed_frame.png',dpi=900)
plt.show()


plt.figure()
plt.imshow(image, vmin=0, vmax=2)
plt.gca().invert_yaxis()
plt.title('SkyView cut-out with PSF')
plt.colorbar()
plt.savefig('PSF/Observed_skyview.png',dpi=900)
plt.show()


plt.figure()
plt.imshow(frame_zoom, vmin=0, vmax=2)
plt.gca().invert_yaxis()
plt.title('Frame cut-out with PSF')
plt.colorbar()
plt.savefig('PSF/Observed_frame_zoom.png',dpi=900)
plt.show()


plt.figure()
plt.imshow(frame_zoom-image)
plt.gca().invert_yaxis()
plt.title('Frame/Skyview cut-outs residuals with PSF')
plt.colorbar()
plt.savefig('PSF/Observed_skyview_frame_zoom_residuals.png',dpi=900)
plt.show()

print('----------------------------------------------')


plt.figure()
plt.imshow(frame_zoom, vmin=0, vmax=2)
plt.gca().invert_yaxis()
plt.title('Frame cut-out with PSF')
plt.colorbar()
plt.savefig('PSF/Observed_frame_zoom.png',dpi=900)
plt.show()


plt.figure()
plt.imshow(psf)   #, cmap=cm.gray_r, norm=LogNorm(vmin=1e-5, vmax=1))
plt.gca().invert_yaxis()
plt.title('PSF')
plt.colorbar()
plt.savefig('PSF/PSF.png',dpi=900)
plt.show()



plt.figure()
plt.imshow(gaussian_modeled_intrinsic_galaxy, vmin=0, vmax=2) #, cmap=cm.gray_r, norm=LogNorm(vmin=0.01, vmax=1))
plt.gca().invert_yaxis()
plt.title('Gaussian modeled intrinsic galaxy')
plt.colorbar()
plt.savefig('PSF/Modeled_intrinsic_frame_zoom.png',dpi=900)
plt.show()


#plt.figure()
#plt.imshow(deconvolved_image_zoom)
#plt.gca().invert_yaxis()
#plt.title('Frame cut-out without PSF')
#plt.colorbar()
#plt.savefig('PSF/Substracted.png')
#plt.show()

d = frame_zoom-gaussian_modeled_intrinsic_galaxy
plt.figure()
norm = mcolors.TwoSlopeNorm(vmin=d.min(), vmax = d.max(), vcenter=0)
plt.imshow(d, cmap=plt.cm.RdBu, norm=norm)
plt.gca().invert_yaxis()
plt.title('Observed galaxy - modeled intrinsic galaxy')
plt.colorbar()
plt.savefig('PSF/Observed_intrinsic_modeled_Frame_zoom_residuals.png',dpi=900)
plt.show()


plt.figure()
plt.imshow(model_observed, vmin=0, vmax=2)
plt.gca().invert_yaxis()
plt.title('Intrinsic modeled galaxy + PSF')
plt.colorbar()
plt.savefig('PSF/Observed_modeled_Frame_zoom.png',dpi=900)
plt.show()


plt.figure()
norm = mcolors.TwoSlopeNorm(vmin=frame_zoom_model_observed_residual.min(), vmax = frame_zoom_model_observed_residual.max(), vcenter=0)
plt.imshow(frame_zoom_model_observed_residual, cmap=plt.cm.RdBu, norm=norm)
plt.gca().invert_yaxis()
plt.title('Intrinsic modeled galaxy + PSF - Observed galaxy residuals')
plt.colorbar()
plt.savefig('PSF/Observed_Intrinsic_plus_PSF_modeled_Frame_zoom_residuals.png',dpi=900)
plt.show()

print('e observed galaxy:',e_direct)
print('e intrinsic galaxy:',e)

Chi2 = (frame_zoom_model_observed_residual**2/average_noise_sigma**2).sum()
n_x,n_y = frame_zoom.shape
n_param = 7
n_data = n_x*n_y
dof = n_data - n_param
Chi2_redu = Chi2/dof







print('Best-fit parameters are:', amplitude,sigma_x,sigma_y,180*theta/np.pi)
print('Parameter errors are:',amplitude_err,sigma_x_err,sigma_y_err,180*theta_err/np.pi)
print('Lower bounds are:',lower_bounds)
print('Initial guesses are:',p_guess)
print('Upper guesses are:',upper_bounds)

print('Chi squared =',Chi2)
print('Chi squared reduced =',Chi2_redu)

#print(np.max(image_zoom),np.max(deconvolved_image))


print('Jet PA = ',PA_values[source],'deg (from +y counterclock.)')
print('Minor PA =',round(theta_minor,2),'+/-',round(theta_err,2),'deg (from +y counterclock.)')
print('Angle between jet and minor ax. =',round(minor_jet_angle,2),'+/-',round(theta_err,2),'deg')

#deconvolved_image_zoom_log = deconvolved_image_zoom.copy()
#deconvolved_image_zoom_log[deconvolved_image_zoom_log < 0.7] = 0
plt.figure()
plt.imshow(gaussian_modeled_intrinsic_galaxy)  #, cmap=cm.gray_r, norm=LogNorm(vmin=0.01, vmax=1))
plt.contour(gaussian_modeled_intrinsic_galaxy, [0.10*np.amax(gaussian_modeled_intrinsic_galaxy),np.amax(gaussian_modeled_intrinsic_galaxy)], colors='w')
plt.plot([x0,xf1],[y0,yf1],'w-')
plt.plot([x0,xf2],[y0,yf2],'w-')
plt.plot([x0,xf3],[y0,yf3],'w-')
plt.plot([x0,xf4],[y0,yf4],'w-',label='axis')
plt.plot([x0,xf_jet],[y0,yf_jet],'r-',label='Jet')
#plt.plot(xf1,yf1,'ro',label='Minor ax. PA')
plt.gca().invert_yaxis()
#plt.colorbar()
#plt.title('Gaussian modeled intrinsic galaxy and jet')
plt.show()    

The parameters are inside the bounds. At least for this case.

Check the ellipticity between the observed galaxy and the modeled intrinsic galaxy. 

The frame images are turned 90 degrees.

Until now I was taking the optical zoomed image with the original psf imagr from SDSS. But they are probably not of the same scale! I imagine this is a problem

In [ ]:
source = np.where(Source_coords_csv['name'] == 'J0843+4537')[0][0]
real_vlbi_name = glob.glob('images\\'+PA_names[source]+'\\J*map.fits')
if len(real_vlbi_name) > 0:
    real_vlbi_name = real_vlbi_name[-1]
else:
    print(PA_names[source])
    print('Source is a variation of some other source with a letter, fits not in VLBI')
    PAS.append(np.NAN)
    

data = fits.open(real_vlbi_name)
image = data[0].data
headers = data[0].header
image = image[0][0]
PA_vlbi = headers['BPA']
pixscale_arcsec = data[0].header['CDELT2']*360
galaxy_center = np.where(image[200:301,200:301] == np.amax(image[200:301,200:301]))
galaxy_center_x = np.arange(200,301)[galaxy_center[1][0]]
galaxy_center_y = np.arange(200,301)[galaxy_center[0][0]]

image_copy = image.copy()
zoom_image = image_copy[int(galaxy_center_y)-n:int(galaxy_center_y)+n,int(galaxy_center_x)-n:int(galaxy_center_x)+n]
masked_zoom_image = zoom_image.copy()
masked_zoom_image[abs(masked_zoom_image) > 1e-3] = 10*masked_zoom_image[abs(masked_zoom_image) > 1e-3]
positions_above_zero = np.where(masked_zoom_image > 1e-3)
masked_zoom_image[positions_above_zero] = np.log10(masked_zoom_image[positions_above_zero])

x0 = n
y0 = n

l = 15
#Jet
PA_jet_angle_from_x = (PA_values[source]+90)*np.pi/180
xf_jet = x0 + l*np.cos(PA_jet_angle_from_x)
yf_jet = y0 + l*np.sin(PA_jet_angle_from_x)

RA_pix_deg = abs(headers['CDELT1'])
DEC_pix_deg = abs(headers['CDELT2'])
RA_pix_mas = RA_pix_deg*3600000
DEC_pix_mas = DEC_pix_deg*3600000

half_side_length_mas = int(round(masked_zoom_image.shape[0]*RA_pix_mas/2))
xtick_pos = [0,int(round(masked_zoom_image.shape[0]/2)),int(masked_zoom_image.shape[0])]
xtick_labels = [-half_side_length_mas,0,+half_side_length_mas]
ytick_pos = [0,int(round(masked_zoom_image.shape[0]/2)),int(masked_zoom_image.shape[0])]
ytick_labels = [-half_side_length_mas,0,+half_side_length_mas]


plt.figure() 
plt.imshow(masked_zoom_image)
plt.plot([x0,xf_jet],[y0,yf_jet],'r-',label='Jet')
ax = plt.gca()
ax.invert_yaxis()
ax.set_xticks(xtick_pos)
ax.set_xticklabels(xtick_labels)
ax.set_yticks(xtick_pos)
ax.set_yticklabels(xtick_labels)
plt.xlabel('RA (mas)')
plt.ylabel('DEC (mas)')
plt.title(PA_names[source]+' VLBI')
plt.legend()
plt.savefig('VLBI.png',dpi=1000)
plt.show()

names = np.array(Source_coords_csv['name'])
index = np.where(names == 'J0843+4537')[0][0]


pos = coords.SkyCoord(Source_coords_RA[index],Source_coords_DEC[index], unit=(u.hourangle, u.deg), frame='icrs')
im = SkyView.get_images(pos, survey='SDSSr', grid=True, gridlabels=True,  radius=45*u.arcsec)
#si lo metes con pixels, luego la imagen no hay que invertirla
image = im[0][0].data.copy()
headers = im[0][0].header
galaxy_center = np.where(image == np.amax(image))
galaxy_center_x = galaxy_center[1][0]
galaxy_center_y = galaxy_center[0][0]
n = 40
image_copy = image.copy()
image_zoom = image_copy[galaxy_center_y-n:galaxy_center_y+n,galaxy_center_x-n:galaxy_center_x+n]

half_side_length_arcsec = (image_zoom.shape[0]/image.shape[0])*45
#print('half side is',half_side_length_arcsec,'arcsec')

xtick_pos = [0,int(round(image_zoom.shape[0]/2)),int(image_zoom.shape[0])]
xtick_labels = [-half_side_length_arcsec,0,+half_side_length_arcsec]
ytick_pos = [0,int(round(image_zoom.shape[0]/2)),int(image_zoom.shape[0])]
ytick_labels = [-half_side_length_arcsec,0,+half_side_length_arcsec]

plt.figure()
plt.imshow(image_zoom)
plt.title(Source_coords_csv['name'][3456]+' Optical')
ax = plt.gca()
ax.invert_yaxis()
ax.set_xticks(xtick_pos)
ax.set_xticklabels(xtick_labels)
ax.set_yticks(ytick_pos)
ax.set_yticklabels(ytick_labels)
plt.xlabel('RA (arcsec)')
plt.ylabel('DEC (arcsec)')
plt.savefig('Optical.png',dpi=1000)
plt.show()


params,pcov = fit(image_zoom)


x = np.arange(0, image_zoom.shape[1], 1) #Stars at 0, increases by 1, goes to length of axis
y = np.arange(0, image_zoom.shape[0], 1) #Stars at 0, increases by 1, goes to length of axis
xx, yy = np.meshgrid(x, y) #creates a grid to plot the 
xycoords = (xx, yy)
amplitude,x0,y0,sigma_x,sigma_y,theta,offset = params
theta_err = 180*pcov[5,5]**0.5/np.pi
sigma_x_err = pcov[3,3]**0.5
sigma_y_err = pcov[4,4]**0.5
sigma_y_sigma_x_err = ((-sigma_y*sigma_x_err/sigma_x**2)**2 + (sigma_y_err/sigma_x)**2)**0.5
if sigma_x > sigma_y:
    a = sigma_x
    a_err = sigma_x_err
    b = sigma_y
    b_err = sigma_y_err
else:
    a = sigma_y
    a_err = sigma_y_err
    b = sigma_x
    b_err = sigma_x_err
b_a = b/a
b_a_err = ((-b*a_err/a**2)**2 + (b_err/a)**2)**0.5
e = (1-b_a**2)**0.5
e_err = -b_a*b_a_err/e

vector = twoD_Gaussian(xycoords, amplitude, x0, y0, sigma_x, sigma_y, theta, offset)
matrix = vector.reshape(image_zoom.shape[0],image_zoom.shape[1])



#print('amplitude, x0, y0, sigma_x, sigma_y, theta, offset')
#print(params)
theta_from_positive_y = 180*(-theta)/np.pi - 90
if sigma_y > sigma_x:
    theta_minor = theta_from_positive_y
    theta_major = theta_from_positive_y - 90
else:
    theta_minor = theta_from_positive_y - 90
    theta_major = theta_from_positive_y

k_minor = int(abs(theta_minor)/360) 
theta_minor = theta_minor + 360*k_minor
if 360 >= abs(theta_minor) >= 180:
    theta_minor = 360 - abs(theta_minor)

k_major = int(abs(theta_major)/360) 
theta_major = theta_major + 360*k_major
if 360 >= abs(theta_major) >= 180:
    theta_major = 360 - abs(theta_major)    
minor_jet_angle = abs(PA_values[source] - theta_minor)
if minor_jet_angle > 90:
    minor_jet_angle = 180 - minor_jet_angle

average_noise = abs(np.mean(image_zoom[0:10,0:10]))
Signal_amp = offset + amplitude
signal_noise = Signal_amp/average_noise


l = b*3
#minor
xf1 = x0 + l*np.cos((theta_minor+90)*np.pi/180)
yf1 = y0 + l*np.sin((theta_minor+90)*np.pi/180)
xf2 = x0 + l*np.cos((theta_minor-90)*np.pi/180)
yf2 = y0 + l*np.sin((theta_minor-90)*np.pi/180)

l = a*3
#major
xf3 = x0 + l*np.cos((theta_major+90)*np.pi/180)
yf3 = y0 + l*np.sin((theta_major+90)*np.pi/180)
xf4 = x0 + l*np.cos((theta_major-90)*np.pi/180)
yf4 = y0 + l*np.sin((theta_major-90)*np.pi/180)

l = 15
#Jet
PA_jet_angle_from_x = (PA_values[source]+90)*np.pi/180
xf_jet = x0 + l*np.cos(PA_jet_angle_from_x)
yf_jet = y0 + l*np.sin(PA_jet_angle_from_x)




plt.figure()
plt.imshow(image_zoom)
#plt.colorbar()
plt.contour(xx, yy, matrix, [0.10*np.amax(matrix),np.amax(matrix)], colors='w')
plt.plot([x0,xf1],[y0,yf1],'w-')
plt.plot([x0,xf2],[y0,yf2],'w-')
plt.plot([x0,xf3],[y0,yf3],'w-')
plt.plot([x0,xf4],[y0,yf4],'w-',label='axis')
#plt.plot([x0,xf_jet],[y0,yf_jet],'r-',label='Jet')
plt.plot(xf1,yf1,'ro',label='Minor ax. PA')
ax = plt.gca()
ax.invert_yaxis()
ax.set_xticks(xtick_pos)
ax.set_xticklabels(xtick_labels)
ax.set_yticks(ytick_pos)
ax.set_yticklabels(ytick_labels)
plt.xlabel('RA (arcsec)')
plt.ylabel('DEC (arcsec)')
plt.legend()
plt.title(PA_names[source]+' Gaussian fit')
plt.savefig('Fit.png',dpi=1000)
plt.show()


#plt.figure()
#plt.imshow(matrix)
#plt.colorbar()
#plt.contour(xx, yy, matrix, [0.05*np.amax(matrix),np.amax(matrix)], colors='w')
#plt.plot([x0,xf1],[y0,yf1],'w-')
#plt.plot([x0,xf2],[y0,yf2],'w-')
#plt.plot([x0,xf3],[y0,yf3],'w-')
#plt.plot([x0,xf4],[y0,yf4],'w-',label='axis')
#plt.plot([x0,xf_jet],[y0,yf_jet],'r-',label='Jet')
#plt.plot(xf1,yf1,'ro',label='Minor ax. PA')
#plt.gca().invert_yaxis()
#plt.legend()
#plt.title('Gaussian fit')
#plt.show()


plt.figure()
plt.imshow(image_zoom)
#plt.colorbar()
plt.contour(xx, yy, matrix, [0.10*np.amax(matrix),np.amax(matrix)], colors='w')
plt.plot([x0,xf1],[y0,yf1],'w-')
plt.plot([x0,xf2],[y0,yf2],'w-')
plt.plot([x0,xf3],[y0,yf3],'w-')
plt.plot([x0,xf4],[y0,yf4],'w-',label='axis')
plt.plot([x0,xf_jet],[y0,yf_jet],'r-',label='Jet')
plt.plot(xf1,yf1,'ro',label='Minor ax. PA')
ax = plt.gca()
ax.invert_yaxis()
ax.set_xticks(xtick_pos)
ax.set_xticklabels(xtick_labels)
ax.set_yticks(ytick_pos)
ax.set_yticklabels(ytick_labels)
plt.xlabel('RA (arcsec)')
plt.ylabel('DEC (arcsec)')
plt.legend()
ax.set_title(PA_names[source]+' Angle between jet and minor axis',pad=20)
plt.savefig('Angle.png',dpi=1000)
plt.show()

In [ ]:
band = 'r'
names = []
Signal_to_noise_ratio = []
pa_major = []
pa_minor = []
jet_minor_difference = []
jet_minor_difference_err = []
b_a_list = []
b_a_errs = []
ell = []
ell_errs = []
semimajor_axis = []
semimajor_axis_err = []
semiminor_axis = []
semiminor_axis_err = []
sersic_I = []
sersic_I_err = []
sersic_r = []
sersic_r_err = []
sersic_n = []
sersic_n_err = []
sersic_ellip = []
sersic_ellip_err = []
sersic_theta = []
sersic_theta_err = []
types = []



#print(np.max(image_zoom),np.max(deconvolved_image))
SDSS_array = np.array(SDSS['Source_coords_index'])

#i = 36
#for source in [SDSS_array[i]]:
for i,source in enumerate(SDSS_array):   #len(Source_coords_csv['name'])
    if np.isnan(source):
        continue
    source = int(source)
    #print(source)
    try:
        n = 40
        real_vlbi_name = glob.glob('images\\'+PA_names[source]+'\\J*map.fits')
        if len(real_vlbi_name) > 0:
            real_vlbi_name = real_vlbi_name[-1]
        else:
            print(PA_names[source])
            print('Source is a variation of some other source with a letter, fits not in VLBI')
            #PAS.append(np.NAN)
            continue

        data = fits.open(real_vlbi_name)
        vlbi_image = data[0].data
        vlbi_headers = data[0].header
        vlbi_image = vlbi_image[0][0]
        PA_vlbi = vlbi_headers['BPA']
        vlbi_pixscale_arcsec = vlbi_headers['CDELT2']
        vlbi_galaxy_center = np.where(vlbi_image[200:301,200:301] == np.amax(vlbi_image[200:301,200:301]))
        vlbi_galaxy_center_x = np.arange(200,301)[vlbi_galaxy_center[1][0]]
        vlbi_galaxy_center_y = np.arange(200,301)[vlbi_galaxy_center[0][0]]

        vlbi_image_copy = vlbi_image.copy()
        vlbi_zoom_image = vlbi_image_copy[int(vlbi_galaxy_center_y)-n:int(vlbi_galaxy_center_y)+n,int(vlbi_galaxy_center_x)-n:int(vlbi_galaxy_center_x)+n]
        vlbi_masked_zoom_image = vlbi_zoom_image.copy()
        #vlbi_masked_zoom_image[abs(vlbi_masked_zoom_image) > 1e-3] = 10*vlbi_masked_zoom_image[abs(vlbi_masked_zoom_image) > 1e-3]
        positions_above_zero = np.where(vlbi_masked_zoom_image > 1e-3)
        vlbi_masked_zoom_image[positions_above_zero] = np.log10(vlbi_masked_zoom_image[positions_above_zero])

        run = str(SDSS['run'][i])
        long_run = run.zfill(6)
        camcol = str(SDSS['camcol'][i])
        field = str(SDSS['field'][i])
        long_field = field.zfill(4)
        psfile_string = 'https://data.sdss.org/sas/dr17/eboss/photo/redux/301/{}/objcs/{}/psField-{}-{}-{}.fit'.format(run,camcol,long_run,camcol,long_field)
        psfile_data = fits.open(psfile_string)
        #print(psfile_string)
        XPOS = SDSS['colc'][i]
        YPOS = SDSS['rowc'][i]

        frame_string= 'https://data.sdss.org/sas/dr17/eboss/photoObj/frames/301/{}/{}/frame-{}-{}-{}-{}.fits.bz2'.format(run,camcol,band,long_run,camcol,long_field)
        frame = fits.open(frame_string)
        frame_image = frame[0].data
        n = 25
        frame_image_copy = frame_image.copy()
        frame_zoom = frame_image_copy[int(round(YPOS))-n:int(round(YPOS))+n+1,int(round(XPOS))-n:int(round(XPOS))+n+1]
        indexes = np.where(frame_zoom == np.amax(frame_zoom))
        #ceny,cenx = indexes
        
        
        frame_0 = frame[0]
        frame_1 = frame[1]
        frame_2 = frame[2]
        frame_3 = frame[3]

        frame_0_image = frame_0.data
        frame_1_image = frame_1.data
        frame_2_image = frame_2.data
        frame_3_image = frame_3.data

        frame_0_header = frame_0.header
        frame_1_header = frame_1.header
        frame_2_header = frame_2.header
        frame_3_header = frame_3.header
        
        #print(frame_0_header)
        EXPTIME = frame_0_header['EXPTIME']
        NMGY = frame_0_header['NMGY']
        Ref_pix_x = frame_0_header['CRPIX1']
        Ref_pix_y = frame_0_header['CRPIX2']
        Ref_RA = frame_0_header['CRVAL1']
        Ref_DEC = frame_0_header['CRVAL2']
        Pix_res_RA = frame_0_header['CD1_2']
        Pix_res_DEC = frame_0_header['CD2_1']
        

        vlbi_RA_pix_mas = vlbi_pixscale_arcsec*3600000
        vlbi_DEC_pix_mas = vlbi_pixscale_arcsec*3600000
        #XPOS = Ref_pix_x + (ra-Ref_RA)/Pix_res_RA   #abs(headers['CDELT1'])   # 
        #YPOS = Ref_pix_y + (dec-Ref_DEC)/Pix_res_DEC   #abs(headers['CDELT2'])  #  
        #RUN = frame_data0_header['RUN']
        #FRAME = frame_data0_header['FRAME']
        #CAMCOL = frame_data0_header['CAMCOL']

        atlas_string= 'https://data.sdss.org/sas/dr17/eboss/photo/redux/301/{}/objcs/{}/fpAtlas-{}-{}-{}.fit'.format(run,camcol,long_run,camcol,long_field)
        #print(atlas_string)

        atlas_data=fits.open(atlas_string)
        atlas_data0 = atlas_data[0]
        atlas_data1 = atlas_data[1]


        atlas_data0_image = atlas_data0.data
        atlas_data1_image = atlas_data1.data

        atlas_data0_header = atlas_data0.header
        atlas_data1_header = atlas_data1.header

        GAIN = float(atlas_data0_header['GAIN'].split()[2])

        
        vlbi_half_side_length_mas = int(round(vlbi_masked_zoom_image.shape[0]*vlbi_RA_pix_mas/2))
        vlbi_xtick_pos = [0,int(round(vlbi_masked_zoom_image.shape[0]/2)),int(vlbi_masked_zoom_image.shape[0])]
        vlbi_xtick_labels = [+vlbi_half_side_length_mas,0,-vlbi_half_side_length_mas]
        vlbi_ytick_pos = [0,int(round(vlbi_masked_zoom_image.shape[0]/2)),int(vlbi_masked_zoom_image.shape[0])]
        vlbi_ytick_labels = [-vlbi_half_side_length_mas,0,+vlbi_half_side_length_mas]
        
        
        Source_coords_z = np.array(Source_coords_csv['Z'])[source]
        pos = coords.SkyCoord(Source_coords_RA[source],Source_coords_DEC[source], unit=(u.hourangle, u.deg), frame='icrs')
        im = SkyView.get_images(pos, survey='SDSS'+band, grid=True, gridlabels=True, pixels=str(frame_zoom.shape)[1:-1])
        #si lo metes con pixels, luego la imagen no hay que invertirla
        image = im[0][0].data.copy()
        headers = im[0][0].header
        #print(headers)
        galaxy_center = np.where(image == np.amax(image))
        galaxy_center_x = galaxy_center[1][0]
        galaxy_center_y = galaxy_center[0][0]
        
        pixel_scale_arcsec = 0.4 #de la documentacion
        
        half_side_length_arcsec = int(image.shape[0]*pixel_scale_arcsec/2)
#print('half side is',half_side_length_arcsec,'arcsec')

        xtick_pos = [0,int(round(image.shape[0]/2)),int(image.shape[0])]
        xtick_labels = [+half_side_length_arcsec,0,-half_side_length_arcsec]
        ytick_pos = [0,int(round(image.shape[0]/2)),int(image.shape[0])]
        ytick_labels = [-half_side_length_arcsec,0,+half_side_length_arcsec]
        #ceny,cenx = galaxy_center_y,galaxy_center_x
        #n = 40
        #image_copy = image.copy()
        #image_zoom = image_copy[galaxy_center_y-n:galaxy_center_y+n,galaxy_center_x-n:galaxy_center_x+n]
    except Exception as e:
        #xtra = {'Name':[PA_names[source]],'pa_major_gaussian':[np.nan],'pa_minor_gaussian':[np.nan],'jet_minor_angle':[np.nan],'pa_gaussian_err':[np.nan],'sigma_y_sigma_x_gaussian':[np.nan],'sigma_y_sigma_x_gaussian_err':[np.nan],'b_a_gaussian':[np.nan],'b_a_gaussian_err':[np.nan],'e_gaussian':[np.nan],'e_gaussian_err':[np.nan],'Good Case':[0]}
        #df['Signal_to_noise_ratio'][source] = np.nan
        #df = df.append(pd.DataFrame(xtra),ignore_index=True)
        #df.to_csv('Gaussian.csv',index=False)
        #print(PA_names[source])
        print(e)
        print('Images/data download error')
        #Signal_to_noise_ratio.append(np.nan)
        continue
        

        
    try:
        
        psf = reconstructPSF(psfile_string, 'r', YPOS, XPOS)
        #psf = scipy.ndimage.rotate(psf,+90)
        #pos0 = int(frame_zoom.shape[0]/2)
        #pos1 = int(frame_zoom.shape[1]/2)
        #origin_tuple = (int(frame_zoom.shape[0]/2),int(frame_zoom.shape[1]/2))  
        #PSF = EPSFModel(psf, flux=frame_zoom[pos0][pos1], x_0=pos1, y_0=pos0, normalize=True, normalization_correction=1.0, origin=origin_tuple, oversampling=1, fill_value=0.0)
        #posflux_name = np.array([pos1,pos0,frame_zoom[pos0][pos1]])
        #deconvolved_image_zoom = subtract_psf(frame_zoom, PSF, posflux=posflux_name)
        #frame_zoom = scipy.ndimage.rotate(frame_zoom, -90)
        #deconvolved_image_zoom = scipy.ndimage.rotate(deconvolved_image_zoom, -90)
        n_image = image.shape[0]
        alist=[image[0,:-1], image[:-1,-1], image[-1,::-1], image[-2:0:-1,0]]
        image_rim = np.concatenate(alist)
        average_noise = abs(np.mean(image_rim))
        average_noise_sigma = np.std(image_rim)
        background = average_noise
        rms = average_noise_sigma
        # Let's take a guess at the intrinsic properties of the galaxy.
        # Let's take a guess at the intrinsic properties of the galaxy.
        lower_bounds = [0,0,0,0,0,-np.inf]
        p_guess = np.amax(image)*2,25,25, 5, 5, np.pi
        upper_bounds = [np.amax(image)*100,51,51,25,25,np.inf]
        n_param = len(lower_bounds)
        # And put it all into the function minimization library.
        res_lsq = least_squares(gaussian_loss_function, p_guess, bounds=(lower_bounds,upper_bounds), args=(image, psf,rms))
        # And put it all into the function minimization library.
        #res_lsq = least_squares(loss_function, p_guess, args=(frame_zoom, psf))
        #print('Best-fit parameters are:', res_lsq.x)
        #print('True parameters are', p_gal)
        #print('They should match!')

        lower_bounds_sersic = [0,0,0.5,0,0,0,-np.inf]
        p_guess_sersic = np.amax(image)/2,1000,4, 25, 25, 0.5,np.pi
        upper_bounds_sersic = [np.inf,np.inf,16,51,51,1,np.inf]
        n_param_sersic = len(lower_bounds_sersic)
        # And put it all into the function minimization library.
        res_lsq_sersic = least_squares(sersic_loss_function, p_guess_sersic, bounds=(lower_bounds_sersic,upper_bounds_sersic), args=(image, psf,rms))



        #params,pcov = fit_psf(frame_zoom)
        #x = np.arange(0, frame_zoom.shape[1], 1) #Stars at 0, increases by 1, goes to length of axis
        #y = np.arange(0, frame_zoom.shape[0], 1) #Stars at 0, increases by 1, goes to length of axis
        #xx, yy = np.meshgrid(x, y) #creates a grid to plot the 
        #xycoords = (xx, yy)
        #amplitude,x0,y0,sigma_x,sigma_y,theta,offset = params
        offset = average_noise
        amplitude = res_lsq.x[0] 
        x0 = res_lsq.x[1]
        y0 = res_lsq.x[2]
        sigma_x = res_lsq.x[3] 
        sigma_y = res_lsq.x[4] 
        theta = res_lsq.x[5]

        J = res_lsq.jac
        C = np.linalg.inv(J.T@J)
        
        offset_err = average_noise_sigma
        amplitude_err = np.sqrt(C[0,0])
        x0_err = np.sqrt(C[1,1])
        y0_err = np.sqrt(C[1,1]) 
        sigma_x_err = np.sqrt(C[2,2])
        sigma_y_err = np.sqrt(C[3,3]) 
        theta_err = np.sqrt(C[4,4])
        theta_err = 180*theta_err/np.pi
        
        #theta_err = 180*pcov[5,5]**0.5/np.pi
        #sigma_x_err = pcov[3,3]**0.5
        #sigma_y_err = pcov[4,4]**0.5
        
        res_lsq_direct = least_squares(direct_gaussian_fit_loss_function, p_guess, bounds=(lower_bounds,upper_bounds), args=([image,rms]))
        offset_direct = average_noise
        amplitude_direct = res_lsq_direct.x[0] 
        x0_direct = res_lsq_direct.x[1]
        y0_direct = res_lsq_direct.x[2]
        sigma_x_direct = res_lsq_direct.x[3] 
        sigma_y_direct = res_lsq_direct.x[4] 
        theta_direct = res_lsq_direct.x[5]

        J_direct = res_lsq_direct.jac
        C_direct = np.linalg.inv(J_direct.T@J_direct)
        
        offset_err_direct = average_noise_sigma
        amplitude_err_direct = np.sqrt(C_direct[0,0])
        x0_err_direct = np.sqrt(C_direct[1,1])
        y0_err_direct = np.sqrt(C_direct[2,2])
        sigma_x_err_direct = np.sqrt(C_direct[3,3]) 
        sigma_y_err_direct = np.sqrt(C_direct[4,4]) 
        theta_err_direct = np.sqrt(C_direct[5,5])
        theta_err_direct = 180*theta_err_direct/np.pi
        #theta_err = 180*pcov[5,5]**0.5/np.pi
        #sigma_x_err = pcov[3,3]**0.5
        #sigma_y_err = pcov[4,4]**0.5
        sigma_y_sigma_x_err_direct = ((-sigma_y_direct*sigma_x_err_direct/sigma_x_direct**2)**2 + (sigma_y_err_direct/sigma_x_direct)**2)**0.5
        if sigma_x_direct > sigma_y_direct:
            a_direct = sigma_x_direct
            a_err_direct = sigma_x_err_direct
            b_direct = sigma_y_direct
            b_err_direct = sigma_y_err_direct
        else:
            a_direct = sigma_y_direct
            a_err_direct = sigma_y_err_direct
            b_direct = sigma_x_direct
            b_err_direct = sigma_x_err_direct
        b_a_direct = b_direct/a_direct
        b_a_err_direct = ((-b_direct*a_err_direct/a_direct**2)**2 + (b_err_direct/a_direct)**2)**0.5
        e_direct = (1-b_a_direct**2)**0.5
        e_err_direct = abs(-b_a_direct*b_a_err_direct/e_direct)

        #npix_zoom = round((image_zoom.shape[0] - psf.shape[0])/2)
        #npix = round((image.shape[0] - psf.shape[0])/2) + 1


        #psf = resize(psf, image_zoom.shape)
        #posflux_name = np.array([pos,pos,image_zoom[pos,pos]])
        #deconvolved_image = restoration.richardson_lucy(image, psf)  #,num_iter=30)
        #posflux_name = np.array([10,10,1])
        #posflux_name = posflux_name.reshape((-1,1))
        #deconvolved_image = subtract_psf(image, psf, posflux=posflux_name)
        #plt.figure()
        #plt.imshow(deconvolved_image)
        #plt.gca().invert_yaxis()
        #plt.colorbar()
        #plt.show()


        #galaxy_center = np.where(deconvolved_image == np.amax(deconvolved_image))
        #galaxy_center_x = galaxy_center[1][0]
        #galaxy_center_y = galaxy_center[0][0]

        #n = 40
        #deconvolved_image_copy = deconvolved_image.copy()
        #deconvolved_image_zoom = deconvolved_image_copy[150-n:150+n+1,150-n:150+n+1]
        
        print('-----------------------------------------')
        print('-----------------------------------------')
        print(PA_names[source],'  ',i)
        print('Astrogeo z = {:.2f}'.format(Source_coords_z))
        
        
        
        
        #plt.figure()
        #plt.imshow(frame_image, cmap=cm.gray_r, norm=LogNorm(vmin=0.01, vmax=1))
        #plt.gca().invert_yaxis()
        #plt.title('Frame with PSF')
        #plt.colorbar()
        #plt.savefig('PSF/Original.png')
        #plt.show()


        #fig, (ax1, ax2, ax3) = plt.subplots(1, 3,figsize=(8, 8))
        #fig.tight_layout()
        
        #im = ax1.imshow(image, vmin=0, vmax=2)
        #ax1.invert_yaxis()
        #divider = make_axes_locatable(ax1)
        #cax = divider.append_axes('right', size='5%', pad=0.05)
        #ax1.set_title('SkyView cut-out with PSF')
        #plt.colorbar(im, cax=cax, orientation='vertical')
        #plt.savefig('PSF/Original.png')


        #im = ax2.imshow(frame_zoom, vmin=0, vmax=2)
        #ax2.invert_yaxis()
        #ax2.set_title('Frame cut-out with PSF')
        #divider = make_axes_locatable(ax2)
        #cax = divider.append_axes('right', size='5%', pad=0.05)
        #plt.colorbar(im, cax=cax, orientation='vertical')
        #plt.savefig('PSF/Original.png')
       
        #d = frame_zoom-image
        #norm = mcolors.TwoSlopeNorm(vmin=d.min(), vmax = d.max(), vcenter=0)
        #im = ax3.imshow(d,cmap=plt.cm.RdBu, norm=norm)
        #ax3.invert_yaxis()
        #divider = make_axes_locatable(ax3)
        #cax = divider.append_axes('right', size='5%', pad=0.05)
        #ax3.set_title('Residuals')
        #plt.savefig('PSF/Original.png')
        #plt.colorbar(im, cax=cax, orientation='vertical')
        
        #fig.show()

        #print('----------------------------------------------')


       



        #plt.figure()
        #plt.imshow(deconvolved_image_zoom) #, cmap=cm.gray_r, norm=LogNorm(vmin=0.01, vmax=1))
        #plt.gca().invert_yaxis()
        #plt.title('Frame cut-out without PSF')
        #plt.colorbar()
        #plt.savefig('PSF/Substracted.png')
        #plt.show()
        
        #plt.figure()
        #plt.imshow(frame_zoom-deconvolved_image_zoom)
        #plt.gca().invert_yaxis()
        #plt.title('PSF/Frame cut-out residuals')
        #plt.colorbar()
        #plt.savefig('PSF/Residuals.png')
        #plt.show()
               
        
    
        #params,pcov = fit(deconvolved_image_zoom)
        #xtra = {'Name':[PA_names[source]],'pa_major_gaussian':[np.nan],'pa_minor_gaussian':[np.nan],'jet_minor_angle':[np.nan],'pa_gaussian_err':[np.nan],'sigma_y_sigma_x_gaussian':[np.nan],'sigma_y_sigma_x_gaussian_err':[np.nan],'b_a_gaussian':[np.nan],'b_a_gaussian_err':[np.nan],'e_gaussian':[np.nan],'e_gaussian_err':[np.nan],'Good Case':[0]}
        #df['Signal_to_noise_ratio'][source] = np.nan
        #df = df.append(pd.DataFrame(xtra),ignore_index=True)
        #df.to_csv('Gaussian.csv',index=False)
        #print(PA_names[source])
        #print('Fitting failed')
        #Signal_to_noise_ratio.append(np.nan)
        #continue
        #x = np.arange(0, deconvolved_image_zoom.shape[1], 1) #Stars at 0, increases by 1, goes to length of axis
        #y = np.arange(0, deconvolved_image_zoom.shape[0], 1) #Stars at 0, increases by 1, goes to length of axis
        #xx, yy = np.meshgrid(x, y) #creates a grid to plot the 
        #xycoords = (xx, yy)
        #amplitude,x0,y0,sigma_x,sigma_y,theta,offset = params
        #theta_err = 180*pcov[5,5]**0.5/np.pi
        #sigma_x_err = pcov[3,3]**0.5
        #sigma_y_err = pcov[4,4]**0.5
        sigma_y_sigma_x_err = ((-sigma_y*sigma_x_err/sigma_x**2)**2 + (sigma_y_err/sigma_x)**2)**0.5
        if sigma_x > sigma_y:
            a = sigma_x
            a_err = sigma_x_err
            b = sigma_y
            b_err = sigma_y_err
        else:
            a = sigma_y
            a_err = sigma_y_err
            b = sigma_x
            b_err = sigma_x_err
        b_a = b/a
        b_a_err = ((-b*a_err/a**2)**2 + (b_err/a)**2)**0.5
        e = (1-b_a**2)**0.5
        e_err = abs(-b_a*b_a_err/e)
        
        
        
        
        I_e = res_lsq_sersic.x[0]
        r_eff = res_lsq_sersic.x[1]
        n = res_lsq_sersic.x[2]
        x0_sersic = res_lsq_sersic.x[3]
        y0_sersic = res_lsq_sersic.x[4]
        ellip_sersic = res_lsq_sersic.x[4] 
        theta_sersic = res_lsq_sersic.x[5] 

        J_sersic = res_lsq_sersic.jac
        C_sersic = np.linalg.inv(J_sersic.T@J_sersic)
        
        I_e_err = np.sqrt(C_sersic[0,0])
        r_eff_err = np.sqrt(C_sersic[1,1])
        n_err = np.sqrt(C_sersic[1,1]) 
        x0_sersic_err = np.sqrt(C_sersic[2,2])
        y0_sersic_err = np.sqrt(C_sersic[3,3]) 
        ellip_sersic_err = np.sqrt(C_sersic[4,4])
        theta_sersic_err = np.sqrt(C_sersic[5,5])
        theta_sersic_err = 180*theta_sersic_err/np.pi
        
        
        
        gaussian_modeled_intrinsic_galaxy = Gaussian(*res_lsq.x).get_postage(51)
        #vector = twoD_Gaussian(xycoords, amplitude, x0, y0, sigma_x, sigma_y, theta, offset)
        #matrix = vector.reshape(deconvolved_image_zoom.shape[0],deconvolved_image_zoom.shape[1])

        sersic_modeled_intrinsic_galaxy = Sersic(*res_lsq_sersic.x).get_postage(51)
        
        #print('amplitude, x0, y0, sigma_x, sigma_y, theta, offset')
        #print(params)
        theta_from_positive_y = 180*(theta)/np.pi - 90
        if sigma_y > sigma_x:
            theta_minor = theta_from_positive_y
            theta_major = theta_from_positive_y - 90
        else:
            theta_minor = theta_from_positive_y - 90
            theta_major = theta_from_positive_y
        
        theta_minor = np.fmod(theta_minor+360,360)
            
        theta_major = np.fmod(theta_major+360,360)
        
        minor_jet_angle = abs(PA_values[source] - theta_minor)
        if 360 < minor_jet_angle:
            minor_jet_angle = np.fmod(minor_jet_angle,360)    
        if 90 < minor_jet_angle <= 180:
            minor_jet_angle = 180 - minor_jet_angle
        if 180 < minor_jet_angle <= 270:
            minor_jet_angle = minor_jet_angle - 180
        if 270 < minor_jet_angle <= 360:
            minor_jet_angle = 360 - minor_jet_angle
            
            
        
        theta_from_positive_y_direct = 180*(theta_direct)/np.pi - 90
        if sigma_y_direct > sigma_x_direct:
            theta_minor_direct = theta_from_positive_y_direct
            theta_major_direct = theta_from_positive_y_direct - 90
        else:
            theta_minor_direct = theta_from_positive_y_direct - 90
            theta_major_direct = theta_from_positive_y_direct
        
        theta_minor_direct = np.fmod(theta_minor_direct+360,360)
            
        theta_major_direct = np.fmod(theta_major_direct+360,360)
        
        minor_jet_angle_direct = abs(PA_values[source] - theta_minor_direct)
        if 360 < minor_jet_angle_direct:
            minor_jet_angle_direct = np.fmod(minor_jet_angle_direct,360)
        if 90 < minor_jet_angle_direct <= 180:
            minor_jet_angle_direct = 180 - minor_jet_angle_direct
        if 180 < minor_jet_angle_direct <= 270:
            minor_jet_angle_direct = minor_jet_angle_direct - 180
        if 270 < minor_jet_angle_direct <= 360:
            minor_jet_angle_direct = 360 - minor_jet_angle_direct
        
        
        Signal_amp = offset + amplitude
        signal_noise = Signal_amp/average_noise
        
        model_observed = convolve_fft(gaussian_modeled_intrinsic_galaxy, psf, boundary='fill')
        model_observed_sersic = convolve_fft(sersic_modeled_intrinsic_galaxy, psf, boundary='fill')
        #frame_zoom = scipy.ndimage.rotate(frame_zoom, -90)
        #gaussian_modeled_intrinsic_galaxy = scipy.ndimage.rotate(gaussian_modeled_intrinsic_galaxy, -90)
        #model_observed = scipy.ndimage.rotate(model_observed,-90)
        image_model_observed_residual = image - model_observed
        image_model_observed_residual_sersic = image - model_observed_sersic
        #if signal_noise < 15:
            #pa_major_gaussian.append(np.nan)
            #pa_minor_gaussian.append(np.nan)
            #jet_minor_angle.append(np.nan)
            #pa_gaussian_err.append(np.nan)
            #sigma_y_sigma_x_gaussian.append(np.nan)
            #sigma_y_sigma_x_gaussian_err.append(np.nan)
            #b_a_gaussian.append(np.nan)
            #b_a_gaussian_err.append(np.nan)
            #e_gaussian.append(np.nan)
            #e_gaussian_err.append(np.nan)
            
            #print(PA_names[source])
            #print('Low S/N')
            #continue
        
        l = b*2
        #minor
        xf1 = x0 + l*np.cos((theta_minor+90)*np.pi/180)
        yf1 = y0 + l*np.sin((theta_minor+90)*np.pi/180)
        xf2 = x0 + l*np.cos((theta_minor-90)*np.pi/180)
        yf2 = y0 + l*np.sin((theta_minor-90)*np.pi/180)
       
        l = a*2
        #major
        xf3 = x0 + l*np.cos((theta_major+90)*np.pi/180)
        yf3 = y0 + l*np.sin((theta_major+90)*np.pi/180)
        xf4 = x0 + l*np.cos((theta_major-90)*np.pi/180)
        yf4 = y0 + l*np.sin((theta_major-90)*np.pi/180)
        
        
        
        
        #if b_a > 0.7 or signal_noise < 50:
            #xtra = {'Name':[PA_names[source]],'pa_major_gaussian':[theta_major],'pa_minor_gaussian':[theta_minor],'jet_minor_angle':[minor_jet_angle],'pa_gaussian_err':[theta_err],'sigma_y_sigma_x_gaussian':[sigma_y/sigma_x],'sigma_y_sigma_x_gaussian_err':[sigma_y_sigma_x_err],'b_a_gaussian':[b_a],'b_a_gaussian_err':[b_a_err],'e_gaussian':[e],'e_gaussian_err':[e_err],'Good Case':[0]}
            #df['Signal_to_noise_ratio'][source] = signal_noise
            #df = df.append(pd.DataFrame(xtra),ignore_index=True)
            #df.to_csv('Gaussian.csv',index=False)
            #Signal_to_noise_ratio.append(signal_noise)
            #continue
            
        
        names.append(PA_names[source])
        Signal_to_noise_ratio.append(signal_noise)
        pa_major.append(theta_major)
        pa_minor.append(theta_minor)
        jet_minor_difference.append(minor_jet_angle)
        jet_minor_difference_err.append(theta_err)
        b_a_list.append(b/a)
        b_a_errs.append(b_a_err)
        ell.append(e)
        ell_errs.append(e_err)
        semimajor_axis.append(a)
        semimajor_axis_err.append(a_err)
        semiminor_axis.append(b)
        semiminor_axis_err.append(b_err)
        sersic_I.append(I_e)
        sersic_I_err.append(I_e_err)
        sersic_r.append(r_eff)
        sersic_r_err.append(r_eff_err)
        sersic_n.append(n)
        sersic_n_err.append(n_err)
        sersic_ellip.append(ellip_sersic)
        sersic_ellip_err.append(ellip_sersic_err)
        sersic_theta.append(theta_sersic)
        sersic_theta_err.append(theta_sersic_err)
        types.append(SDSS.type[i])
            

        #run = headers[-3][46:52]
        #camcol = headers[-3][53]
        #field = headers[-3][55:59]
        #dec = headers['CRVAL2']   #float(headers[-43][10:])
        #ra = headers['CRVAL1'] #float(headers[-44][10:])
        

            
        #xtra = {'Name':[PA_names[source]],'pa_major_gaussian':[theta_major],'pa_minor_gaussian':[theta_minor],'jet_minor_angle':[minor_jet_angle],'pa_gaussian_err':[theta_err],'sigma_y_sigma_x_gaussian':[sigma_y/sigma_x],'sigma_y_sigma_x_gaussian_err':[sigma_y_sigma_x_err],'b_a_gaussian':[b_a],'b_a_gaussian_err':[b_a_err],'e_gaussian':[e],'e_gaussian_err':[e_err],'Good Case':[1]}
        #df['Signal_to_noise_ratio'] = signal_noise
        #Signal_to_noise_ratio.append(signal_noise)
        #pa_minor.append(minor_jet_angle)
        #df = df.append(pd.DataFrame(xtra),ignore_index=True)
        #df.to_csv('Gaussian.csv',index=False)
        #print(PA_names[source])
        #print('Average_Noise =',round(average_noise,2))
        #print('Signal amplitude =',round(Signal_amp,2))
        
        
        
        
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3,figsize=(10, 10))
        
        
        im = ax1.imshow(image, vmin=0, vmax=2)
        ax1.invert_yaxis()
        ax1.set_xticks(xtick_pos)
        ax1.set_xticklabels(xtick_labels)
        ax1.set_yticks(ytick_pos)
        ax1.set_yticklabels(ytick_labels)
        ax1.set_xlabel('RA (arcsec)')
        ax1.set_ylabel('DEC (arcsec)')
        divider = make_axes_locatable(ax1)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        textstr1 = '\n'.join((
        r'$Peak=%.2f \pm %.2f Nmg$' % (amplitude_direct, amplitude_err_direct),
        r'$\theta_M=%.0f\degree \pm %.2f\degree$' % (theta_major_direct,abs(theta_err_direct))))
        textstr2 = '\n'.join((
        r'$\sigma_{x}=%.2f \pm %.2f px$' % (sigma_x_direct, sigma_x_err_direct),
        r'$\sigma_{y}=%.2f \pm %.2f px$' % (sigma_y_direct, sigma_y_err_direct)))
        
        props = dict(boxstyle='round', facecolor='wheat', alpha=1)
        #ax1.text(0.05, 0.95, textstr1, transform=ax1.transAxes, fontsize=8,
        #verticalalignment='top', bbox=props)
        #ax1.text(0.05, 0.25, textstr2, transform=ax1.transAxes, fontsize=8,
        #verticalalignment='top', bbox=props)
        ax1.set_title('J0015+0828 Optical host-galaxy')
        plt.colorbar(im, cax=cax, orientation='vertical')
        ax1.set_aspect(1)
        #plt.savefig('PSF/Original.png')

        
        im = ax2.imshow(psf)
        ax2.invert_yaxis()
        divider = make_axes_locatable(ax2)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        ax2.set_title('PSF')
        ax2.set_aspect(1)
        plt.colorbar(im, cax=cax, orientation='vertical')
        
        
        im = ax3.imshow(gaussian_modeled_intrinsic_galaxy, vmin=0, vmax=2)
        ax3.invert_yaxis()
        ax3.set_xticks(xtick_pos)
        ax3.set_xticklabels(xtick_labels)
        ax3.set_yticks(ytick_pos)
        ax3.set_yticklabels(ytick_labels)
        ax3.set_xlabel('RA (arcsec)')
        ax3.set_ylabel('DEC (arcsec)')
        divider = make_axes_locatable(ax3)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        textstr1 = '\n'.join((
        r'$Peak=%.2f \pm %.2f Nmg$' % (amplitude, amplitude_err),
        r'$\theta_M=%.0f\degree \pm %.2f\degree$' % (theta_major,abs(theta_err))))
        textstr2 = '\n'.join((
        r'$\sigma_{x}=%.2f \pm %.2f px$' % (sigma_x, sigma_x_err),
        r'$\sigma_{y}=%.2f \pm %.2f px$' % (sigma_y, sigma_y_err)))
        
        props = dict(boxstyle='round', facecolor='wheat', alpha=1)
        #ax3.text(0.05, 0.95, textstr1, transform=ax3.transAxes, fontsize=8,
        #verticalalignment='top', bbox=props)
        #ax3.text(0.05, 0.25, textstr2, transform=ax3.transAxes, fontsize=8,
        #verticalalignment='top', bbox=props)
        ax3.set_title('J0015+0828 Gaussian best fit model')
        plt.colorbar(im, cax=cax, orientation='vertical')
        ax3.set_aspect(1)
        fig.tight_layout()
        #plt.savefig('Kyle_fits/J0009+1803_1.png',facecolor='w',dpi=900)
        plt.show()
        
        
        
        #plt.figure()
        #d = image-gaussian_modeled_intrinsic_galaxy
        #norm = mcolors.TwoSlopeNorm(vmin=d.min(), vmax = d.max(), vcenter=0)
        #plt.imshow(d, cmap=plt.cm.RdBu, norm=norm)
        #plt.gca().invert_yaxis()
        #plt.title('Observed galaxy with PSF - modeled galaxy without PSF')
        #plt.colorbar()
        #plt.savefig('PSF/Observed_intrinsic_modeled_Frame_zoom_residuals_small.png',dpi=900)
        #plt.show()


        fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(8, 8))
        
        
        im = ax1.imshow(model_observed, vmin=0, vmax=2)
        ax1.invert_yaxis()
        divider = make_axes_locatable(ax1)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        ax1.set_title('Model with PSF')
        plt.colorbar(im, cax=cax, orientation='vertical')
        ax1.set_aspect(1)
        
        
        norm = mcolors.TwoSlopeNorm(vmin=image_model_observed_residual.min(), vmax = image_model_observed_residual.max(), vcenter=0)
        im = ax2.imshow(image_model_observed_residual, cmap=plt.cm.RdBu, norm=norm)
        divider = make_axes_locatable(ax2)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        ax2.invert_yaxis()
        ax2.set_title('Model with PSF vs. Skyview with PSF residuals')
        plt.colorbar(im, cax=cax, orientation='vertical')
        #plt.savefig('PSF/Observed_Intrinsic_plus_PSF_modeled_Frame_zoom_residuals_small.png',dpi=900)
        ax2.set_aspect(1)
        fig.tight_layout()
        #plt.savefig('Kyle_fits/J0009+1803_2.png',facecolor='w',dpi=900)
        plt.show()
        
        
        
        print('S/N =',signal_noise)
        print('Jet PA = ',PA_values[source],'deg (from +y counterclock.)')
        print('Minor PA =',round(theta_minor,2),'+/-',round(theta_err,2),'deg (from +y counterclock.)')
        print('Angle between jet and minor ax. =',round(minor_jet_angle,2),'+/-',round(theta_err,2),'deg')
        print('b/a =',round(b/a,2),'+/-',round(b_a_err,2))
        print('e =',round(e,2),'+/-',round(e_err,2))
        
        
        
        
        
        
        
        
        fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(8, 8))
        
        
        #Jet VLBI image
        jet_side_length = vlbi_masked_zoom_image.shape[0]
        x0_jet = int(round((jet_side_length/2)))
        y0_jet = int(round((jet_side_length/2)))
        l = 15
        PA_jet_angle_from_x = (PA_values[source]+90)*np.pi/180
        xf_jet = x0_jet + l*np.cos(PA_jet_angle_from_x)
        yf_jet = y0_jet + l*np.sin(PA_jet_angle_from_x)
        
        im = ax1.imshow(vlbi_masked_zoom_image)
        ax1.invert_yaxis()
        
        ax1.set_xticks(vlbi_xtick_pos)
        ax1.set_xticklabels(vlbi_xtick_labels)
        ax1.set_yticks(vlbi_ytick_pos)
        ax1.set_yticklabels(vlbi_ytick_labels)
        ax1.set_xlabel('RA (mas)')
        ax1.set_ylabel('DEC (mas)')
        ax1.plot([x0_jet,xf_jet],[y0_jet,yf_jet],'r-',label='jet')
        #plt.colorbar()
        
        ax1.legend()
        ax1.set_title('J0015+0828 VLBI jet')
        ax1.set_aspect(1)
        
        
        #Jet in model image
        jet_side_length = gaussian_modeled_intrinsic_galaxy.shape[0]
        #x0_jet = int(round((jet_side_length/2)))
        #y0_jet = int(round((jet_side_length/2)))
        l = 5
        PA_jet_angle_from_x = (PA_values[source]+90)*np.pi/180
        xf_jet = x0 + l*np.cos(PA_jet_angle_from_x)
        yf_jet = y0 + l*np.sin(PA_jet_angle_from_x)
        
        #deconvolved_image_zoom_log = deconvolved_image_zoom.copy()
        #deconvolved_image_zoom_log[deconvolved_image_zoom_log < 0.7] = 0
        im = ax2.imshow(gaussian_modeled_intrinsic_galaxy,vmin=0,vmax=2)  #, cmap=cm.gray_r, norm=LogNorm(vmin=0.01, vmax=1))
        divider = make_axes_locatable(ax2)
        cax = divider.append_axes('right', size='5%', pad=0.05)
        ax2.contour(gaussian_modeled_intrinsic_galaxy, [0.10*np.amax(gaussian_modeled_intrinsic_galaxy),np.amax(gaussian_modeled_intrinsic_galaxy)], colors='w')
        ax2.plot([x0,xf1],[y0,yf1],'w:')
        ax2.plot([x0,xf2],[y0,yf2],'w:')
        ax2.plot([x0,xf3],[y0,yf3],'w-')
        ax2.plot([x0,xf4],[y0,yf4],'w-',label='axis')
        ax2.plot([x0,xf_jet],[y0,yf_jet],'r-',label='Jet')
        #plt.plot(xf1,yf1,'ro',label='Minor ax. PA')
        ax2.invert_yaxis()
        ax2.set_xticks(xtick_pos)
        ax2.set_xticklabels(xtick_labels)
        ax2.set_yticks(ytick_pos)
        ax2.set_yticklabels(ytick_labels)
        ax2.set_xlabel('RA (arcsec)')
        ax2.set_ylabel('DEC (arcsec)')
        plt.colorbar(im,cax=cax,orientation='vertical')
        #textstr1 = '\n'.join((
        #r'Jet PA = {:.0f}$\degree$ (from +y counterclock.)'.format(PA_values[source]),
        #r'Minor axis PA = {:.0f}$\degree$ $\pm$ {:.0f}$\degree$ (from +y counterclock.)'.format(theta_minor,abs(theta_err))))
        textstr2 = r'Angle between jet and minor ax. = {:.0f}$\degree$ $\pm$ {:.0f}$\degree$'.format(minor_jet_angle,theta_err)
        
        props = dict(boxstyle='round', facecolor='wheat', alpha=1)
        #ax2.text(0.02, 0.95, textstr1, transform=ax2.transAxes, fontsize=8,
        #verticalalignment='top', bbox=props)
        ax2.text(0.05, 0.10, textstr2, transform=ax2.transAxes, fontsize=8,
        verticalalignment='top', bbox=props)
        ax2.set_title('J0015+0828 Angle between jet and minor axis')
        ax2.set_aspect(1)
        fig.tight_layout()
        plt.show()
        
        
        plt.figure()
        plt.imshow(sersic_modeled_intrinsic_galaxy)
        plt.gca().invert_yaxis()
        plt.colorbar()
        plt.title('Best fit Sérsic model')
        plt.show()
        
        plt.figure()
        plt.imshow(model_observed_sersic)
        plt.gca().invert_yaxis()
        plt.colorbar()
        plt.title('Best fit Sérsic model+PSF')
        plt.show()
        
        
        norm = mcolors.TwoSlopeNorm(vmin=image_model_observed_residual_sersic.min(), vmax = image_model_observed_residual_sersic.max(), vcenter=0)
        plt.figure()
        im = plt.imshow(image_model_observed_residual_sersic, cmap=plt.cm.RdBu, norm=norm)
        divider = make_axes_locatable(plt.gca())
        cax = divider.append_axes('right', size='5%', pad=0.05)
        plt.gca().invert_yaxis()
        plt.colorbar(im, cax=cax, orientation='vertical')
        plt.title('Best fit Sérsic model+PSF - Obseved galaxy')
        plt.show()
        
        
        

        print('e observed galaxy:',e_direct)
        print('e model galaxy:',e)

        Chi2 = (image_model_observed_residual**2/average_noise_sigma**2).sum()
        n_x,n_y = image.shape
        n_data = n_x*n_y
        dof = n_data - n_param
        Chi2_redu = Chi2/dof
        
        Chi2_sersic = (image_model_observed_residual_sersic**2/average_noise_sigma**2).sum()
        n_x,n_y = image.shape
        n_data = n_x*n_y
        dof = n_data - n_param_sersic
        Chi2_redu_sersic = Chi2_sersic/dof

        
        print('Best-fit parameters are:', amplitude,sigma_x,sigma_y,180*theta/np.pi)
        print('Parameter errors are:',amplitude_err,sigma_x_err,sigma_y_err,theta_err)
        print('Lower bounds are:',lower_bounds)
        print('Initial guesses are:',p_guess)
        print('Upper guesses are:',upper_bounds)
        #print('Chi squared =',Chi2)
        print('Chi squared reduced =',Chi2_redu)
        
        print('')
        
        print('Sérsic best-fit parameters are:', I_e,r_eff,n,x0_sersic,y0_sersic,ellip_sersic,180*theta_sersic/np.pi)
        print('Parameter errors are:',I_e_err,r_eff_err,n_err,x0_sersic_err,y0_sersic_err,ellip_sersic_err,theta_sersic_err)
        print('Chi squared reduced Sérsic =',Chi2_redu_sersic)
        
        
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)
        #xtra = {'Name':[PA_names[source]],'pa_major_gaussian':[np.nan],'pa_minor_gaussian':[np.nan],'jet_minor_angle':[np.nan],'pa_gaussian_err':[np.nan],'sigma_y_sigma_x_gaussian':[np.nan],'sigma_y_sigma_x_gaussian_err':[np.nan],'b_a_gaussian':[np.nan],'b_a_gaussian_err':[np.nan],'e_gaussian':[np.nan],'e_gaussian_err':[np.nan],'Good Case':[0]}
        #df['Signal_to_noise_ratio'][source] = np.nan
        #df = df.append(pd.DataFrame(xtra),ignore_index=True)
        #df.to_csv('Gaussian.csv',index=False)
        #print(PA_names[source])
        #print(e)
        #Signal_to_noise_ratio.append(signal_noise)
        continue
        
             
#data = {'Name':names,'Signal_to_noise_ratio':Signal_to_noise_ratio,'pa_major':pa_major,'pa_minor':pa_minor,'jet_minor_difference':jet_minor_difference,'jet_minor_difference_err':jet_minor_difference_err,'b_a_list':b_a_list,'b_a_errs':b_a_errs,'ell':ell,'ell_errs':ell_errs,'a':semimajor_axis,'a_err':semimajor_axis_err,'b':semiminor_axis,'b_err':semiminor_axis_err,'sersic_I':sersic_I,'sersic_I_err':sersic_I_err,'sersic_r':sersic_r,'sersic_r_err':sersic_r_err,'sersic_n':sersic_n,'sersic_n_err':sersic_n_err,'sersic_ellip':sersic_ellip,'sersic_ellip_err':sersic_ellip_err,'sersic_theta':sersic_theta,'sersic_theta_err':sersic_theta_err,'type_SDSS':types}
#SDSS_new_results = pd.DataFrame(data)
#SDSS_new_results.to_csv('SDSS_new_results.csv',index=False)  

In [ ]:

#SDSS_results = pd.read_csv('SDSS_results.csv')
#SDSS_results['Name'] = names
#SDSS_results['Signal_to_noise_ratio'] = Signal_to_noise_ratio
#SDSS_results['pa_major'] = pa_major
#SDSS_results['pa_minor'] = pa_minor
#SDSS_results['jet_minor_difference'] = jet_minor_difference
#SDSS_results['jet_minor_difference_err'] = jet_minor_difference_err
#SDSS_results['b_a_list'] = b_a_list
#SDSS_results['b_a_errs'] = b_a_errs
#SDSS_results['ell'] = ell
#SDSS_results['ell_errs'] = ell_errs

In [ ]:
SDSS_new_results = pd.read_csv('SDSS_new_results.csv',header=0)
SDSS_new_results

In [ ]:
#for a,b in zip(SDSS['SDSS names'][0:1880],SDSS_new_results['Name']):
    #print(a,b)

In [ ]:
#raw = fits.open('Kyle_fits/J0009+0507_raw.fits')
#raw_image = raw[0].data

#PSF = fits.open('Kyle_fits/J0009+0507_PSF.fits')
#PSF_image = PSF[0].data


#plt.figure()
#plt.imshow(raw_image)
#plt.show()

#plt.figure()
#plt.imshow(PSF_image)
#plt.show()

In [ ]:
a = 10
b_as = np.linspace(0,1,11)
fig,ax = plt.subplots(4,3,figsize=(10,10))
for i,b_a in enumerate(b_as):
    ax[i//3,i%3].imshow(Gaussian(1,a,b_a*a,0).get_postage(51))
    ax[i//3,i%3].set_title('b/a = {:.1f}'.format(b_a))

fig.tight_layout()

In [ ]:
plt.figure()
plt.hist(jet_minor_difference_high_ell,bins=10,color='blue',label='b/a < 0.7, S/N > 50',alpha=0.8)
plt.xlabel('Jet-minor axis angle difference (deg)')
plt.legend()
plt.show()

In [ ]:
#data = {'Name':names,'Signal_to_noise_ratio':Signal_to_noise_ratio,'pa_major':pa_major,'pa_minor':pa_minor,'jet_minor_difference':jet_minor_difference,'jet_minor_difference_err':jet_minor_difference_err,'b_a_list':b_a_list,'b_a_errs':b_a_errs,'ell':ell,'ell_errs':ell_errs}
#df = pd.DataFrame(data)
#df.to_csv('SDSS_results.csv',index=False)

This is for the cases with S/N > 50 and b/a < 0.7

In [ ]:
source = int(SDSS['Source_coords_index'][0])
pos = coords.SkyCoord(Source_coords_RA[source],Source_coords_DEC[source], unit=(u.hourangle, u.deg), frame='icrs')
im = SkyView.get_images(pos, survey='SDSSr', grid=True, gridlabels=True,  radius=30*u.arcsec)

image = im[0][0].data
headers = im[0][0].header

run = headers[-3][46:52]
camcol = headers[-3][53]
field = headers[-3][55:59]
dec = headers['CRVAL2']   #float(headers[-43][10:])
ra = headers['CRVAL1'] #float(headers[-44][10:])
print(run,camcol,field)
short_run = ''
for e in run:
    if e != '0':
        short_run = short_run + e
frame_string= 'https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/{}/{}/frame-{}-{}-{}-{}.fits.bz2'.format(short_run,camcol,band,run,camcol,field)
print(frame_string)
frame_data=fits.open(frame_string)
frame_data0 = frame_data[0]
frame_data1 = frame_data[1]
frame_data2 = frame_data[2]
frame_data3 = frame_data[3]

frame_data0_image = frame_data0.data
frame_data1_image = frame_data1.data
frame_data2_image = frame_data2.data
frame_data3_image = frame_data3.data

frame_data0_header = frame_data0.header
frame_data1_header = frame_data1.header
frame_data2_header = frame_data2.header
frame_data3_header = frame_data3.header

EXPTIME = frame_data0_header['EXPTIME']
NMGY = frame_data0_header['NMGY']
Ref_pix_x = frame_data0_header['CRPIX1']
Ref_pix_y = frame_data0_header['CRPIX2']
Ref_RA = frame_data0_header['CRVAL1']
Ref_DEC = frame_data0_header['CRVAL2']
Pix_res_RA = frame_data0_header['CD1_2']
Pix_res_DEC = frame_data0_header['CD2_1']
XPOS = Ref_pix_x + (ra-Ref_RA)/Pix_res_RA   #abs(headers['CDELT1'])   # 
YPOS = Ref_pix_y + (dec-Ref_DEC)/Pix_res_DEC   #abs(headers['CDELT2'])  #  
RUN = frame_data0_header['RUN']
FRAME = frame_data0_header['FRAME']
CAMCOL = frame_data0_header['CAMCOL']

atlas_string= 'https://data.sdss.org/sas/dr17/eboss/photo/redux/301/{}/objcs/{}/fpAtlas-{}-{}-{}.fit'.format(short_run,camcol,run,camcol,field)
print(atlas_string)

atlas_data=fits.open(atlas_string)
atlas_data0 = atlas_data[0]
atlas_data1 = atlas_data[1]


atlas_data0_image = atlas_data0.data
atlas_data1_image = atlas_data1.data

atlas_data0_header = atlas_data0.header
atlas_data1_header = atlas_data1.header

GAIN = float(atlas_data0_header['GAIN'].split()[2])

psfile_string = 'https://data.sdss.org/sas/dr17/eboss/photo/redux/301/{}/objcs/{}/psField-{}-{}-{}.fit'.format(short_run,camcol,run,camcol,field)
print(psfile_string)
psfile_data = fits.open(psfile_string)

psf = reconstructPSF(psfile_string, band, YPOS, XPOS)



#npix_zoom = round((image_zoom.shape[0] - psf.shape[0])/2)
npix = round((image.shape[0] - psf.shape[0])/2) + 1


plt.figure()
plt.imshow(image)     #, cmap=cm.gray_r, norm=LogNorm(vmin=0.01, vmax=1))
plt.gca().invert_yaxis()
plt.colorbar()
plt.show()

psf = np.pad(psf,[(npix,npix),(npix,npix)],mode='constant')

plt.figure()
plt.imshow(psf)
plt.gca().invert_yaxis()
plt.colorbar()
plt.show()


deconvolved_image = restoration.richardson_lucy(image, psf)  #,num_iter=30)

plt.figure()
plt.imshow(deconvolved_image)
plt.gca().invert_yaxis()
plt.colorbar()
plt.show()


#galaxy_center = np.where(deconvolved_image == np.amax(deconvolved_image))
#galaxy_center_x = galaxy_center[1][0]
#galaxy_center_y = galaxy_center[0][0]

n = 40
deconvolved_image_copy = deconvolved_image.copy()
deconvolved_image_zoom = deconvolved_image_copy[150-n:150+n+1,150-n:150+n+1]

plt.figure()
plt.imshow(deconvolved_image_zoom)
plt.gca().invert_yaxis()
plt.colorbar()
plt.show()

In [ ]:
frame1 = fits.open('https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/5598/6/frame-r-005598-6-0135.fits.bz2')
frame2 = fits.open('https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/5636/6/frame-r-005636-6-0141.fits.bz2')
frame3 = fits.open('https://dr12.sdss.org/sas/dr12/boss/photoObj/frames/301/5641/6/frame-r-005641-6-0017.fits.bz2')

frame1_image = frame1[0].data
frame2_image = frame2[0].data
frame3_image = frame3[0].data


plt.figure()
plt.imshow(frame1_image, cmap=cm.gray_r, norm=LogNorm(vmin=0.01, vmax=1))
plt.gca().invert_yaxis()
plt.show()

plt.figure()
plt.imshow(frame2_image, cmap=cm.gray_r, norm=LogNorm(vmin=0.01, vmax=1))
plt.gca().invert_yaxis()
plt.show()

plt.figure()
plt.imshow(frame3_image, cmap=cm.gray_r, norm=LogNorm(vmin=0.01, vmax=1))
plt.gca().invert_yaxis()
plt.show()

In [ ]:
plt.figure()
plt.imshow(frame_data0_image, cmap=cm.gray_r, norm=LogNorm(vmin=0.01, vmax=1))
#plt.plot(XPOS,YPOS,'ro')
plt.gca().invert_yaxis()
plt.colorbar()
plt.show()

In [ ]:
plt.figure()
plt.imshow(frame_data0_image[round(YPOS)-3:round(YPOS)+10,round(XPOS)-10:round(XPOS)+10],cmap=cm.gray_r, norm=LogNorm(vmin=0.01, vmax=1))
plt.gca().invert_yaxis()
plt.colorbar()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.utils.data import get_readable_fileobj
#from pydl.photoop.image import sdss_psf_recon
pfAtlas = fits.open('https://data.sdss.org/sas/dr17/eboss/photo/redux/301/1462/objcs/2/fpAtlas-001462-2-0654.fit')

data0 = pfAtlas[0]
data1 = pfAtlas[1]

data0_image = data0.data
data1_image = Table(data1.data)


data0_header = data0.header
data1_header = data1.header


data1_image

Este archivo contiene todos los objetos dentro del field, run, col específicos de su nombre. Quizás puedo usar la página de búsqueda para inteoducir las coordenadas del objeto, luego de ahí coger el número de run, rerun, col, etc y descargar el archivo psField correspondiente. De ahí coger la PSF.

Las unidades están en nanomaggies: 1 nanomaggie = 3.631e-6 Jy = 3.631e-32 Wm-2Hz-1. Esto es un flujo, el paper de Galfit dice que si la unidad está en flujo, el tiempo de exposición debe ser 1 segundo. Creo que si descargas las imágenes de atlas, estas deberían estar en unidades de counts.

In [ ]:
nasa_sloan = fits.open('nsa_v1_0_1.fits')

In [ ]:
print(len(nasa_sloan[1].data))

In [ ]:
nasa_sloan0_data = nasa_sloan[0].data
nasa_sloan0_header = nasa_sloan[0].header
nasa_sloan1_data = nasa_sloan[1].data
nasa_sloan1_header = nasa_sloan[1].header

In [ ]:
nasa_sloan1_data = Table(nasa_sloan1_data)
nasa_sloan1_data

In [ ]:
#Z = []
#RUN = []
#CAMCOL = []
#FIELD = []
#XPOS = []
#YPOS = []
#SERSIC_BA = []
#SERSIC_PHI = []
#NSAID = []
#SDSSID = []
#for element in nasa_sloan1_data:
    #Z.append(element[11])
    #RUN.append(element[14])
    #CAMCOL.append(element[15])
    #FIELD.append(element[16])
    #XPOS.append(element[18])
    #YPOS.append(element[19])
    #NSAID.append(element[20])
    #SDSSID.append(element[4])
    #SERSIC_BA.append(element[56])
    #SERSIC_PHI.append(element[57])
    
    


In [ ]:
nasa_sloan_csv = pd.read_csv('nasa_sloan.csv')
nasa_sloan_csv
#nasa_sloan_csv.to_csv('nasa_sloan.csv',index=False)

In [ ]:
#list_dec = Source_coords_csv['XMatch_dec_common']
#list_ra = Source_coords_csv['XMatch_ra_common']
#for dec in list_dec:
    #coger solo los indices buenos
    

In [ ]:
Source_coords_csv = pd.read_csv('Source_coords_csv.csv')
Source_coords_csv
#J0004+2946

In [ ]:
Sources = pd.read_csv('Sources.csv')
#Sources = Sources.drop(columns = ['sep'],axis=1)
Sources
Sources.to_csv('Sources.csv',index=False)

In [ ]:
Sources1 = Sources[0:1000]
Sources1.to_csv('Sources1.csv',index=False)
Sources2 = Sources[1000:2000]
Sources2.to_csv('Sources2.csv',index=False)
Sources3 = Sources[2000:3000]
Sources3.to_csv('Sources3.csv',index=False)
Sources4 = Sources[3000:4000]
Sources4.to_csv('Sources4.csv',index=False)
Sources5 = Sources[4000:5000]
Sources5.to_csv('Sources5.csv',index=False)
Sources6 = Sources[5000:6000]
Sources6.to_csv('Sources6.csv',index=False)
Sources7 = Sources[6000:7000]
Sources7.to_csv('Sources7.csv',index=False)
Sources8 = Sources[7000:8000]
Sources8.to_csv('Sources8.csv',index=False)
Sources9 = Sources[8000:9000]
Sources9.to_csv('Sources9.csv',index=False)
Sources10 = Sources[9000:]
Sources10.to_csv('Sources10.csv',index=False)

In [ ]:
SDSS1 = pd.read_csv('SDSS1.csv',header=1)
SDSS2 = pd.read_csv('SDSS2.csv',header=1)
SDSS3 = pd.read_csv('SDSS3.csv',header=1)
SDSS4 = pd.read_csv('SDSS4.csv',header=1)
SDSS5 = pd.read_csv('SDSS5.csv',header=1)
SDSS6 = pd.read_csv('SDSS6.csv',header=1)
SDSS7 = pd.read_csv('SDSS7.csv',header=1)
SDSS8 = pd.read_csv('SDSS8.csv',header=1)
SDSS9 = pd.read_csv('SDSS9.csv',header=1)
SDSS10 = pd.read_csv('SDSS10.csv',header=1)

In [ ]:
frames=[SDSS1,SDSS2,SDSS3,SDSS4,SDSS5,SDSS6,SDSS7,SDSS8,SDSS9,SDSS10]
SDSS = pd.concat(frames,ignore_index=True)
SDSS.to_csv('SDSS.csv',index=False)

In [ ]:
SDSS = pd.read_csv('SDSS.csv')
SDSS

In [ ]:
names = []
for index in SDSS['Source_coords_index']:
    if np.isnan(index):
        names.append(np.nan)
    else:
        names.append(np.array(Source_coords_csv['name'])[int(index)])

In [ ]:
ira = []
idec = []
i = 0
for ra,dec in zip(SDSS['ra'],SDSS['dec']):
    abs_ra = abs(Source_coords_csv['ra_deg'] - ra)
    abs_dec = abs(Source_coords_csv['dec_deg'] - dec)
    indexra = np.where((abs_ra < 0.001) == 1)
    indexdec = np.where((abs_dec < 0.001) == 1)
    if (len(indexra) > 1) or (len(indexdec) > 1):
        print('Varias coincidencias en la iteración {}'.format(i))
    ira.append(indexra[0])
    idec.append(indexdec[0])
    i = i + 1

In [ ]:
SDSS['Source_coords_index'] = indexes

In [ ]:
counts = 0
indexes = []
for i in range(len(ira)):
    #print('')
    #print(ira[i],idec[i])
    ira_set = set(ira[i])
    intersection = ira_set. intersection(idec[i]) 
    intersection_as_list = list(intersection)
    if len(intersection_as_list) > 0:
        indexes.append(int(intersection_as_list[0]))
        counts = counts + 1
    if len(intersection_as_list) == 0:
        indexes.append(np.nan) 

In [ ]:
SDSS.to_csv('SDSS.csv',index=False)

Comparar estos objetos con los de source_coords_csv

In [ ]:
nasa_ra = np.array(nasa_sloan_csv['Nasa_RA'])
nasa_dec = np.array(nasa_sloan_csv['Nasa_DEC'])
nasa_ra_str = [str(ra) for ra in nasa_ra]
nasa_dec_str = [str(dec) for dec in nasa_dec]
nasa_ra_lower_value = []
nasa_ra_upper_value = []
nasa_dec_lower_value = []
nasa_dec_upper_value = []
for i,ra in enumerate(nasa_ra_str):
    dot = ra.index('.')
    nasa_ra_lower_value.append(float(ra[:dot+4]+'0'))
    nasa_ra_upper_value.append(float(ra[:dot+4]+'999'))

for i,dec in enumerate(nasa_dec_str):
    dot = dec.index('.')
    nasa_dec_lower_value.append(float(dec[:dot+5]+'0'))
    nasa_dec_upper_value.append(float(dec[:dot+5]+'999'))

nasa_ra_lower_value = np.array(nasa_ra_lower_value)
nasa_ra_upper_value = np.array(nasa_ra_upper_value)
nasa_dec_lower_value = np.array(nasa_dec_lower_value)
nasa_dec_upper_value = np.array(nasa_dec_upper_value)

Source_dec = np.array(Source_coords_csv['dec_deg'])
Source_ra = np.array(Source_coords_csv['ra_deg'])

common_dec_indexes = []
for dec in Source_dec:
    index = []
    if dec < 0:
        for i in range(len(nasa_dec_lower_value)):
            if nasa_dec_lower_value[i] >= dec >= nasa_dec_upper_value[i]:
                index.append(i)
    if dec >= 0:
        for i in range(len(nasa_dec_lower_value)):
            if nasa_dec_lower_value[i] <= dec <= nasa_dec_upper_value[i]:
                index.append(i)
    common_dec_indexes.append(index)
    
common_ra_indexes = []
for ra in Source_ra:
    index = []
    for i in range(len(nasa_ra_lower_value)):
        if nasa_ra_lower_value[i] <= ra <= nasa_ra_upper_value[i]:
            index.append(i)
    common_ra_indexes.append(index)
            
    
#common_ra = np.intersect1d(nasa_ra,Source_ra)
#common_dec = np.intersect1d(nasa_dec,Source_dec)

In [ ]:
#ID = np.array((nasa_sloan_csv['Sloan_ID']))
#indexes = np.array(XMatch['Source_coords_index'])
#ID_XMatch = ID[indexes]
#XMatch.insert(18,'Sloan_ID',ID_XMatch)

In [ ]:
psf1 = fits.open('psf_7907_6_197.fits')[0].data
psf2 = fits.open('psf_5933_4_23.fits')[0].data
psf3 = fits.open('psf_7885_2_64.fits')[0].data
psf4 = fits.open('psf_8062_2_84.fits')[0].data
psf5 = fits.open('psf_3634_5_104.fits')[0].data
psf6 = fits.open('psf_3628_1_127.fits')[0].data
psf7 = fits.open('psf_5069_5_246.fits')[0].data
psf8 = fits.open('psf_6514_3_69.fits')[0].data
psf9 = fits.open('psf_3559_5_77.fits')[0].data
psf10 = fits.open('psf_6585_3_128.fits')[0].data
psf11 = fits.open('psf_2825_3_59.fits')[0].data
psf12 = fits.open('psf_2864_4_201.fits')[0].data
psf13 = fits.open('psf_5080_4_162.fits')[0].data
psf14 = fits.open('psf_6752_4_36.fits')[0].data
psf15 = fits.open('psf_2392_2_170.fits')[0].data
psf16 = fits.open('psf_4134_1_125.fits')[0].data
psf17 = fits.open('psf_8083_2_92.fits')[0].data
psf18 = fits.open('psf_7765_5_129.fits')[0].data
psf19 = fits.open('psf_1739_5_250.fits')[0].data
psfs = [psf1,psf2,psf3,psf4,psf5,psf6,psf7,psf8,psf9,psf10,psf11,psf12,psf13,psf14,psf15,psf16,psf17,psf18,psf19]

In [ ]:
Test = pd.read_csv('Test.txt')
Test

In [ ]:
Test['ra'] = list(XMatch['Source_coords_RA'])
Test['dec'] = list(XMatch['Source_coords_DEC'])

In [ ]:
Test.to_csv('Test.csv',index=False)

In [ ]:
XMatch = pd.read_csv('XMatch.csv')
XMatch

In [ ]:
ite = 20
PAS = []
for source in range(ite):
    pa = -999
    counts2 = 0
    n = 50
    real_vlbi_name = glob.glob('images\\'+PA_names[source]+'\\J*map.fits')
    if len(real_vlbi_name) > 0:
        real_vlbi_name = real_vlbi_name[-1]
    else:
        print(PA_names[source])
        print('Source is a variation of some other source with a letter, fits not in VLBI')
        PAS.append(np.NAN)
        continue

    data = fits.open(real_vlbi_name)
    image = data[0].data
    headers = data[0].header
    image = image[0][0]
    PA_vlbi = headers['BPA']
    pixscale_arcsec = data[0].header['CDELT2']*360
    galaxy_center = np.where(image[200:301,200:301] == np.amax(image[200:301,200:301]))
    galaxy_center_x = np.arange(200,301)[galaxy_center[1][0]]
    galaxy_center_y = np.arange(200,301)[galaxy_center[0][0]]
    
    

In [ ]:
for i,source in enumerate(XMatch['Source_coords_index']):   #len(Source_coords_csv['name'])
    
    pa = -999
    counts2 = 0
    n = 40
    real_vlbi_name = glob.glob('images\\'+PA_names[source]+'\\J*map.fits')
    if len(real_vlbi_name) > 0:
        real_vlbi_name = real_vlbi_name[-1]
    else:
        print(PA_names[source])
        print('Source is a variation of some other source with a letter, fits not in VLBI')
        PAS.append(np.NAN)
        continue

    data = fits.open(real_vlbi_name)
    image = data[0].data
    headers = data[0].header
    image = image[0][0]
    PA_vlbi = headers['BPA']
    pixscale_arcsec = data[0].header['CDELT2']*360
    galaxy_center = np.where(image[200:301,200:301] == np.amax(image[200:301,200:301]))
    galaxy_center_x = np.arange(200,301)[galaxy_center[1][0]]
    galaxy_center_y = np.arange(200,301)[galaxy_center[0][0]]
    
    image_copy = image.copy()
    zoom_image = image_copy[int(galaxy_center_y)-n:int(galaxy_center_y)+n,int(galaxy_center_x)-n:int(galaxy_center_x)+n]
    masked_zoom_image = zoom_image.copy()
    masked_zoom_image[abs(masked_zoom_image) > 1e-3] = 10*masked_zoom_image[abs(masked_zoom_image) > 1e-3]
    positions_above_zero = np.where(masked_zoom_image > 1e-3)
    masked_zoom_image[positions_above_zero] = np.log10(masked_zoom_image[positions_above_zero])
    
    
    
    
    pos = coords.SkyCoord(Source_coords_RA[source],Source_coords_DEC[source], unit=(u.hourangle, u.deg), frame='icrs')
    im = SkyView.get_images(pos, survey='SDSSr', grid=True, gridlabels=True,  radius=45*u.arcsec)
    #si lo metes con pixels, luego la imagen no hay que invertirla
    image = im[0][0].data.copy()
    galaxy_center = np.where(image == np.amax(image))
    galaxy_center_x = galaxy_center[1][0]
    galaxy_center_y = galaxy_center[0][0]

    n = 40
    image_copy = image.copy()
    image_zoom = image_copy[galaxy_center_y-n:galaxy_center_y+n,galaxy_center_x-n:galaxy_center_x+n]


    try:
        params,pcov = fit(image_zoom)
    except Exception:
        continue

    x = np.arange(0, image_zoom.shape[1], 1) #Stars at 0, increases by 1, goes to length of axis
    y = np.arange(0, image_zoom.shape[0], 1) #Stars at 0, increases by 1, goes to length of axis
    xx, yy = np.meshgrid(x, y) #creates a grid to plot the 
    xycoords = (xx, yy)
    amplitude,x0,y0,sigma_x,sigma_y,theta,offset = params
    theta_err = 180*pcov[5,5]**0.5/np.pi
    sigma_x_err = pcov[3,3]**0.5
    sigma_y_err = pcov[4,4]**0.5
    sigma_y_sigma_x_err = ((-sigma_y*sigma_x_err/sigma_x**2)**2 + (sigma_y_err/sigma_x)**2)**0.5
    if sigma_x > sigma_y:
        a = sigma_x
        a_err = sigma_x_err
        b = sigma_y
        b_err = sigma_y_err
    else:
        a = sigma_y
        a_err = sigma_y_err
        b = sigma_x
        b_err = sigma_x_err
    b_a = b/a
    b_a_err = ((-b*a_err/a**2)**2 + (b_err/a)**2)**0.5
    e = (1-b_a**2)**0.5
    e_err = -b_a*b_a_err/e

    vector = twoD_Gaussian(xycoords, amplitude, x0, y0, sigma_x, sigma_y, theta, offset)
    matrix = vector.reshape(image_zoom.shape[0],image_zoom.shape[1])

    theta_from_positive_y = 180*(-theta)/np.pi - 90
    if sigma_y > sigma_x:
        theta_minor = theta_from_positive_y
        theta_major = theta_from_positive_y - 90
    else:
        theta_minor = theta_from_positive_y - 90
        theta_major = theta_from_positive_y

    k_minor = int(abs(theta_minor)/360) 
    theta_minor = theta_minor + 360*k_minor
    if 360 >= abs(theta_minor) >= 180:
        theta_minor = 360 - abs(theta_minor)

    k_major = int(abs(theta_major)/360) 
    theta_major = theta_major + 360*k_major
    if 360 >= abs(theta_major) >= 180:
        theta_major = 360 - abs(theta_major)    
    minor_jet_angle = abs(PA_values[source] - theta_minor)
    if minor_jet_angle > 90:
        minor_jet_angle = 180 - minor_jet_angle

    average_noise = abs(np.mean(image_zoom[0:10,0:10]))
    Signal_amp = offset + amplitude
    signal_noise = Signal_amp/average_noise


    l = b*3
    #minor
    xf1 = x0 + l*np.cos((theta_minor+90)*np.pi/180)
    yf1 = y0 + l*np.sin((theta_minor+90)*np.pi/180)
    xf2 = x0 + l*np.cos((theta_minor-90)*np.pi/180)
    yf2 = y0 + l*np.sin((theta_minor-90)*np.pi/180)

    l = a*3
    #major
    xf3 = x0 + l*np.cos((theta_major+90)*np.pi/180)
    yf3 = y0 + l*np.sin((theta_major+90)*np.pi/180)
    xf4 = x0 + l*np.cos((theta_major-90)*np.pi/180)
    yf4 = y0 + l*np.sin((theta_major-90)*np.pi/180)

    l = 15
    #Jet
    PA_jet_angle_from_x = (PA_values[source]+90)*np.pi/180
    xf_jet = x0 + l*np.cos(PA_jet_angle_from_x)
    yf_jet = y0 + l*np.sin(PA_jet_angle_from_x)


    print(PA_names[source])
    print('S/N =',signal_noise)
    print('Jet PA = ',PA_values[source],'deg (from +y counterclock.)')
    print('Minor PA =',round(theta_minor,2),'+/-',round(theta_err,2),'deg (from +y counterclock.)')
    print('Angle between jet and minor ax. =',round(minor_jet_angle,2),'+/-',round(theta_err,2),'deg')
    print('b/a =',round(b/a,2),'+/-',round(b_a_err,2))
    print('e =',round(e,2),'+/-',round(e_err,2))


    plt.figure()
    plt.imshow(image_zoom)
    plt.colorbar()
    plt.contour(xx, yy, matrix, [0.05*np.amax(matrix),np.amax(matrix)], colors='w')
    plt.plot([x0,xf1],[y0,yf1],'w-')
    plt.plot([x0,xf2],[y0,yf2],'w-')
    plt.plot([x0,xf3],[y0,yf3],'w-')
    plt.plot([x0,xf4],[y0,yf4],'w-',label='axis')
    plt.plot([x0,xf_jet],[y0,yf_jet],'r-',label='Jet')
    plt.plot(xf1,yf1,'ro',label='Minor ax. PA')
    plt.gca().invert_yaxis()
    plt.legend()
    plt.title(XMatch['Source_coords_name'][i])
    plt.show()
    
    plt.figure()
    plt.imshow(psfs[i])
    plt.gca().invert_yaxis()
    plt.title(XMatch['Source_coords_name'][i]+' PSF')
    plt.show()
    
    
    plt.figure() 
    plt.imshow(masked_zoom_image)
    plt.plot([x0,xf_jet],[y0,yf_jet],'r-',label='Jet')
    plt.gca().invert_yaxis()
    plt.title(PA_names[source]+' VLBI')
    plt.show()

In [ ]:
common_dec_csv = list(Source_coords_csv['XMatch_dec_common'])
Source_coords_name = []
Nasa_sloan_Name = []
Source_coords_RA = []
Source_coords_DEC = []
Nasa_RA = []
Nasa_DEC = []
Source_coords_index = []
Nasa_sloan_index = []
Z = []
phi_major_E_of_N_sersic = []
phi_major_W_of_N_gaussian = []
Jet_PA_W_of_N = []
BA_sersic = []
BA_gaussian = []
RUN = []
CAMCOL = []
FIELD = []
XPOS = []
YPOS = []


for i,element in enumerate(common_dec_csv):
    if element == '[]':
        continue
    else:
        index =''
        for s in element: 
            if s.isdigit():
                index = index + str(s)
        index= int(index)
        Source_coords_name.append(Source_coords_csv['name'][i])
        Nasa_sloan_Name.append(nasa_sloan_csv['Nasa_name'][index])
        Source_coords_RA.append(Source_coords_csv['ra_deg'][i])
        Source_coords_DEC.append(Source_coords_csv['dec_deg'][i])
        Nasa_RA.append(nasa_sloan_csv['Nasa_RA'][index])
        Nasa_DEC.append(nasa_sloan_csv['Nasa_DEC'][index])
        Source_coords_index.append(i)
        Nasa_sloan_index.append(index)
        Z.append(nasa_sloan_csv['Z'][index])
        phi_major_E_of_N_sersic.append(nasa_sloan_csv['SERSIC_PHI'][index])
        phi_major_W_of_N_gaussian.append(Gaussian['pa_major_gaussian'][i])
        Jet_PA_W_of_N.append(PA_values[i])
        BA_sersic.append(nasa_sloan_csv['SERSIC_BA'][index])
        BA_gaussian.append(Gaussian['b_a_gaussian'][i])
        RUN.append(nasa_sloan_csv['RUN'][index])
        CAMCOL.append(nasa_sloan_csv['CAMCOL'][index])
        FIELD.append(nasa_sloan_csv['FIELD'][index])
        XPOS.append(nasa_sloan_csv['XPOS'][index])
        YPOS.append(nasa_sloan_csv['YPOS'][index]) 

        
        
        

XMatch['Source_coords_name'] = Source_coords_name
XMatch['Nasa_sloan_Name'] = Nasa_sloan_Name
XMatch['Source_coords_RA'] = Source_coords_RA
XMatch['Source_coords_DEC'] = Source_coords_DEC
XMatch['Nasa_RA'] = Nasa_RA
XMatch['Nasa_DEC'] = Nasa_DEC
XMatch['Source_coords_index'] = Source_coords_index
XMatch['Nasa_sloan_index'] = Nasa_sloan_index
XMatch['Z'] = Z
XMatch['phi_major_E_of_N_sersic'] = phi_major_E_of_N_sersic
XMatch['phi_major_W_of_N_gaussian'] = phi_major_W_of_N_gaussian
XMatch['Jet_PA_W_of_N'] = Jet_PA_W_of_N
XMatch['BA_sersic'] = BA_sersic
XMatch['BA_gaussian'] = BA_gaussian
XMatch['RUN'] = RUN
XMatch['CAMCOL'] = CAMCOL
XMatch['FIELD'] = FIELD
XMatch['XPOS'] = XPOS
XMatch['YPOS'] = YPOS
XMatch.to_csv('XMatch.csv',index=False)

In [ ]:
#i = 0
#XMatches_indexes_dec = []
#XMatches_indexes_ra = []
#for ra,dec in zip(common_ra_indexes,common_dec_indexes):
    #if (ra != dec) or ((ra,dec) == ([],[])):
        #XMatches_indexes_dec.append([])
        #XMatches_indexes_ra.append([])
        #continue
    #else:
        #XMatches_indexes_dec.append(ra)
        #XMatches_indexes_ra.append(dec)
    #i = i + 1

Una vez que tenga los índices comunes, puedo sacar del archivo de nasa-sloan los datos de camcol,field,run,row,col del objeto e introducir esos en la url para descargar la psField. Con ella, usar el código de python o ubuntu para sacar la PSF de ese archivo en la row,col específicas (filtro r).

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord
ra_list = []
dec_list = []
for ra,dec in zip(Source_coords_csv['ra'],Source_coords_csv['dec']):
    #print(ra,dec)
    try:
        c = SkyCoord(str(ra)+' '+str(dec), unit=(u.hourangle, u.deg))
        ra_list.append(c.ra.deg)
        dec_list.append(c.dec.deg)
    except:
        ra_list.append(-999)
        dec_list.append(-999)
#d = SkyCoord('+41:12', unit=u.deg)
#d.degree



In [ ]:
#list_names_combined = np.array(list_names_first_part)+np.array(list_names_last_part)

In [ ]:
#list_names_first_part = list(Source_coords_csv['name_first_part'])
#list_names_first_part = [str(e) for e in list_names_first_part]
#list_names_first_part = [e.zfill(4) for e in list_names_first_part]
#list_names_last_part = list(Source_coords_csv['name_last_part'])
#list_names_last_part = [str(e) for e in list_names_last_part]
#list_combined_strings = [(e+j) for e,j in zip(list_names_first_part,list_names_last_part)]
#XMatch_indexes_Source_csv = []
#XMatch_indexes_nasa_sloan = []
#for i,obj in enumerate(nasa_sloan1_data):
    #nasa_name = obj[0]
    #slash = nasa_name.find('-')
    #plus = nasa_name.find('+')
    #if slash == -1:
        #symbol = plus
    #if plus == -1:
        #symbol = slash
    #nasa_name_numbers = nasa_name[1:5]+nasa_name[symbol+1:symbol+5]
    #if nasa_name_numbers in list_combined_strings:  
        #index_Source = list_combined_strings.index(nasa_name_numbers)
        #XMatch_indexes_Source_csv.append(index_Source) 
        #index_nasa_sloan = i
        #XMatch_indexes_nasa_sloan.append(index_nasa_sloan)

In [ ]:
Gaussian = pd.read_csv('Gaussian.csv')
Gaussian

In [ ]:
XMatch_indexes_Source_csv = XMatch['Source_coords_index']
angles = abs(np.array((XMatch['phi_major_W_of_N_sersic'])) - np.array((XMatch['Jet_PA_W_of_N'])))
for i,angle in enumerate(angles):
    if angle > 180:
        angles[i] = 360 - angles[i]
for i,angle in enumerate(angles):
    if angle > 90:
        angles[i] = 180 - angles[i]
plt.hist(angles,bins=6)
plt.show()

In [ ]:
#nasa_sloan_redshift = []
#for element in nasa_sloan1_data[XMatch_indexes_nasa_sloan]:
    #nasa_sloan_redshift.append(element[11])
#XMatch['Z_nasa_sloan'] = nasa_sloan_redshift

In [ ]:
#b_a_sersic = []
#phi_E_of_N_sersic = []
#for e in nasa_sloan1_data[XMatch_indexes_nasa_sloan]:
    #b_a_sersic.append(e[56])
    #phi_E_of_N_sersic.append(e[57])

In [ ]:
#redshift is the 12 element
#print(nasa_sloan1_data)

Do a Xmatch bewteen these objects and the vlbi ones using these coordinates.

In [ ]:
ite = 20
PAS = []
for source in range(ite):
    pa = -999
    counts2 = 0
    n = 50
    real_vlbi_name = glob.glob('images\\'+PA_names[source]+'\\J*map.fits')
    if len(real_vlbi_name) > 0:
        real_vlbi_name = real_vlbi_name[-1]
    else:
        print(PA_names[source])
        print('Source is a variation of some other source with a letter, fits not in VLBI')
        PAS.append(np.NAN)
        continue

    data = fits.open(real_vlbi_name)
    image = data[0].data
    headers = data[0].header
    image = image[0][0]
    PA_vlbi = headers['BPA']
    pixscale_arcsec = data[0].header['CDELT2']*360
    galaxy_center = np.where(image[200:301,200:301] == np.amax(image[200:301,200:301]))
    galaxy_center_x = np.arange(200,301)[galaxy_center[1][0]]
    galaxy_center_y = np.arange(200,301)[galaxy_center[0][0]]
    while abs(pa-PA_values[source]) > 50:
        if counts2 == 5:
            break
        n = n - 5
        #Creo que valía con coger solo los archivos que se llamen ...C...mat.fits





        
        image_copy = image.copy()
        zoom_image = image_copy[int(galaxy_center_y)-n:int(galaxy_center_y)+n,int(galaxy_center_x)-n:int(galaxy_center_x)+n]
        masked_zoom_image = zoom_image.copy()
        masked_zoom_image[abs(masked_zoom_image) > 1e-3] = 10*masked_zoom_image[abs(masked_zoom_image) > 1e-3]

        neighborhood_size = n/7
        threshold = 0.005

        data_max = filters.maximum_filter(masked_zoom_image, neighborhood_size)
        maxima = (masked_zoom_image == data_max)
        data_min = filters.minimum_filter(masked_zoom_image, neighborhood_size)
        diff = ((data_max - data_min) > threshold)
        maxima[diff == 0] = 0
        if len(maxima) != 0:
            labeled, num_objects = ndimage.label(maxima)
            slices = ndimage.find_objects(labeled)
            x, y = [], []
            for dy,dx in slices:
                x_center = (dx.start + dx.stop - 1)/2
                x.append(x_center)
                y_center = (dy.start + dy.stop - 1)/2    
                y.append(y_center)

        counts = 0
        while len(x) < 2:
            if counts == 500:
                #print('Less than 2 max points found')
                break
            threshold = threshold - 0.005
            data_max = filters.maximum_filter(masked_zoom_image, neighborhood_size)
            maxima = (masked_zoom_image == data_max)
            data_min = filters.minimum_filter(masked_zoom_image, neighborhood_size)
            diff = ((data_max - data_min) > threshold)
            maxima[diff == 0] = 0

            labeled, num_objects = ndimage.label(maxima)
            slices = ndimage.find_objects(labeled)
            x, y = [], []
            for dy,dx in slices:
                x_center = (dx.start + dx.stop - 1)/2
                x.append(x_center)
                y_center = (dy.start + dy.stop - 1)/2    
                y.append(y_center)
            counts = counts + 1

        if len(x) == 0:
            print(PA_names[source])
            print('No max found')
            positions_above_zero = np.where(masked_zoom_image > 1e-3)
            masked_zoom_image[positions_above_zero] = np.log10(masked_zoom_image[positions_above_zero])
            plt.figure()
            plt.imshow(masked_zoom_image)
            plt.gca().invert_yaxis()
            plt.colorbar()
            plt.show()
            break

        if len(x) == 1:
            #si lo metes con pixels, luego la imagen no hay que invertirla
            args_ = (zoom_image,)
            optimal_params = scipy.optimize.differential_evolution(func = f, bounds = ((5,20), (0.1,0.7), (10,170)), args = args_, strategy='best1bin', maxiter=1000, popsize=15, tol=0.01, mutation=(0.5, 1), recombination=0.7, seed=None, callback=None, disp=False, polish=True, init='latinhypercube', atol=0, updating='immediate', workers=1, x0=None)

        while len(x) > 2:
            threshold = threshold + 0.005
            data_max = filters.maximum_filter(masked_zoom_image, neighborhood_size)
            maxima = (masked_zoom_image == data_max)
            data_min = filters.minimum_filter(masked_zoom_image, neighborhood_size)
            diff = ((data_max - data_min) > threshold)
            maxima[diff == 0] = 0

            labeled, num_objects = ndimage.label(maxima)
            slices = ndimage.find_objects(labeled)
            x, y = [], []
            for dy,dx in slices:
                x_center = (dx.start + dx.stop - 1)/2
                x.append(x_center)
                y_center = (dy.start + dy.stop - 1)/2    
                y.append(y_center)
        if len(x) == 2:
            if abs(x[1]-x[0]) != 0:
                pa = np.arctan(abs(y[1]-y[0])/abs(x[1]-x[0]))*180/np.pi
            else:
                pa = 90

            tup = (masked_zoom_image[int(y[0]),int(x[0])],masked_zoom_image[int(y[1]),int(x[1])])
            for i in range(2):
                if tup[i] == max(tup):
                    max_i = i
                else:
                    min_i = i

            galaxy_center_x = x[max_i]
            galaxy_center_y = y[max_i]
            lobe_center_x = x[min_i]
            lobe_center_y = y[min_i]

            if lobe_center_y >= galaxy_center_y:
                if lobe_center_x < galaxy_center_x:
                    pa = 180 - pa
            else:
                if lobe_center_x >= galaxy_center_y:
                    pa = -pa
                if lobe_center_x < galaxy_center_y:
                    pa = pa + 180
            pa = pa - 90      
        
        #meter todo lo anterior pero sumando valor a la n?
        
        counts2 = counts2 + 1
    


    
    
    if len(x) == 1:
        args_ = (zoom_image,)
        optimal_params = scipy.optimize.differential_evolution(func = f, bounds = ((5,20), (0.1,0.7), (10,170)), args = args_, strategy='best1bin', maxiter=1000, popsize=15, tol=0.01, mutation=(0.5, 1), recombination=0.7, seed=None, callback=None, disp=False, polish=True, init='latinhypercube', atol=0, updating='immediate', workers=1, x0=None)

        if optimal_params.success:
            optimal_sma = optimal_params.x[0]
            optimal_eps = optimal_params.x[1]
            optimal_pa = optimal_params.x[2]
            print(PA_names[source])
            print('optimal init. sma = ',optimal_sma,'  ','optimal init. eps = ',optimal_eps,'  ','optimal init. pa = ',optimal_pa)

            geometry = EllipseGeometry(x0=n, y0=n, sma=optimal_sma, eps=optimal_eps, pa=optimal_pa * np.pi / 180.)
            aper = EllipticalAperture((geometry.x0, geometry.y0), geometry.sma,  geometry.sma * (1 - geometry.eps), geometry.pa)
            ellipse = Ellipse(zoom_image, geometry)
            isolist = ellipse.fit_image()
            model_image = build_ellipse_model(zoom_image.shape, isolist)

            residuals = zoom_image - model_image

            ellipses = []
            for sma in isolist.sma[1:]:
                iso = isolist.get_closest(sma)
                ellipse = iso.sampled_coordinates()
                ellipses.append(ellipse)

            index_contour = np.where(isolist.intens/np.max(isolist.intens) < 0.15)[0][0]


            print('No jet structure found')
            print('Calculated PA of the sMa is',isolist.pa[index_contour]*180/np.pi-90,'degrees')
            print('Calculated PA of the sma is',isolist.pa[index_contour]*180/np.pi-180,'degrees')
            print('PA from the paper is',PA_values[source],'degrees')
            
            positions_above_zero = np.where(masked_zoom_image > 1e-3)
            masked_zoom_image[positions_above_zero] = np.log10(masked_zoom_image[positions_above_zero])

            plt.figure()
            plt.imshow(masked_zoom_image)
            #for i in range(len(isolist.sma[1:])):
                #plt.plot(ellipses[i][0],ellipses[i][1],'w-')
            plt.plot(ellipses[index_contour][0],ellipses[index_contour][1],'w-')
            plt.gca().invert_yaxis()
            plt.colorbar()
            plt.show()


            #plt.figure()
            #plt.imshow(model_image)
            #for i in range(len(isolist.sma[1:])):
                #plt.plot(ellipses[i][0],ellipses[i][1],'w-')
            #plt.plot(ellipses[index_contour][0],ellipses[index_contour][1],'w-')
            #plt.gca().invert_yaxis()
            #plt.colorbar()
            #plt.title('Model image')
            #plt.show()


            #plt.figure()
            #plt.imshow(residuals)
            #for i in range(len(isolist.sma[1:])):
                #plt.plot(ellipses[i][0],ellipses[i][1],'w-')
            #plt.plot(ellipses[index_contour][0],ellipses[index_contour][1],'w-')
            #plt.gca().invert_yaxis()
            #plt.colorbar()
            #plt.title('Residuals')
            #plt.show()


            #plt.figure()
            #plt.scatter(isolist.sma, isolist.intens)
            #plt.scatter(isolist.sma[index_contour], isolist.intens[index_contour],c='r')
            #plt.xlabel('sma')
            #plt.ylabel('Intensity')
            #plt.show()


            #plt.figure()
            #plt.scatter(isolist.sma, isolist.pa*180/np.pi)
            #plt.scatter(isolist.sma[index_contour], isolist.pa[index_contour]*180/np.pi,c='r')
            #plt.xlabel('sma')
            #plt.ylabel('PA(º)')
            #plt.show()

        else:
            print(PA_names[source])
            print(optimal_params.message)
            continue
            


    if len(x) == 2:
        if abs(x[1]-x[0]) != 0:
            pa = np.arctan(abs(y[1]-y[0])/abs(x[1]-x[0]))*180/np.pi
        else:
            pa = 90

        tup = (masked_zoom_image[int(y[0]),int(x[0])],masked_zoom_image[int(y[1]),int(x[1])])
        for i in range(2):
            if tup[i] == max(tup):
                max_i = i
            else:
                min_i = i

        galaxy_center_x = x[max_i]
        galaxy_center_y = y[max_i]
        lobe_center_x = x[min_i]
        lobe_center_y = y[min_i]

        if lobe_center_y >= galaxy_center_y:
            if lobe_center_x < galaxy_center_x:
                pa = 180 - pa
        else:
            if lobe_center_x >= galaxy_center_y:
                pa = -pa
            if lobe_center_x < galaxy_center_y:
                pa = pa + 180
        pa = pa - 90
        PAS.append(pa)
        print(PA_names[source])
        print('Calculated PA is',pa,'degrees')
        print('PA from paper is',PA_values[source])
        positions_above_zero = np.where(masked_zoom_image > 1e-3)
        masked_zoom_image[positions_above_zero] = np.log10(masked_zoom_image[positions_above_zero])
        plt.figure()
        plt.imshow(masked_zoom_image)
        plt.plot(x,y, 'r.')
        plt.plot(x,y,'r-')
        plt.colorbar()
        plt.gca().invert_yaxis()
        plt.show()
       

In [ ]:
#for i in range(len(PAS)):
    #print(PAS[i],PA_values[i])

In [ ]:
#wrong_indexes = np.where(abs(PA_values[0:len(PAS)]-PAS) > 45)[0]
#print(wrong_indexes)
#print(len(wrong_indexes))

In [ ]:
#nan_indexes = np.where(np.isnan(PAS[0:42]) == True)[0]
#print(nan_indexes)
#print(len(nan_indexes))

In [ ]:
#-----------------------------Optical--------------------------------------------

In [ ]:
Source_names = []
pa_differences = []
best_pas = []
best_epss = []
best_smas = []
ellipse_coords_x = []
ellipse_coords_y = []
images_zoom = []
model_images =[]
residualss = []
a = []
b = []
e = []
b_divided_a = []
pa_abs_difference_minor_axis = []
pa_major = []
pa_major_err = []
major_axis_profiles = []
luminosities_profile = []
luminosities_profile_err = []
ellipse_total_luminosities = []
indexes_contour = []

In [ ]:
for source in range(29,40):   
    try:
        pos = coords.SkyCoord(Source_coords_RA[source],Source_coords_DEC[source], unit=(u.hourangle, u.deg), frame='icrs')
        im = SkyView.get_images(pos, survey='SDSSr', grid=True, gridlabels=True,  radius=45*u.arcsec)
        #si lo metes con pixels, luego la imagen no hay que invertirla
        image = im[0][0].data.copy()
        galaxy_center = np.where(image == np.amax(image))
        galaxy_center_x = galaxy_center[1][0]
        galaxy_center_y = galaxy_center[0][0]

        n = 40
        image_copy = image.copy()
        image_zoom = image_copy[galaxy_center_y-n:galaxy_center_y+n,galaxy_center_x-n:galaxy_center_x+n]
        image_log = image_zoom.copy()
        image_log[image_log < 1e-2] = 0
        image_log[image_log > 1e-1] = np.log10(image_log[image_log > 1e-1])
    except Exception:
        print(PA_names[source])
        print('Not in SDSS footprint')
        print('PA from the paper is',PA_values[source])
        Source_names.append(PA_names[source])
        pa_differences.append(np.nan)
        best_pas.append(np.nan)
        best_epss.append(np.nan)
        best_smas.append(np.nan)
        ellipse_coords_x.append(np.nan)
        ellipse_coords_y.append(np.nan)
        images_zoom.append(np.nan)
        model_images.append(np.nan)
        residualss.append(np.nan)
        a.append(np.nan)
        b.append(np.nan)
        e.append(np.nan)
        b_divided_a.append(np.nan)
        pa_abs_difference_minor_axis.append(np.nan)
        pa_major.append(np.nan)
        pa_major_err.append(np.nan)
        major_axis_profiles.append(np.nan)
        luminosities_profile.append(np.nan)
        luminosities_profile_err.append(np.nan)
        ellipse_total_luminosities.append(np.nan)
        indexes_contour.append(np.nan)
        continue
    #xid = SDSS.query_region(pos, spectro=False, radius=45*u.arcsec)
    #source_pos = coords.SkyCoord(str(xid['ra'][0]),str(xid['ra'][0]), unit=(u.deg, u.deg), frame='icrs')

    #im = SDSS.get_images(matches=xid, band='g')
    
    Chi_squared = []
    values = []
    i = 0
    for init_pa,init_eps,init_sma in itertools.product(np.arange(1,179,10), np.arange(0.,0.7,0.1),np.arange(1,40,2)):
        total_ite = len(np.arange(1,179,10))*len(np.arange(0.,0.7,0.1))*len(np.arange(1,40,2))
        progress = str(round(100*i/total_ite,2))
        print(progress+'%', end='\r', flush=True) 
        values.append((init_pa,init_eps,init_sma))
        try:
            geometry = EllipseGeometry(x0=n, y0=n, sma=init_sma, eps=init_eps, pa=init_pa * np.pi / 180.)
            aper = EllipticalAperture((geometry.x0, geometry.y0), geometry.sma,  geometry.sma * (1 - geometry.eps), geometry.pa)
            ellipse = Ellipse(image_zoom, geometry)
            isolist = ellipse.fit_image()
            model_image = build_ellipse_model(image_zoom.shape, isolist)
        except Exception:
            Chi_squared.append(1e5)
        else:
            Chi_squared.append(np.sum(abs((image_zoom - model_image)**2/image_zoom)))
        i = i + 1

    try:
        Chi_squared = np.array(Chi_squared)
        best_index = np.where(Chi_squared == np.min(Chi_squared))[0][0]

        best_init_pa = values[best_index][0]
        best_init_eps = values[best_index][1]
        best_init_sma = values[best_index][2]
        
        
        

        geometry = EllipseGeometry(x0=n, y0=n, sma=best_init_sma, eps=best_init_eps, pa=best_init_pa * np.pi / 180.)
        aper = EllipticalAperture((geometry.x0, geometry.y0), geometry.sma,  geometry.sma * (1 - geometry.eps), geometry.pa)
        ellipse = Ellipse(image_zoom, geometry)
        isolist = ellipse.fit_image()
        model_image = build_ellipse_model(image_zoom.shape, isolist)

        residuals = image_zoom - model_image

        ellipses = []
        for sma in isolist.sma[1:]:
            iso = isolist.get_closest(sma)
            ellipse = iso.sampled_coordinates()
            ellipses.append(ellipse)

        index_contour = np.where(isolist.intens/np.max(isolist.intens) < 0.15)[0][0]
        
        eps = isolist.eps[index_contour]
        sMa = isolist.sma[index_contour]
        sma = np.sqrt(1-eps**2)*sMa
        sma_divided_sMa = sma/sMa


        calculated_pa = isolist.pa[index_contour]*180/np.pi-90
        calculated_pa_err = isolist.pa_err[index_contour]*180/np.pi
        pa_difference = abs(PA_values[source]-calculated_pa)
        
        pa_difference_90 = abs(pa_difference - 90)
        pa_difference_270 = abs(pa_difference - 270)
        if pa_difference_90 > 180:
            pa_difference_90 = abs(360-pa_difference_90)
        if pa_difference_270 > 180:
            pa_difference_270 = abs(360-pa_difference_270)                   
                           
        pa_minor_difference = min(pa_difference_90,pa_difference_270)                   
                           
        major_axis_profile = isolist.sma
        luminosity_profile = isolist.intens
        luminosity_profile_err = isolist.int_err
        total_luminosity_inside_ellipse = isolist.tflux_e[index_contour]
                           
        print(PA_names[source])
        print('Absolute angle difference between jet and sMa:',pa_difference,'degrees')
        #print('Calculated PA is',isolist.pa[index_contour]*180/np.pi-90,'degrees')
        #print('PA from the paper is',PA_values[source],'degrees')


        plt.figure()
        plt.imshow(image_zoom)
        #for i in range(len(isolist.sma[1:])):
            #plt.plot(ellipses[i][0],ellipses[i][1],'w-')
        plt.plot(ellipses[index_contour][0],ellipses[index_contour][1],'w-')
        plt.gca().invert_yaxis()
        plt.colorbar()
        plt.title('Original image')
        plt.show()


        plt.figure()
        plt.imshow(model_image)
        #for i in range(len(isolist.sma[1:])):
            #plt.plot(ellipses[i][0],ellipses[i][1],'w-')
        plt.plot(ellipses[index_contour][0],ellipses[index_contour][1],'w-')
        plt.gca().invert_yaxis()
        plt.colorbar()
        plt.title('Model image')
        plt.show()


        plt.figure()
        plt.imshow(residuals)
        #for i in range(len(isolist.sma[1:])):
            #plt.plot(ellipses[i][0],ellipses[i][1],'w-')
        plt.plot(ellipses[index_contour][0],ellipses[index_contour][1],'w-')
        plt.gca().invert_yaxis()
        plt.colorbar()
        plt.title('Residuals')
        plt.show()
        
        

        plt.figure()
        plt.scatter(isolist.sma, isolist.intens)
        plt.scatter(isolist.sma[index_contour], isolist.intens[index_contour],c='r')
        plt.xlabel('sma')
        plt.ylabel('Intensity')
        plt.show()


        plt.figure()
        plt.scatter(isolist.sma, isolist.pa*180/np.pi)
        plt.scatter(isolist.sma[index_contour], isolist.pa[index_contour]*180/np.pi,c='r')
        plt.xlabel('sma')
        plt.ylabel('PA(º)')
        plt.show()
        
        
        Source_names.append(PA_names[source])
        best_pas.append(best_init_pa)
        best_epss.append(best_init_eps)
        best_smas.append(best_init_sma)
        pa_differences.append(pa_difference)
        ellipse_coords_x.append(ellipses[index_contour][0])
        ellipse_coords_y.append(ellipses[index_contour][1])
        images_zoom.append(image_zoom)
        model_images.append(model_image)
        residualss.append(residuals)
        a.append(sMa)
        b.append(sma)
        e.append(eps)
        b_divided_a.append(sma_divided_sMa)
        pa_abs_difference_minor_axis.append(pa_minor_difference)
        pa_major.append(calculated_pa)
        pa_major_err.append(calculated_pa_err)
        major_axis_profiles.append(major_axis_profile)
        luminosities_profile.append(luminosity_profile)
        luminosities_profile_err.append(luminosity_profile_err)
        ellipse_total_luminosities.append(total_luminosity_inside_ellipse)
        indexes_contour.append(index_contour)
        
    except Exception:
        print(PA_names[source])
        print('Fitting failed')
        print('PA from the paper is',PA_values[source])
        Source_names.append(PA_names[source])
        pa_differences.append(np.nan)
        best_pas.append(np.nan)
        best_epss.append(np.nan)
        best_smas.append(np.nan)
        ellipse_coords_x.append(np.nan)
        ellipse_coords_y.append(np.nan)
        images_zoom.append(np.nan)
        model_images.append(np.nan)
        residualss.append(np.nan)
        a.append(np.nan)
        b.append(np.nan)
        e.append(np.nan)
        b_divided_a.append(np.nan)
        pa_abs_difference_minor_axis.append(np.nan)
        pa_major.append(np.nan)
        pa_major_err.append(np.nan)
        major_axis_profiles.append(np.nan)
        luminosities_profile.append(np.nan)
        luminosities_profile_err.append(np.nan)
        ellipse_total_luminosities.append(np.nan)
        indexes_contour.append(np.nan)
        pass
    
   

In [ ]:
optical = pd.read_csv('optical.csv')

In [ ]:
optical

In [ ]:
#pa_differences = optical['pa_differences']

#pa_differences_90 = list(abs(pa_differences - 90))
#pa_differences_270 = list(abs(pa_differences - 270))

#for i in range(len(pa_differences_90)):
    #if pa_differences_90[i] > 180:
        #pa_differences_90[i] = abs(360-pa_differences_90[i])

#for i in range(len(pa_differences_270)):
    #if pa_differences_270[i] > 180:
        #pa_differences_270[i] = abs(360-pa_differences_270[i]) 
        
#pa_abs_difference_minor_axis = []
#for pa_90,pa_270 in zip(pa_differences_90,pa_differences_270):
        #pa_abs_difference_minor_axis.append(min(pa_90,pa_270))
        
#optical['pa_abs_difference_minor_axis'] = pa_abs_difference_minor_axis

In [ ]:
print(len(pa_differences),len(best_pas),len(best_epss),len(best_smas),len(ellipse_coords_x),len(ellipse_coords_y),len(images_zoom),len(model_images),len(residualss),len(a),len(b),len(e),len(b_divided_a),len(pa_abs_difference_minor_axis),len(pa_major),len(pa_major_err),len(major_axis_profiles),len(luminosities_profile),len(luminosities_profile_err),len(ellipse_total_luminosities),len(indexes_contour)) 

In [ ]:
xtra = {'Source': Source_names, 'pa_differences': pa_differences, 'best_pas': best_pas, 'best_epss': best_epss, 'best_smas': best_smas, 'ellipse_coords_x': ellipse_coords_x, 'ellipse_coords_y': ellipse_coords_y, 'images_zoom': images_zoom, 'model_images': model_images, 'residualss': residualss, 'a': a, 'b': b, 'e': e, 'b_divided_a': b_divided_a, 'pa_abs_difference_minor_axis': pa_abs_difference_minor_axis, 'pa_major': pa_major, 'pa_major_err': pa_major_err, 'major_axis_profiles': major_axis_profiles, 'luminosities_profile': luminosities_profile, 'luminosities_profile_err': luminosities_profile_err, 'ellipse_total_luminosities': ellipse_total_luminosities, 'indexes_contour': indexes_contour}
optical = optical.append(pd.DataFrame(xtra),ignore_index=True)

In [ ]:
optical.to_csv('optical.csv',index=False)

galfit: save this fits file as spf.fits. The best would be to get the spf of every SDSS frame and input it to galfit.

In [ ]:
x = np.arange(0, 50, 1) #image.shape[1]   #Stars at 0, increases by 1, goes to length of axis
y = np.arange(0, 50, 1) #image.shape[0] #Stars at 0, increases by 1, goes to length of axis
xx, yy = np.meshgrid(x, y) #create
xycoords = (xx, yy)
psf = twoD_Gaussian(xycoords, 1, 25, 25, 1.24, 1.24, 0, 0)
psf = psf.reshape(50,50)
plt.figure()
plt.imshow(psf)
plt.show()

In [ ]:
Signal_to_noise_ratio = []


for source in range(len(Source_coords_csv['name'])):   #len(Source_coords_csv['name'])
    print(source,end='\r', flush=True)
    try:
        pos = coords.SkyCoord(Source_coords_RA[source],Source_coords_DEC[source], unit=(u.hourangle, u.deg), frame='icrs')
        im = SkyView.get_images(pos, survey='SDSSr', grid=True, gridlabels=True,  radius=45*u.arcsec)
        #si lo metes con pixels, luego la imagen no hay que invertirla
        image = im[0][0].data.copy()
        galaxy_center = np.where(image == np.amax(image))
        galaxy_center_x = galaxy_center[1][0]
        galaxy_center_y = galaxy_center[0][0]

        n = 40
        image_copy = image.copy()
        image_zoom = image_copy[galaxy_center_y-n:galaxy_center_y+n,galaxy_center_x-n:galaxy_center_x+n]
    except Exception:
        #xtra = {'Name':[PA_names[source]],'pa_major_gaussian':[np.nan],'pa_minor_gaussian':[np.nan],'jet_minor_angle':[np.nan],'pa_gaussian_err':[np.nan],'sigma_y_sigma_x_gaussian':[np.nan],'sigma_y_sigma_x_gaussian_err':[np.nan],'b_a_gaussian':[np.nan],'b_a_gaussian_err':[np.nan],'e_gaussian':[np.nan],'e_gaussian_err':[np.nan],'Good Case':[0]}
        #df['Signal_to_noise_ratio'][source] = np.nan
        #df = df.append(pd.DataFrame(xtra),ignore_index=True)
        #df.to_csv('Gaussian.csv',index=False)
        #print(PA_names[source])
        #print('Outside of SDSS')
        Signal_to_noise_ratio.append(np.nan)
        continue

    try:
        params,pcov = fit(image_zoom)
    except Exception:
        #xtra = {'Name':[PA_names[source]],'pa_major_gaussian':[np.nan],'pa_minor_gaussian':[np.nan],'jet_minor_angle':[np.nan],'pa_gaussian_err':[np.nan],'sigma_y_sigma_x_gaussian':[np.nan],'sigma_y_sigma_x_gaussian_err':[np.nan],'b_a_gaussian':[np.nan],'b_a_gaussian_err':[np.nan],'e_gaussian':[np.nan],'e_gaussian_err':[np.nan],'Good Case':[0]}
        #df['Signal_to_noise_ratio'][source] = np.nan
        #df = df.append(pd.DataFrame(xtra),ignore_index=True)
        #df.to_csv('Gaussian.csv',index=False)
        #print(PA_names[source])
        #print('Fitting failed')
        Signal_to_noise_ratio.append(np.nan)
        continue
    try:
        x = np.arange(0, image_zoom.shape[1], 1) #Stars at 0, increases by 1, goes to length of axis
        y = np.arange(0, image_zoom.shape[0], 1) #Stars at 0, increases by 1, goes to length of axis
        xx, yy = np.meshgrid(x, y) #creates a grid to plot the 
        xycoords = (xx, yy)
        amplitude,x0,y0,sigma_x,sigma_y,theta,offset = params
        theta_err = 180*pcov[5,5]**0.5/np.pi
        sigma_x_err = pcov[3,3]**0.5
        sigma_y_err = pcov[4,4]**0.5
        sigma_y_sigma_x_err = ((-sigma_y*sigma_x_err/sigma_x**2)**2 + (sigma_y_err/sigma_x)**2)**0.5
        if sigma_x > sigma_y:
            a = sigma_x
            a_err = sigma_x_err
            b = sigma_y
            b_err = sigma_y_err
        else:
            a = sigma_y
            a_err = sigma_y_err
            b = sigma_x
            b_err = sigma_x_err
        b_a = b/a
        b_a_err = ((-b*a_err/a**2)**2 + (b_err/a)**2)**0.5
        e = (1-b_a**2)**0.5
        e_err = -b_a*b_a_err/e
        
        vector = twoD_Gaussian(xycoords, amplitude, x0, y0, sigma_x, sigma_y, theta, offset)
        matrix = vector.reshape(image_zoom.shape[0],image_zoom.shape[1])

        
        
        #print('amplitude, x0, y0, sigma_x, sigma_y, theta, offset')
        #print(params)
        theta_from_positive_y = 180*(-theta)/np.pi - 90
        if sigma_y > sigma_x:
            theta_minor = theta_from_positive_y
            theta_major = theta_from_positive_y - 90
        else:
            theta_minor = theta_from_positive_y - 90
            theta_major = theta_from_positive_y
        
        k_minor = int(abs(theta_minor)/360) 
        theta_minor = theta_minor + 360*k_minor
        if 360 >= abs(theta_minor) >= 180:
            theta_minor = 360 - abs(theta_minor)
            
        k_major = int(abs(theta_major)/360) 
        theta_major = theta_major + 360*k_major
        if 360 >= abs(theta_major) >= 180:
            theta_major = 360 - abs(theta_major)    
        minor_jet_angle = abs(PA_values[source] - theta_minor)
        if minor_jet_angle > 90:
            minor_jet_angle = 180 - minor_jet_angle
        
        average_noise = abs(np.mean(image_zoom[0:10,0:10]))
        Signal_amp = offset + amplitude
        signal_noise = Signal_amp/average_noise
        #if signal_noise < 15:
            #pa_major_gaussian.append(np.nan)
            #pa_minor_gaussian.append(np.nan)
            #jet_minor_angle.append(np.nan)
            #pa_gaussian_err.append(np.nan)
            #sigma_y_sigma_x_gaussian.append(np.nan)
            #sigma_y_sigma_x_gaussian_err.append(np.nan)
            #b_a_gaussian.append(np.nan)
            #b_a_gaussian_err.append(np.nan)
            #e_gaussian.append(np.nan)
            #e_gaussian_err.append(np.nan)
            
            #print(PA_names[source])
            #print('Low S/N')
            #continue
        
        l = b*3
        #minor
        xf1 = x0 + l*np.cos((theta_minor+90)*np.pi/180)
        yf1 = y0 + l*np.sin((theta_minor+90)*np.pi/180)
        xf2 = x0 + l*np.cos((theta_minor-90)*np.pi/180)
        yf2 = y0 + l*np.sin((theta_minor-90)*np.pi/180)
       
        l = a*3
        #major
        xf3 = x0 + l*np.cos((theta_major+90)*np.pi/180)
        yf3 = y0 + l*np.sin((theta_major+90)*np.pi/180)
        xf4 = x0 + l*np.cos((theta_major-90)*np.pi/180)
        yf4 = y0 + l*np.sin((theta_major-90)*np.pi/180)
        
        l = 15
        #Jet
        PA_jet_angle_from_x = (PA_values[source]+90)*np.pi/180
        xf_jet = x0 + l*np.cos(PA_jet_angle_from_x)
        yf_jet = y0 + l*np.sin(PA_jet_angle_from_x)
        
    
        if b_a > 0.7 or signal_noise < 50:
            #xtra = {'Name':[PA_names[source]],'pa_major_gaussian':[theta_major],'pa_minor_gaussian':[theta_minor],'jet_minor_angle':[minor_jet_angle],'pa_gaussian_err':[theta_err],'sigma_y_sigma_x_gaussian':[sigma_y/sigma_x],'sigma_y_sigma_x_gaussian_err':[sigma_y_sigma_x_err],'b_a_gaussian':[b_a],'b_a_gaussian_err':[b_a_err],'e_gaussian':[e],'e_gaussian_err':[e_err],'Good Case':[0]}
            #df['Signal_to_noise_ratio'][source] = signal_noise
            #df = df.append(pd.DataFrame(xtra),ignore_index=True)
            #df.to_csv('Gaussian.csv',index=False)
            Signal_to_noise_ratio.append(signal_noise)
            continue
            
        #xtra = {'Name':[PA_names[source]],'pa_major_gaussian':[theta_major],'pa_minor_gaussian':[theta_minor],'jet_minor_angle':[minor_jet_angle],'pa_gaussian_err':[theta_err],'sigma_y_sigma_x_gaussian':[sigma_y/sigma_x],'sigma_y_sigma_x_gaussian_err':[sigma_y_sigma_x_err],'b_a_gaussian':[b_a],'b_a_gaussian_err':[b_a_err],'e_gaussian':[e],'e_gaussian_err':[e_err],'Good Case':[1]}
        #df['Signal_to_noise_ratio'] = signal_noise
        Signal_to_noise_ratio.append(signal_noise)
        #df = df.append(pd.DataFrame(xtra),ignore_index=True)
        #df.to_csv('Gaussian.csv',index=False)
        print(PA_names[source])
        #print('Average_Noise =',round(average_noise,2))
        #print('Signal amplitude =',round(Signal_amp,2))
        print('S/N =',signal_noise)
        print('Jet PA = ',PA_values[source],'deg (from +y counterclock.)')
        print('Minor PA =',round(theta_minor,2),'+/-',round(theta_err,2),'deg (from +y counterclock.)')
        print('Angle between jet and minor ax. =',round(minor_jet_angle,2),'+/-',round(theta_err,2),'deg')
        print('b/a =',round(b/a,2),'+/-',round(b_a_err,2))
        print('e =',round(e,2),'+/-',round(e_err,2))
        
        
        plt.figure()
        plt.imshow(image_zoom)
        plt.colorbar()
        plt.contour(xx, yy, matrix, [0.05*np.amax(matrix),np.amax(matrix)], colors='w')
        plt.plot([x0,xf1],[y0,yf1],'w-')
        plt.plot([x0,xf2],[y0,yf2],'w-')
        plt.plot([x0,xf3],[y0,yf3],'w-')
        plt.plot([x0,xf4],[y0,yf4],'w-',label='axis')
        plt.plot([x0,xf_jet],[y0,yf_jet],'r-',label='Jet')
        plt.plot(xf1,yf1,'ro',label='Minor ax. PA')
        plt.gca().invert_yaxis()
        plt.legend()
        plt.title('Original image')
        plt.show()

        
        plt.figure()
        plt.imshow(matrix)
        plt.colorbar()
        plt.contour(xx, yy, matrix, [0.05*np.amax(matrix),np.amax(matrix)], colors='w')
        plt.plot([x0,xf1],[y0,yf1],'w-')
        plt.plot([x0,xf2],[y0,yf2],'w-')
        plt.plot([x0,xf3],[y0,yf3],'w-')
        plt.plot([x0,xf4],[y0,yf4],'w-',label='axis')
        plt.plot([x0,xf_jet],[y0,yf_jet],'r-',label='Jet')
        plt.plot(xf1,yf1,'ro',label='Minor ax. PA')
        plt.gca().invert_yaxis()
        plt.legend()
        plt.title('Gaussian fit')
        plt.show()
        
        
        plt.figure()
        plt.imshow(image_zoom-matrix)
        plt.colorbar()
        plt.gca().invert_yaxis()
        plt.title('Residuals')
        plt.show()
        
    except Exception as e:
        #xtra = {'Name':[PA_names[source]],'pa_major_gaussian':[np.nan],'pa_minor_gaussian':[np.nan],'jet_minor_angle':[np.nan],'pa_gaussian_err':[np.nan],'sigma_y_sigma_x_gaussian':[np.nan],'sigma_y_sigma_x_gaussian_err':[np.nan],'b_a_gaussian':[np.nan],'b_a_gaussian_err':[np.nan],'e_gaussian':[np.nan],'e_gaussian_err':[np.nan],'Good Case':[0]}
        #df['Signal_to_noise_ratio'][source] = np.nan
        #df = df.append(pd.DataFrame(xtra),ignore_index=True)
        #df.to_csv('Gaussian.csv',index=False)
        #print(PA_names[source])
        #print(e)
        Signal_to_noise_ratio.append(np.nan)
        continue


In [ ]:
Gaussian = pd.read_csv('Gaussian.csv')
Gaussian

In [ ]:
pa_major_err = Gaussian['pa_gaussian_err']
e = Gaussian['e_gaussian']

plt.figure()
plt.plot(e,pa_major_err,'b.')
plt.xlabel('e')
plt.ylabel('pa_minor_err')
plt.ylim([0,100])
plt.show()

In [ ]:
pa_abs_difference_minor_axis = np.array((abs(Gaussian['jet_minor_angle'])))
pa_abs_difference_minor_axis[pa_abs_difference_minor_axis > 90] = 180 - pa_abs_difference_minor_axis[pa_abs_difference_minor_axis > 90]
pa_abs_difference_minor_axis = np.array(pa_abs_difference_minor_axis)
ell = list(Gaussian['e_gaussian'])
S_N = list(Gaussian['Signal_to_noise_ratio'])
high_e_index = []
low_e_index = []
e_filter_1 = 0.8
e_filter_2 = 0.6
s_n_filter = 10
for i,(e,s_n) in enumerate(zip(ell,S_N)):
    if (e > e_filter_1) and (s_n > s_n_filter):
        high_e_index.append(i)
    if (e > e_filter_2) and (s_n > s_n_filter):
        low_e_index.append(i)
high_e_index = np.array(high_e_index)
low_e_index = np.array(low_e_index)

pa_abs_difference_minor_axis_high_e = pa_abs_difference_minor_axis[high_e_index]
pa_abs_difference_minor_axis_low_e = pa_abs_difference_minor_axis[low_e_index]
colors = [0]*len(pa_abs_difference_minor_axis)
colors = np.array(colors)
#colors[high_e_index] = 'blue'
#colors[low_e_index] = 'red'

plt.figure()
#plt.hist(pa_abs_difference_minor_axis,bins=20,color=colors)
#plt.hist(pa_abs_difference_minor_axis_low_e,bins=10,color='red',label='e < 0.7',alpha=0.7)
#plt.hist(pa_abs_difference_minor_axis_low_e,bins=2,color='red',label='e > {}'.format(e_filter_2),alpha=1)
plt.hist(pa_abs_difference_minor_axis,bins=2,color='blue',label='All'.format(e_filter_1,s_n_filter),alpha=1)
plt.xlabel('Angle difference bewteen jet and minor axis (º)')
plt.ylabel('Number of sources')
plt.legend()
plt.show()


plt.figure()
#plt.hist(pa_abs_difference_minor_axis,bins=20,color=colors)
#plt.hist(pa_abs_difference_minor_axis_low_e,bins=10,color='red',label='e < 0.7',alpha=0.7)
plt.hist(pa_abs_difference_minor_axis_low_e,bins=2,color='red',label='e > {}, S/N > {}'.format(e_filter_2,s_n_filter),alpha=1)
#plt.hist(pa_abs_difference_minor_axis_high_e,bins=2,color='blue',label='e > {}'.format(e_filter_1),alpha=1)
plt.xlabel('Angle difference bewteen jet and minor axis (º)')
plt.ylabel('Number of sources')
plt.legend()
plt.show()


plt.figure()
#plt.hist(pa_abs_difference_minor_axis,bins=20,color=colors)
#plt.hist(pa_abs_difference_minor_axis_low_e,bins=10,color='red',label='e < 0.7',alpha=0.7)
#plt.hist(pa_abs_difference_minor_axis_low_e,bins=2,color='red',label='e > {}'.format(e_filter_2),alpha=1)
plt.hist(pa_abs_difference_minor_axis_high_e,bins=2,color='blue',label='e > {} and S/N > {}'.format(e_filter_1,s_n_filter),alpha=1)
plt.xlabel('Angle difference bewteen jet and minor axis (º)')
plt.ylabel('Number of sources')
plt.legend()
plt.show()

pyl.figure()
y,binEdges = np.histogram(pa_abs_difference_minor_axis_high_e,bins=2)
bincenters = 0.5*(binEdges[1:]+binEdges[:-1])
menStd     = np.sqrt(y)
width      = 45
pyl.bar(bincenters, y, width=width, color='r', yerr=menStd)
pyl.xlabel('Angle difference bewteen jet and minor axis (º)')
pyl.ylabel('Number of sources')
plt.show()



In [ ]:
100*y[0]/(y[0]+y[1])

In [ ]:
Source_coords_csv

In [ ]:
source = np.where(Source_coords_csv['name'] == 'J0843+4537')[0][0]
real_vlbi_name = glob.glob('images\\'+PA_names[source]+'\\J*map.fits')
if len(real_vlbi_name) > 0:
    real_vlbi_name = real_vlbi_name[-1]
else:
    print(PA_names[source])
    print('Source is a variation of some other source with a letter, fits not in VLBI')
    PAS.append(np.NAN)
    

data = fits.open(real_vlbi_name)
image = data[0].data
headers = data[0].header
image = image[0][0]
PA_vlbi = headers['BPA']
pixscale_arcsec = data[0].header['CDELT2']*360
galaxy_center = np.where(image[200:301,200:301] == np.amax(image[200:301,200:301]))
galaxy_center_x = np.arange(200,301)[galaxy_center[1][0]]
galaxy_center_y = np.arange(200,301)[galaxy_center[0][0]]

image_copy = image.copy()
zoom_image = image_copy[int(galaxy_center_y)-n:int(galaxy_center_y)+n,int(galaxy_center_x)-n:int(galaxy_center_x)+n]
masked_zoom_image = zoom_image.copy()
masked_zoom_image[abs(masked_zoom_image) > 1e-3] = 10*masked_zoom_image[abs(masked_zoom_image) > 1e-3]
positions_above_zero = np.where(masked_zoom_image > 1e-3)
masked_zoom_image[positions_above_zero] = np.log10(masked_zoom_image[positions_above_zero])

x0 = n
y0 = n

l = 15
#Jet
PA_jet_angle_from_x = (PA_values[source]+90)*np.pi/180
xf_jet = x0 + l*np.cos(PA_jet_angle_from_x)
yf_jet = y0 + l*np.sin(PA_jet_angle_from_x)

RA_pix_deg = abs(headers['CDELT1'])
DEC_pix_deg = abs(headers['CDELT2'])
RA_pix_mas = RA_pix_deg*3600000
DEC_pix_mas = DEC_pix_deg*3600000

half_side_length_mas = int(round(masked_zoom_image.shape[0]*RA_pix_mas/2))
xtick_pos = [0,int(round(masked_zoom_image.shape[0]/2)),int(masked_zoom_image.shape[0])]
xtick_labels = [-half_side_length_mas,0,+half_side_length_mas]
ytick_pos = [0,int(round(masked_zoom_image.shape[0]/2)),int(masked_zoom_image.shape[0])]
ytick_labels = [-half_side_length_mas,0,+half_side_length_mas]


plt.figure() 
plt.imshow(masked_zoom_image)
plt.plot([x0,xf_jet],[y0,yf_jet],'r-',label='Jet')
ax = plt.gca()
ax.invert_yaxis()
ax.set_xticks(xtick_pos)
ax.set_xticklabels(xtick_labels)
ax.set_yticks(xtick_pos)
ax.set_yticklabels(xtick_labels)
plt.xlabel('RA (mas)')
plt.ylabel('DEC (mas)')
plt.title(PA_names[source]+' VLBI')
plt.legend()
plt.savefig('VLBI.png',dpi=1000)
plt.show()

names = np.array(Source_coords_csv['name'])
index = np.where(names == 'J0843+4537')[0][0]


pos = coords.SkyCoord(Source_coords_RA[index],Source_coords_DEC[index], unit=(u.hourangle, u.deg), frame='icrs')
im = SkyView.get_images(pos, survey='SDSSr', grid=True, gridlabels=True,  radius=45*u.arcsec)
#si lo metes con pixels, luego la imagen no hay que invertirla
image = im[0][0].data.copy()
headers = im[0][0].header
galaxy_center = np.where(image == np.amax(image))
galaxy_center_x = galaxy_center[1][0]
galaxy_center_y = galaxy_center[0][0]
n = 40
image_copy = image.copy()
image_zoom = image_copy[galaxy_center_y-n:galaxy_center_y+n,galaxy_center_x-n:galaxy_center_x+n]

half_side_length_arcsec = (image_zoom.shape[0]/image.shape[0])*45
#print('half side is',half_side_length_arcsec,'arcsec')

xtick_pos = [0,int(round(image_zoom.shape[0]/2)),int(image_zoom.shape[0])]
xtick_labels = [-half_side_length_arcsec,0,+half_side_length_arcsec]
ytick_pos = [0,int(round(image_zoom.shape[0]/2)),int(image_zoom.shape[0])]
ytick_labels = [-half_side_length_arcsec,0,+half_side_length_arcsec]

plt.figure()
plt.imshow(image_zoom)
plt.title(Source_coords_csv['name'][3456]+' Optical')
ax = plt.gca()
ax.invert_yaxis()
ax.set_xticks(xtick_pos)
ax.set_xticklabels(xtick_labels)
ax.set_yticks(ytick_pos)
ax.set_yticklabels(ytick_labels)
plt.xlabel('RA (arcsec)')
plt.ylabel('DEC (arcsec)')
plt.savefig('Optical.png',dpi=1000)
plt.show()


params,pcov = fit(image_zoom)


x = np.arange(0, image_zoom.shape[1], 1) #Stars at 0, increases by 1, goes to length of axis
y = np.arange(0, image_zoom.shape[0], 1) #Stars at 0, increases by 1, goes to length of axis
xx, yy = np.meshgrid(x, y) #creates a grid to plot the 
xycoords = (xx, yy)
amplitude,x0,y0,sigma_x,sigma_y,theta,offset = params
theta_err = 180*pcov[5,5]**0.5/np.pi
sigma_x_err = pcov[3,3]**0.5
sigma_y_err = pcov[4,4]**0.5
sigma_y_sigma_x_err = ((-sigma_y*sigma_x_err/sigma_x**2)**2 + (sigma_y_err/sigma_x)**2)**0.5
if sigma_x > sigma_y:
    a = sigma_x
    a_err = sigma_x_err
    b = sigma_y
    b_err = sigma_y_err
else:
    a = sigma_y
    a_err = sigma_y_err
    b = sigma_x
    b_err = sigma_x_err
b_a = b/a
b_a_err = ((-b*a_err/a**2)**2 + (b_err/a)**2)**0.5
e = (1-b_a**2)**0.5
e_err = -b_a*b_a_err/e

vector = twoD_Gaussian(xycoords, amplitude, x0, y0, sigma_x, sigma_y, theta, offset)
matrix = vector.reshape(image_zoom.shape[0],image_zoom.shape[1])



#print('amplitude, x0, y0, sigma_x, sigma_y, theta, offset')
#print(params)
theta_from_positive_y = 180*(-theta)/np.pi - 90
if sigma_y > sigma_x:
    theta_minor = theta_from_positive_y
    theta_major = theta_from_positive_y - 90
else:
    theta_minor = theta_from_positive_y - 90
    theta_major = theta_from_positive_y

k_minor = int(abs(theta_minor)/360) 
theta_minor = theta_minor + 360*k_minor
if 360 >= abs(theta_minor) >= 180:
    theta_minor = 360 - abs(theta_minor)

k_major = int(abs(theta_major)/360) 
theta_major = theta_major + 360*k_major
if 360 >= abs(theta_major) >= 180:
    theta_major = 360 - abs(theta_major)    
minor_jet_angle = abs(PA_values[source] - theta_minor)
if minor_jet_angle > 90:
    minor_jet_angle = 180 - minor_jet_angle

average_noise = abs(np.mean(image_zoom[0:10,0:10]))
Signal_amp = offset + amplitude
signal_noise = Signal_amp/average_noise


l = b*3
#minor
xf1 = x0 + l*np.cos((theta_minor+90)*np.pi/180)
yf1 = y0 + l*np.sin((theta_minor+90)*np.pi/180)
xf2 = x0 + l*np.cos((theta_minor-90)*np.pi/180)
yf2 = y0 + l*np.sin((theta_minor-90)*np.pi/180)

l = a*3
#major
xf3 = x0 + l*np.cos((theta_major+90)*np.pi/180)
yf3 = y0 + l*np.sin((theta_major+90)*np.pi/180)
xf4 = x0 + l*np.cos((theta_major-90)*np.pi/180)
yf4 = y0 + l*np.sin((theta_major-90)*np.pi/180)

l = 15
#Jet
PA_jet_angle_from_x = (PA_values[source]+90)*np.pi/180
xf_jet = x0 + l*np.cos(PA_jet_angle_from_x)
yf_jet = y0 + l*np.sin(PA_jet_angle_from_x)




plt.figure()
plt.imshow(image_zoom)
#plt.colorbar()
plt.contour(xx, yy, matrix, [0.10*np.amax(matrix),np.amax(matrix)], colors='w')
plt.plot([x0,xf1],[y0,yf1],'w-')
plt.plot([x0,xf2],[y0,yf2],'w-')
plt.plot([x0,xf3],[y0,yf3],'w-')
plt.plot([x0,xf4],[y0,yf4],'w-',label='axis')
#plt.plot([x0,xf_jet],[y0,yf_jet],'r-',label='Jet')
plt.plot(xf1,yf1,'ro',label='Minor ax. PA')
ax = plt.gca()
ax.invert_yaxis()
ax.set_xticks(xtick_pos)
ax.set_xticklabels(xtick_labels)
ax.set_yticks(ytick_pos)
ax.set_yticklabels(ytick_labels)
plt.xlabel('RA (arcsec)')
plt.ylabel('DEC (arcsec)')
plt.legend()
plt.title(PA_names[source]+' Gaussian fit')
plt.savefig('Fit.png',dpi=1000)
plt.show()


#plt.figure()
#plt.imshow(matrix)
#plt.colorbar()
#plt.contour(xx, yy, matrix, [0.05*np.amax(matrix),np.amax(matrix)], colors='w')
#plt.plot([x0,xf1],[y0,yf1],'w-')
#plt.plot([x0,xf2],[y0,yf2],'w-')
#plt.plot([x0,xf3],[y0,yf3],'w-')
#plt.plot([x0,xf4],[y0,yf4],'w-',label='axis')
#plt.plot([x0,xf_jet],[y0,yf_jet],'r-',label='Jet')
#plt.plot(xf1,yf1,'ro',label='Minor ax. PA')
#plt.gca().invert_yaxis()
#plt.legend()
#plt.title('Gaussian fit')
#plt.show()


plt.figure()
plt.imshow(image_zoom)
#plt.colorbar()
plt.contour(xx, yy, matrix, [0.10*np.amax(matrix),np.amax(matrix)], colors='w')
plt.plot([x0,xf1],[y0,yf1],'w-')
plt.plot([x0,xf2],[y0,yf2],'w-')
plt.plot([x0,xf3],[y0,yf3],'w-')
plt.plot([x0,xf4],[y0,yf4],'w-',label='axis')
plt.plot([x0,xf_jet],[y0,yf_jet],'r-',label='Jet')
plt.plot(xf1,yf1,'ro',label='Minor ax. PA')
ax = plt.gca()
ax.invert_yaxis()
ax.set_xticks(xtick_pos)
ax.set_xticklabels(xtick_labels)
ax.set_yticks(ytick_pos)
ax.set_yticklabels(ytick_labels)
plt.xlabel('RA (arcsec)')
plt.ylabel('DEC (arcsec)')
plt.legend()
ax.set_title(PA_names[source]+' Angle between jet and minor axis',pad=20)
plt.savefig('Angle.png',dpi=1000)
plt.show()

In [ ]:
stats.ttest_ind()

In [ ]:
good_indexes = []
for i in range(len(Gaussian['Good Case'])):
    if Gaussian['Good Case'][i] == 1:
        good_indexes.append(i)
good_indexes = np.array(good_indexes)

pa_abs_difference_minor_axis = np.array((abs(Gaussian['jet_minor_angle'])))
pa_abs_difference_minor_axis[pa_abs_difference_minor_axis > 90] = 180 - pa_abs_difference_minor_axis[pa_abs_difference_minor_axis > 90]
pa_abs_difference_minor_axis = np.array(pa_abs_difference_minor_axis)
pa_abs_difference_minor_axis_good_case = pa_abs_difference_minor_axis[good_indexes]




colors = [0]*len(pa_abs_difference_minor_axis)
colors = np.array(colors)
#colors[high_e_index] = 'blue'
#colors[low_e_index] = 'red'
plt.figure()
#plt.hist(pa_abs_difference_minor_axis,bins=20,color=colors)
plt.hist(pa_abs_difference_minor_axis_good_case,bins=10,color='red',label='S/N > 50 and b/a < 0.7',alpha=0.7)
#plt.hist(pa_abs_difference_minor_axis,bins=10,color='blue',label='S/N < 50 or b/a < 0.7',alpha=0.7)
plt.xlabel('Angle difference bewteen jet and minor axis (º)')
plt.legend()
plt.show()

In [ ]:
good_indexes = []
for i in range(len(Gaussian['Name'])):
    if Gaussian['signal_noise'][i] == 1:
        good_indexes.append(i)
good_indexes = np.array(good_indexes)

pa_abs_difference_minor_axis = np.array((abs(Gaussian['jet_minor_angle'])))
pa_abs_difference_minor_axis[pa_abs_difference_minor_axis > 90] = 180 - pa_abs_difference_minor_axis[pa_abs_difference_minor_axis > 90]
pa_abs_difference_minor_axis = np.array(pa_abs_difference_minor_axis)
pa_abs_difference_minor_axis_good_case = pa_abs_difference_minor_axis[good_indexes]




colors = [0]*len(pa_abs_difference_minor_axis)
colors = np.array(colors)
#colors[high_e_index] = 'blue'
#colors[low_e_index] = 'red'
plt.figure()
#plt.hist(pa_abs_difference_minor_axis,bins=20,color=colors)
plt.hist(pa_abs_difference_minor_axis_good_case,bins=10,color='red',label='S/N > 50 and b/a < 0.7',alpha=0.7)
#plt.hist(pa_abs_difference_minor_axis,bins=10,color='blue',label='S/N < 50 or b/a < 0.7',alpha=0.7)
plt.xlabel('Angle difference bewteen jet and minor axis (º)')
plt.legend()
plt.show()

In [ ]:
# Create x and y indices
x = np.linspace(0, 200, 201)
y = np.linspace(0, 200, 201)
x, y = np.meshgrid(x, y)

#create data
data = twoD_Gaussian((x, y), 3, 100, 100, 20, 40,np.pi/2, 10)

# plot twoD_Gaussian data generated above
plt.figure()
plt.imshow(data.reshape(201, 201))
plt.colorbar()
plt.show()

Para comprobar si los residuos son Gaussianos o no, quizás hacer un histograma de ellos? Ver si están centrados en el cero y si solo tienen una desviación típica. Si no es así, eso muestra una estructura en los residuos.

In [ ]:
#pa_major_err = []
#for i in range(105,len(optical['Source'])):
    #print(i)
    #if np.isnan(optical['pa_differences'][i]):
        #pa_major_err.append(np.nan)
    #else:
        #index = np.where(np.array(Source_coords_csv['name']) == optical['Source'][i])[0]
        #pos = coords.SkyCoord(Source_coords_RA[index],Source_coords_DEC[index], unit=(u.hourangle, u.deg), frame='icrs') 
        #im = SkyView.get_images(pos, survey='SDSSr', grid=True, gridlabels=True,  radius=45*u.arcsec)
        ##si lo metes con pixels, luego la imagen no hay que invertirla
        #image = im[0][0].data.copy()
        #galaxy_center = np.where(image == np.amax(image))
        #galaxy_center_x = galaxy_center[1][0]
        #galaxy_center_y = galaxy_center[0][0]

        #n = 40
        #image_copy = image.copy()
        #image_zoom = image_copy[galaxy_center_y-n:galaxy_center_y+n,galaxy_center_x-n:galaxy_center_x+n]
        #geometry = EllipseGeometry(x0=n, y0=n, sma=optical['best_smas'][i], eps=optical['best_epss'][i], pa=optical['best_pas'][i] * np.pi / 180.)
        #aper = EllipticalAperture((geometry.x0, geometry.y0), geometry.sma,  geometry.sma * (1 - geometry.eps), geometry.pa)
        #ellipse = Ellipse(image_zoom, geometry)
        #isolist = ellipse.fit_image()     

        #index_contour = np.where(isolist.intens/np.max(isolist.intens) < 0.15)[0][0]
        #calculated_pa_err = isolist.pa_err[index_contour]*180/np.pi
        #pa_major_err.append(calculated_pa_err)

In [ ]:
Source_coords_RA[9016]

In [ ]:
Source_coords_DEC[9016]

In [ ]:
np.where(Source_coords_csv['name'] == 'J2326+5257')[0]


In [ ]:
from scipy.stats import norm
import statistics

delta = 10
center_residuals = residuals[n-delta:n+delta,n-delta:n+delta]
mu, std = norm.fit(center_residuals)




plt.figure()
plt.imshow(center_residuals)
plt.colorbar()
plt.show()

blues = ['b']*center_residuals.shape[0]
plt.figure()
plt.hist(center_residuals.flatten(), density=True, color='blue')

xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
plt.xlabel('Residuals')
plt.ylabel('Counts')
title = "Fit Values: {:.2f} and {:.2f}".format(mu, std)
plt.title(title)
plt.show()

Find the sources with low redshift and try to fit them in optical.

In [ ]:
optical = pd.read_csv('optical.csv')
for i in range(len(optical['pa_differences'])):
    if np.isnan(optical['pa_differences'][i]):
        #print(optical['Source'][i])
        #print('Outside of SDSS footprint')
        continue
    else:
        index = np.where(np.array(Source_coords_csv['name']) == optical['Source'][i])[0]
        pos = coords.SkyCoord(Source_coords_RA[index],Source_coords_DEC[index], unit=(u.hourangle, u.deg), frame='icrs') 
        im = SkyView.get_images(pos, survey='SDSSr', grid=True, gridlabels=True,  radius=45*u.arcsec)
        #si lo metes con pixels, luego la imagen no hay que invertirla
        image = im[0][0].data.copy()
        galaxy_center = np.where(image == np.amax(image))
        galaxy_center_x = galaxy_center[1][0]
        galaxy_center_y = galaxy_center[0][0]

        n = 40
        image_copy = image.copy()
        image_zoom = image_copy[galaxy_center_y-n:galaxy_center_y+n,galaxy_center_x-n:galaxy_center_x+n]
        image_log = image_zoom.copy()
        image_log[image_log < 1e-2] = 0
        image_log[image_log > 1e-1] = np.log10(image_log[image_log > 1e-1])


        if isinstance(optical['ellipse_coords_x'][i], str):
            x = optical['ellipse_coords_x'][i]
            x = list(x.split(" "))
            x = [txt.replace("\r\n", '') for txt in x]
            x = [txt.replace("[", '') for txt in x]
            x = [txt.replace("]", '') for txt in x]
            while('' in x) :
                x.remove('')
            x = [float(element) for element in x]


            y = optical['ellipse_coords_y'][i]
            y = list(y.split(" "))
            y = [txt.replace("\r\n", '') for txt in y]
            y = [txt.replace("[", '') for txt in y]
            y = [txt.replace("]", '') for txt in y]
            while('' in y) :
                y.remove('')
            y = [float(element) for element in y]
        
        else:
            x = optical['ellipse_coords_x'][i]
            y = optical['ellipse_coords_y'][i]
            
            
        
        x_zero = x[0]
        x_half = x[round(len(x)/2)]
        y_zero = y[0]
        y_half = y[round(len(y)/2)]
        
        x0 = round((x_zero+x_half)/2)
        y0 = round((y_zero+y_half)/2)
        
        PA_jet_angle_from_x = (PA_values[index[0]]+90)*np.pi/180
        l = 10
        xf_jet = x0 + l*np.cos(PA_jet_angle_from_x)
        yf_jet = y0 + l*np.sin(PA_jet_angle_from_x)
        xf_jet = round(xf_jet)
        yf_jet = round(yf_jet)
        
        PA_major_angle_from_x = (optical['pa_major'][i]+90)*np.pi/180
        l = 20
        xf_major = x0 + l*np.cos(PA_major_angle_from_x)
        yf_major = y0 + l*np.sin(PA_major_angle_from_x)
        xf_major = round(xf_major)
        yf_major = round(yf_major)
        
        
        PA_minor_1_angle_from_x = (optical['pa_major'][i]+90-90)*np.pi/180
        l = 10
        xf_minor_1 = x0 + l*np.cos(PA_minor_1_angle_from_x)
        yf_minor_1 = y0 + l*np.sin(PA_minor_1_angle_from_x)
        xf_minor_1 = round(xf_minor_1)
        yf_minor_1 = round(yf_minor_1)
        
        PA_minor_2_angle_from_x = (optical['pa_major'][i]+90+90)*np.pi/180
        l = 10
        xf_minor_2 = x0 + l*np.cos(PA_minor_2_angle_from_x)
        yf_minor_2 = y0 + l*np.sin(PA_minor_2_angle_from_x)
        xf_minor_2 = round(xf_minor_2)
        yf_minor_2 = round(yf_minor_2)
            
        print(optical['Source'][i])
        print('Jet PA',PA_values[index[0]],'degrees')
        print('Major axis PA',optical['pa_major'][i])
        print('Major axis PA error',optical['pa_major_err'][i])
        print('Absolute angle difference between jet and major axis =',optical['pa_differences'][i],'degrees')
        print('Absolute angle difference between jet and minor axis =',optical['pa_abs_difference_minor_axis'][i],'degrees')
        print('Ellipticity =',optical['e'][i])
        plt.figure()
        plt.imshow(image_zoom)
        plt.plot(x,y,'w-')
        plt.plot([x0,xf_major],[y0,yf_major],'w-')
        plt.plot([x0,xf_minor_1],[y0,yf_minor_1],'w-')
        plt.plot([x0,xf_minor_2],[y0,yf_minor_2],'w-',label='axis')
        plt.plot([x0,xf_jet],[y0,yf_jet],'r-',label='jet')
        #plt.plot([minor_x1,minor_x2],[minor_y1,minor_y2],'w-')
        #colors = ['b','g','r','c','m','y']
        #labels = ['1','2','3','4','5','6']
        #for i in range(0,6):
            #if i == 5:
                #plt.scatter(x[int(len(x)*i/5)-1],y[int(len(y)*i/5)-1],c=colors[i],label=labels[i])  
            #else:
                #plt.scatter(x[int(len(x)*i/5)],y[int(len(y)*i/5)],c=colors[i],label=labels[i])
        plt.gca().invert_yaxis()
        plt.colorbar()
        plt.legend()
        plt.title('Original image')
        plt.show()


      

In [ ]:
pa_abs_difference_minor_axis = list(optical['pa_abs_difference_minor_axis'])
pa_abs_difference_minor_axis = np.array(pa_abs_difference_minor_axis)
e = optical['e']
high_e_index = []
low_e_index = []
for i in range(len(e)):
    if e[i] > 0.1:
        high_e_index.append(i)
    if e[i] <= 0.1:
        low_e_index.append(i)
high_e_index = np.array(high_e_index)
low_e_index = np.array(low_e_index)

pa_abs_difference_minor_axis_high_e = pa_abs_difference_minor_axis[high_e_index]
pa_abs_difference_minor_axis_low_e = pa_abs_difference_minor_axis[low_e_index]
colors = [0]*len(pa_abs_difference_minor_axis)
colors = np.array(colors)
#colors[high_e_index] = 'blue'
#colors[low_e_index] = 'red'
plt.figure()
#plt.hist(pa_abs_difference_minor_axis,bins=20,color=colors)
plt.hist(pa_abs_difference_minor_axis_low_e,bins=10,color='red',label='e < 0.1',alpha=0.7)
plt.hist(pa_abs_difference_minor_axis_high_e,bins=10,color='blue',label='e > 0.1',alpha=0.7)

plt.legend()
plt.show()

In [ ]:
#import scipy.optimize
#Source_coords_csv = pd.read_csv('Source_coords.csv')
#Source_coords_DEC = Source_coords_csv['dec']
#Source_coords_RA = Source_coords_csv['ra']
#for source in range(10):
    #if source == 6:
        #print(PA_names[source])
        #print('Skipped')
    #pos = coords.SkyCoord(Source_coords_RA[source],Source_coords_DEC[source], unit=(u.hourangle, u.deg), frame='icrs')
    #try:
        #im = SkyView.get_images(pos, survey='SDSSr', grid=True, gridlabels=True,  radius=45*u.arcsec)
    #except url_err.HTTPError:
        #print(PA_names[source])
        #print('Not in SDSS footprint')
        #print('PA from the paper is',PA_values[source])
        #continue

    #si lo metes con pixels, luego la imagen no hay que invertirla
    #image = im[0][0].data.copy()
    #galaxy_center = np.where(image == np.amax(image))
    #galaxy_center_x = galaxy_center[1][0]
    #galaxy_center_y = galaxy_center[0][0]

    #n = 40
    #image_copy = image.copy()
    #image_zoom = image_copy[galaxy_center_y-n:galaxy_center_y+n,galaxy_center_x-n:galaxy_center_x+n]
    #image_log = image_zoom.copy()
    #image_log[image_log < 1e-2] = 0
    #image_log[image_log > 1e-1] = np.log10(image_log[image_log > 1e-1])
    
    
    
    #args_ = (image_zoom,)
    #optimal_params = scipy.optimize.differential_evolution(func = f, bounds = ((5,20), (0.1,0.7), (10,170)), args = args_, strategy='best1bin', maxiter=600, popsize=15, tol=0.01, mutation=(0.5, 1), recombination=0.7, seed=None, callback=None, disp=False, polish=True, init='latinhypercube', atol=0, updating='immediate', workers=1, x0=None)
    #if optimal_params.success:
        #try:
            #optimal_sma = optimal_params.x[0]
            #optimal_eps = optimal_params.x[1]
            #optimal_pa = optimal_params.x[2]
            #print(PA_names[source])
            #print('optimal init. sma = ',optimal_sma,'  ','optimal init. eps = ',optimal_eps,'  ','optimal init. pa = ',optimal_pa)

            #geometry = EllipseGeometry(x0=n, y0=n, sma=optimal_sma, eps=optimal_eps, pa=optimal_pa * np.pi / 180.)
            #aper = EllipticalAperture((geometry.x0, geometry.y0), geometry.sma,  geometry.sma * (1 - geometry.eps), geometry.pa)
            #ellipse = Ellipse(image_zoom, geometry)
            #isolist = ellipse.fit_image()
            #model_image = build_ellipse_model(image_zoom.shape, isolist)

            #residuals = image_zoom - model_image

            #ellipses = []
            #for sma in isolist.sma[1:]:
                #iso = isolist.get_closest(sma)
                #ellipse = iso.sampled_coordinates()
                #ellipses.append(ellipse)

            #index_contour = np.where(isolist.intens/np.max(isolist.intens) < 0.15)[0][0]



            #print('Calculated PA of the sMa is',isolist.pa[index_contour]*180/np.pi-90,'degrees')
            #print('Calculated PA of the sma is',isolist.pa[index_contour]*180/np.pi-180,'degrees')
            #print('PA from the paper is',PA_values[source],'degrees')


            #plt.figure()
            #plt.imshow(image_zoom)
            #for i in range(len(isolist.sma[1:])):
                #plt.plot(ellipses[i][0],ellipses[i][1],'w-')
            #plt.plot(ellipses[index_contour][0],ellipses[index_contour][1],'w-')
            #plt.gca().invert_yaxis()
            #plt.colorbar()
            #plt.title('Original image')
            #plt.show()


            #plt.figure()
            #plt.imshow(model_image)
            #for i in range(len(isolist.sma[1:])):
                #plt.plot(ellipses[i][0],ellipses[i][1],'w-')
            #plt.plot(ellipses[index_contour][0],ellipses[index_contour][1],'w-')
            #plt.gca().invert_yaxis()
            #plt.colorbar()
            #plt.title('Model image')
            #plt.show()


            #plt.figure()
            #plt.imshow(residuals)
            #for i in range(len(isolist.sma[1:])):
                #plt.plot(ellipses[i][0],ellipses[i][1],'w-')
            #plt.plot(ellipses[index_contour][0],ellipses[index_contour][1],'w-')
            #plt.gca().invert_yaxis()
            #plt.colorbar()
            #plt.title('Residuals')
            #plt.show()


            #plt.figure()
            #plt.scatter(isolist.sma, isolist.intens)
            #plt.scatter(isolist.sma[index_contour], isolist.intens[index_contour],c='r')
            #plt.xlabel('sma')
            #plt.ylabel('Intensity')
            #plt.show()


            #plt.figure()
            #plt.scatter(isolist.sma, isolist.pa*180/np.pi)
            #plt.scatter(isolist.sma[index_contour], isolist.pa[index_contour]*180/np.pi,c='r')
            ##plt.xlabel('sma')
            #plt.ylabel('PA(º)')
            #plt.show()
        #except Exception:
            #print(PA_names[source])
            #print('Fitting failed')

    #else:
        #print(PA_names[source])
        #print(optimal_params.message)

In [ ]:
pa_major_err = optical['pa_major_err']
e = optical['e']

plt.figure()
plt.plot(e,pa_major_err,'b.')
plt.xlabel('e')
plt.ylabel('pa_major_err')
plt.ylim([0,100])
plt.show()

Do a plot with the percentages of cases within a range of 45 deg.
Check the gaussian fit for the isophotes.
Compare the footprints of SDSS and my sources.
Compare with the size of psf.
Check DES.
Obtain the mean and std dev of the angle of each bin.
Do the student test.
Plot the mean angle of the <45 bin for different ellipticity cut-off.
Check the psf with some space images.